<a href="https://www.kaggle.com/code/abrachan/parkinson-fog-prediction-torch-gru?scriptVersionId=135576565" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<br>
<br>

# **Reference**:
* `Kaggle Competition`: <a href="https://www.kaggle.com/competitions/tlvmc-parkinsons-freezing-gait-prediction/overview" style="text-decoration:none">Parkinson's Freezing of Gait Prediction</a>
* `Disscusion`: <a href="https://www.kaggle.com/competitions/tlvmc-parkinsons-freezing-gait-prediction/discussion/416057" style="text-decoration:none">2nd place solution</a>
* `Notebook`: <a href="https://www.kaggle.com/code/takoihiraokazu/cv-ensemble-sub-0607-1" style="text-decoration:none">Inference notebook</a>
* `Github`: **TakoiHirokazu** <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction" style="text-decoration:none">Kaggle-Parkinsons-Freezing-of-Gait-Prediction</a> (Training)

Thanks <a href="https://www.kaggle.com/takoihiraokazu" style="text-decoration:none">Takoi</a> and his/her teammates for their sharing solution and code!

<br>

* My own works: <a href="https://www.kaggle.com/code/abrachan/exploratory-data-analysis#Some-Conclusions" style="text-decoration:none">Exploratory Data Analysis</a>


<br>
<br>

**Note**: <br>
The checkpoints using in this notebook which lying in the `/kaggle/input/parkinson-fog-prediction` were obtained from the training stage using **CUDA**. So if you want to run the `【Inference】` by skipping the training stage in this notebook in the kaggle environment, please select the GPU accelerator. Otherwise it will throw exceptions.

`RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.`

<br>

------


<br>

# **Import Libraries**

In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import gc
import sys
import glob
import json
import pickle
import logging
from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm_nb
from contextlib import contextmanager

import time
import datetime

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL.Image as Image

from IPython.display import Video

from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, average_precision_score
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold, StratifiedGroupKFold

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import LayerNorm
from torch.nn import TransformerEncoder, TransformerDecoder

import torch.optim as optim
from torch.optim import lr_scheduler
from transformers import AdamW, get_linear_schedule_with_warmup     # pip install transformers

from torch.cuda import amp
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [2]:
# sys.path.append("../src/")
# from logger import setup_logger, LOGGER    # Abrachan: see my revise below: `def init_logger()`
# from util_tool import reduce_mem_usage

pd.set_option('display.max_columns', 300)

<br>
<br>

# **Config**

In [3]:
# When doing `inference`, set below as False so that we can run the whole notebook directly.
TRAIN_FLAG   = False # True
PREDICT_FLAG = False # True

<br>
<br>

# **Load Data**

In [4]:
root_data = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/'

train_defog = glob.glob(root_data + 'train/defog/**')
train_tdcsfog = glob.glob(root_data + 'train/tdcsfog/**')
train_notype = glob.glob(root_data + 'train/notype/**')

test_defog = glob.glob(root_data + 'test/defog/**')
test_tdcsfog = glob.glob(root_data + 'test/tdcsfog/**')

subjects = pd.read_csv(root_data + 'subjects.csv')
tasks = pd.read_csv(root_data + 'tasks.csv')
events = pd.read_csv(root_data + 'events.csv')

daily_metadata=pd.read_csv(root_data + 'daily_metadata.csv')
tdcsfog_metadata=pd.read_csv(root_data + 'tdcsfog_metadata.csv')
defog_metadata=pd.read_csv(root_data + 'defog_metadata.csv')

sub = pd.read_csv(root_data + 'sample_submission.csv')

For some basic Exploratory Data Analysis (EDA), refers to <a href="https://www.kaggle.com/code/abrachan/exploratory-data-analysis" style="text-decoration:none">my own work</a> if you are interested.

<br>
<br>
<br>

# **tdcsfog** 【Training】

<br>

<br>

## **Feature Engineering**

<br>

### fe001_tdcsfog_base_feature

<br>

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/fe/fe001_tdcsfog_base_feature.ipynb" style="text-decoration:none">fe001_tdcsfog_base_feature.ipynb</a>

In [5]:
fe = "001"
if not os.path.exists(f"./output/fe/fe{fe}"):
    os.makedirs(f"./output/fe/fe{fe}", exist_ok=True)
    os.makedirs(f"./output/fe/fe{fe}/save", exist_ok=True)

In [6]:
# TDCSFOG_META_PATH = "../data/tdcsfog_metadata.csv"
# TDCSFOG_FOLDER = "../data/train/tdcsfog/*.csv"

# meta = pd.read_csv(TDCSFOG_META_PATH)

In [7]:
sub_dict = {}
for n,i in enumerate(tdcsfog_metadata["Subject"].unique()):
    sub_dict[i] = n
sub_dict

{'4dc2f8': 0,
 'f62eec': 1,
 '231c3b': 2,
 'fa8764': 3,
 'c85fdf': 4,
 '31d269': 5,
 '07285e': 6,
 '02bc69': 7,
 '7eb666': 8,
 '220a17': 9,
 '54ee6e': 10,
 '242a3e': 11,
 'e9fc55': 12,
 '66341b': 13,
 '7fcee9': 14,
 'f686f0': 15,
 '312788': 16,
 '4ca9b3': 17,
 '2a39f8': 18,
 'd9312a': 19,
 '3b2b7a': 20,
 '4b39ac': 21,
 '93f49f': 22,
 'af82b2': 23,
 '251738': 24,
 '87174c': 25,
 '3b2403': 26,
 'a03db7': 27,
 'a80ae4': 28,
 '364459': 29,
 'bc3908': 30,
 '4ba1d3': 31,
 '301ada': 32,
 '51574c': 33,
 '8db7dd': 34,
 '69cc45': 35,
 '2d57c2': 36,
 'd8836b': 37,
 'eeaff0': 38,
 '6a3e93': 39,
 'f2c8aa': 40,
 '2c98f7': 41,
 'c8e721': 42,
 'e8919c': 43,
 '24a59d': 44,
 '743f4e': 45,
 '4bb5d0': 46,
 '516a67': 47,
 '48fd62': 48,
 '082f01': 49,
 'c95ab0': 50,
 '59f492': 51,
 '79011a': 52,
 'b19f77': 53,
 '5c0b8a': 54,
 '7688c1': 55,
 '4f13b4': 56,
 '19ea47': 57,
 '9f85da': 58,
 'e39bc5': 59,
 'c7fee4': 60,
 '194d1d': 61}

In [8]:
tdcsfog_metadata["Sub_id"] = tdcsfog_metadata["Subject"].map(sub_dict)


# pip install pyarrow
# pip install fastparquet
tdcsfog_metadata.to_parquet("./output/fe/fe001/fe001_tdcsfog_meta.parquet")

with open(f'./output/fe/fe{fe}/fe{fe}_sub_id.pkl', 'wb') as p:
    pickle.dump(sub_dict, p)

<br>


### fe022_tdcsfog_1000

<br>

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/fe/fe022_tdcsfog_1000.ipynb" style="text-decoration:none">fe022_tdcsfog_1000.ipynb</a>

In [9]:
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler

In [10]:
fe = "022"
if not os.path.exists(f"./output/fe/fe{fe}"):
    os.makedirs(f"./output/fe/fe{fe}")
    os.makedirs(f"./output/fe/fe{fe}/save")

In [11]:
# TDCSFOG_META_PATH = "../data/tdcsfog_metadata.csv"
# TDCSFOG_FOLDER = "../data/train/tdcsfog/*.csv"      

# data_list = glob.glob(TDCSFOG_FOLDER)

# data_list

# Abrachan: The variable `data_list` is the same as `train_tdcdfog` above.

In [12]:
tdcsfog_metadata = pd.read_parquet("./output/fe/fe001/fe001_tdcsfog_meta.parquet")
tdcsfog_metadata.head()

,Id,Subject,Visit,Test,Medication,Sub_id
0,003f117e14,4dc2f8,3,2,on,0
1,009ee11563,f62eec,4,2,on,1
2,011322847a,231c3b,2,2,on,2
3,01d0fe7266,231c3b,2,1,off,2
4,024418ba39,fa8764,19,3,on,3


In [13]:
cols       = ["AccV","AccML","AccAP"]
num_cols   = ['AccV', 'AccML', 'AccAP',  
              'AccV_lag_diff',  'AccV_lead_diff',  'AccV_cumsum', 
              'AccML_lag_diff', 'AccML_lead_diff', 'AccML_cumsum', 
              'AccAP_lag_diff', 'AccAP_lead_diff', 'AccAP_cumsum']
target_cols = ["StartHesitation","Turn","Walking"]

seq_len = 1000
shift = 500
offset = 250

# Abrachan:
# https://www.kaggle.com/competitions/tlvmc-parkinsons-freezing-gait-prediction/discussion/416057
#     Each Id was split into sequences of a specified length. During training, 
#     we used a shorter length (e.g., 1000 for tdcsfog, 5000 for defog)
#     For tdcsfog, sequences were created by shifting 500 steps from the starting position.

In [14]:
num_array = []
target_array = []
subject_list = []
id_list = []
mask_array = []
pred_use_array = []
time_array = []

In [15]:
for i,s in tqdm(zip(tdcsfog_metadata["Id"].values, tdcsfog_metadata["Sub_id"].values), desc="tdcsfog_metadata: "):
# for i,s in tqdm(zip(meta["Id"].values, meta["sub_id"].values)):
    # path = f"../data/train/tdcsfog/{i}.csv"
    path = root_data + f"train/tdcsfog/{i}.csv"
    df = pd.read_csv(path)
    
    batch = (len(df)-1) // shift    # Abrachan: Why minus 1 here?
    
    for c in cols:
        df[f"{c}_lag_diff"] = df[c].diff()
        df[f"{c}_lead_diff"] = df[c].diff(-1)
        df[f"{c}_cumsum"] = df[c].cumsum()
    
    sc = RobustScaler()
    df[num_cols] = sc.fit_transform(df[num_cols].values)
    df[num_cols] = df[num_cols].fillna(0)
    # for c in num_cols:
    #     df[c] = (df[c] - mean_std_dict[c][0]) / mean_std_dict[c][1]
    #     df[c] = df[c].fillna(0)
    
    num    = df[num_cols].values
    target = df[target_cols].values
    time_values   = df["Time"].values
    
    num_array_      = np.zeros([batch, seq_len, 12])
    target_array_   = np.zeros([batch, seq_len,  3])
    time_array_     = np.zeros([batch, seq_len    ], dtype=int)
    
    mask_array_     = np.zeros([batch, seq_len    ], dtype=int)
    pred_use_array_ = np.zeros([batch, seq_len    ], dtype=int)
    
    
    for n,b in enumerate(range(batch)):
        if b == (batch - 1):
            num_ = num[b*shift : ]
            num_array_[b, :len(num_), :] = num_
            
            target_ = target[b*shift : ]
            target_array_[b, :len(target_), :] = target_
            
            mask_array_[b, :len(target_)] = 1
            
            pred_use_array_[b, offset:len(target_)] = 1
            
            time_ = time_values[b*shift : ]
            time_array_[b, :len(time_)] = time_
            
        elif b == 0:
            num_ = num[b*shift : b*shift+seq_len]          # [0:1000]
            num_array_[b, :, :] = num_
            
            target_ = target[b*shift : b*shift+seq_len]    # [0:1000]
            target_array_[b, :, :] = target_
            
            mask_array_[b, :] = 1                          # [b, :1000]
            
            pred_use_array_[b, :offset+shift] = 1          # [b, :750]    <===
            
            time_ = time_values[b*shift : b*shift+seq_len] # [0:1000]
            time_array_[b, :] = time_
        
        else:
            num_ = num[b*shift : b*shift+seq_len]          # for b=1: [500:1500]
            num_array_[b, :, :] = num_
            
            target_ = target[b*shift : b*shift+seq_len]    # for b=1: [500:1500]
            target_array_[b, :, :] = target_
            
            mask_array_[b, :] = 1                          # for b=1: [b, :1000]
            
            pred_use_array_[b, offset:offset+shift] = 1    # for b=1: [b, 250:750]   <===
            
            time_ = time_values[b*shift : b*shift+seq_len] # for b=1: [500:1500]
            time_array_[b,:] = time_
    
    
    num_array.append(num_array_)
    target_array.append(target_array_)
    mask_array.append(mask_array_)
    pred_use_array.append(pred_use_array_)
    time_array.append(time_array_)
    
    subject_list += [s for _ in range(batch)]     # s is `Sub_id`
    id_list      += [i for _ in range(batch)]     # i is `Id`

tdcsfog_metadata: : 833it [00:33, 24.74it/s]


In [16]:
num_array      = np.concatenate(num_array, axis=0)
target_array   = np.concatenate(target_array, axis=0)
mask_array     = np.concatenate(mask_array, axis=0)
pred_use_array = np.concatenate(pred_use_array, axis=0)
time_array     = np.concatenate(time_array, axis=0)

In [17]:
df_id = pd.DataFrame()
df_id["Id"] = id_list
df_id["subject"] = subject_list

df_id

,Id,subject
0,003f117e14,0
1,003f117e14,0
2,003f117e14,0
3,003f117e14,0
4,003f117e14,0
...,...,...
13711,ffda8fadfd,14
13712,ffda8fadfd,14
13713,ffda8fadfd,14
13714,ffda8fadfd,14


In [18]:
np.save(f"./output/fe/fe{fe}/fe{fe}_num_array.npy", num_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_target_array.npy", target_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_mask_array.npy", mask_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_time_array.npy",time_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_pred_use_array.npy", pred_use_array)

In [19]:
df_id.to_parquet(f"./output/fe/fe{fe}/fe{fe}_id.parquet")

<br>

## <font color=red>**Training**</font>

<br>

Code below is common for the tdcsfog notebooks whose names start with "ex", for example: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/exp/ex143_tdcsfog_gru.ipynb" style="text-decoration:none">ex143_tdcsfog_gru.ipynb</a>

<br>

In [20]:
TRAIN_FLAG_TDCSFOG = True

<br>

### helpers

<br>

#### def `set_seed()`

In [21]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

<br>

#### def `init_logger()`

In [22]:
def init_logger(log_file):
    
    from logging import getLogger, INFO, FileHandler, Formatter, StreamHandler
    
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    
    return logger


# LOGGER = init_logger(log_file=logger_path)

<br>

#### def `timer()`

In [23]:
@contextmanager
def timer(name):
    t0 = time.time()
    
    yield 
    # LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s')
    print(f'[{name}] done in {time.time() - t0:.0f} s')
    print("="*66)
    print("\n"*2)
    
# setup_logger(out_file=logger_path)

<br>

#### def `preprocess()`

In [24]:
def preprocess(numerical_array, mask_array,):
    
    attention_mask = (mask_array == 0)

    return {'input_data_numerical_array': numerical_array,
            'input_data_mask_array': mask_array,
            'attention_mask': attention_mask,
           }

<br>

#### class `FogDataset()`

In [25]:
class FogDataset(Dataset):
    def __init__(self, numerical_array, mask_array, train = True, y = None):
        self.numerical_array = numerical_array
        self.mask_array = mask_array
        self.train = train
        self.y = y
    
    def __len__(self):
        return len(self.numerical_array)

    def __getitem__(self, item):
        data = preprocess(self.numerical_array[item], self.mask_array[item],)
        
        # Return the processed data where the lists are converted to `torch.tensor`s
        if self.train : 
            return {
              'input_data_numerical_array': torch.tensor(data['input_data_numerical_array'], dtype=torch.float32),
              'input_data_mask_array'     : torch.tensor(data['input_data_mask_array'], dtype=torch.long),  
              'attention_mask'            : torch.tensor(data["attention_mask"], dtype=torch.bool),
              "y"                         : torch.tensor(self.y[item], dtype=torch.float32)
               }
        else:
            return {
                'input_data_numerical_array': torch.tensor(data['input_data_numerical_array'], dtype=torch.float32),
                'input_data_mask_array'     : torch.tensor(data['input_data_mask_array'], dtype=torch.long),  
                'attention_mask'            : torch.tensor(data["attention_mask"], dtype=torch.bool),
               }

<br>

#### class `FogRnnModel()`

In [26]:
class FogRnnModel(nn.Module):
    def __init__(self, 
                 dropout=0.2,
                 input_numerical_size=12,        # len(num_cols) = 12
                 numeraical_linear_size = 64,
                 model_size = 128,
                 linear_out = 128,
                 out_size=3):
        
        super(FogRnnModel, self).__init__()
        
        # input shape : [batch, seq_len, len(num_cols)]            ->  [24, 1000, 12]
        # output shape: [batch, seq_len, numeraical_linear_size]   ->  [24, 1000, 64]
        self.numerical_linear  = nn.Sequential(nn.Linear(input_numerical_size, numeraical_linear_size),
                                               nn.LayerNorm(numeraical_linear_size)
                                              )
        
        # input shape : [batch, seq_len, numeraical_linear_size]   ->  [24, 1000, 64]
        # h_0 shape   : [D∗num_layers, batch, hidden_size]         ->  [2*2, 24, 128]
        # output shape: [batch, seq_len, D*hidden_size]            ->  [24,1000, 128*2]
        self.rnn = nn.GRU(numeraical_linear_size,    # input_size – The number of expected features in the input x
                          model_size,                # hidden_size – The number of features in the hidden state h
                          num_layers = 2,            # num_layers – Number of recurrent layers. E.g., setting num_layers=2 would mean stacking two GRUs together to form a stacked GRU
                          batch_first=True,          # If True, the input and output tensors are provided as (batch, seq, feature) instead of (seq, batch, feature).
                          bidirectional=True)
        
        # input shape : [batch, seq_len, D*hidden_size]  ->  [24, 1000, 128*2]
        # output shape: [batch, seq_len, linear_out]     ->  [24, 1000, 3]
        self.linear_out  = nn.Sequential(nn.Linear(model_size*2, linear_out),    # [128*2, 3]
                                         nn.LayerNorm(linear_out),
                                         nn.ReLU(),
                                         nn.Dropout(dropout),
                                         nn.Linear(linear_out, out_size)
                                        )
        self._reinitialize()
        
        
    def _reinitialize(self):
        """Tensorflow/Keras-like initialization"""

        for name, p in self.named_parameters():
            if 'rnn' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
    
    
    def forward(self, numerical_array, mask_array, attention_mask):        
        numerical_embedding = self.numerical_linear(numerical_array)
        output, _           = self.rnn(numerical_embedding)
        output              = self.linear_out(output)
        return output

<br>

### load data & preprocessing

In [27]:
id_path        = f"./output/fe/fe022/fe022_id.parquet"

numerical_path = f"./output/fe/fe022/fe022_num_array.npy"
target_path    = f"./output/fe/fe022/fe022_target_array.npy"
mask_path      = f"./output/fe/fe022/fe022_mask_array.npy"
pred_use_path  = f"./output/fe/fe022/fe022_pred_use_array.npy"


df_id           = pd.read_parquet(id_path)

numerical_array = np.load(numerical_path)
target_array    = np.load(target_path)
mask_array      = np.load(mask_path)
pred_use_array  = np.load(pred_use_path)

In [28]:
target1 = []
target2 = []
target3 = []
for i in range(len(target_array)):
    target1.append(np.sum(target_array[i, :, 0]))
    target2.append(np.sum(target_array[i, :, 1]))
    target3.append(np.sum(target_array[i, :, 2]))

In [29]:
df_id["target1"] = target1
df_id["target2"] = target2
df_id["target3"] = target3
df_id["target1_1"] = df_id["target1"] > 0
df_id["target2_1"] = df_id["target2"] > 0
df_id["target3_1"] = df_id["target3"] > 0
df_id["target1_1"] = df_id["target1_1"].astype(np.int)
df_id["target2_1"] = df_id["target2_1"].astype(np.int)
df_id["target3_1"] = df_id["target3_1"].astype(np.int)

In [30]:
df_id["group"] = 0
df_id.loc[df_id["target2_1"] > 0,"group"] = 1
df_id.loc[df_id["target1_1"] > 0,"group"] = 2
df_id.loc[df_id["target3_1"] > 0,"group"] = 3

In [31]:
df_id["group"].value_counts()

0    7720
1    4604
2     782
3     610
Name: group, dtype: int64

<br>

### config

In [32]:
# config
seed = 0
shuffle = True
n_splits = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model config
batch_size = 24
n_epochs = 10
lr = 1e-3
weight_decay = 0.05
num_warmup_steps = 10

<br>

### <font color=maroon>**ex143_tdcsfog_gru.ipynb**</font>

<br>

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/exp/ex143_tdcsfog_gru.ipynb" style="text-decoration:none">ex143_tdcsfog_gru.ipynb</a>

In [33]:
debug = False

ex = "143_tdcsfog"
if not os.path.exists(f"./output/exp/ex{ex}"):
    os.makedirs(f"./output/exp/ex{ex}")
    os.makedirs(f"./output/exp/ex{ex}/ex{ex}_model")
    
logger_path = f"./output/exp/ex{ex}/log_ex_{ex}.txt"
LOGGER = init_logger(log_file=logger_path)

# model_path  = f"./output/exp/ex{ex}/ex{ex}_model/ex{ex}.pth"

<br>

#### main

In [34]:
%%time

# with timer("gru"):
if TRAIN_FLAG:
    set_seed(seed)
    y_oof = np.empty([len(target_array), 1000, 3])      # Abrachan: out of fold
    gkf = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state = seed)
    for fold, (train_idx, valid_idx) in enumerate(gkf.split(numerical_array, 
                                                            y = df_id["group"].values,
                                                            groups=df_id["subject"].values)):
        # LOGGER.info(f"start fold: {fold+1}")
        print(f"start fold: {fold+1}")
        
        with timer(f"fold {fold+1}"):
            train_numerical_array = numerical_array[train_idx]
            train_target_array = target_array[train_idx]
            train_mask_array = mask_array[train_idx]

            val_numerical_array = numerical_array[valid_idx]
            val_target_array = target_array[valid_idx]
            val_mask_array = mask_array[valid_idx]
            val_pred_array = pred_use_array[valid_idx]
            
            train_ = FogDataset(train_numerical_array,
                                train_mask_array,
                                train=True,
                                y=train_target_array)
            val_ = FogDataset(val_numerical_array,
                              val_mask_array,
                              train=True,
                              y=val_target_array)
            
            train_loader = DataLoader(dataset=train_, 
                                      batch_size=batch_size, 
                                      shuffle = True, 
                                      # num_workers=8,
                                     )
            val_loader = DataLoader(dataset=val_, 
                                    batch_size=batch_size, 
                                    shuffle = False , 
                                    # num_workers=8
                                   )
            
            
            model = FogRnnModel()
            model = model.to(device)
            
            param_optimizer = list(model.named_parameters())
            no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
            optimizer_grouped_parameters = [
                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 
                 'weight_decay': weight_decay
                },
                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 
                 'weight_decay': 0.0
                }]
            optimizer = AdamW(optimizer_grouped_parameters,
                              lr=lr,
                              weight_decay=weight_decay,
                              )
            num_train_optimization_steps = int(len(train_loader) * n_epochs)
            scheduler = get_linear_schedule_with_warmup(optimizer,
                                                        num_warmup_steps=num_warmup_steps,
                                                        num_training_steps=num_train_optimization_steps)
            criterion = nn.BCEWithLogitsLoss()
            best_val_score = 0
            
            for epoch in range(n_epochs):
                model.train() 
                train_losses_batch = []
                val_losses_batch = []
                epoch_loss = 0
                train_preds = np.ndarray((0,3))   # array([], shape=(0, 3), dtype=float64)
                
                tk0 = tqdm_nb(train_loader, total=len(train_loader), desc="train_loader: ")
                for d in tk0:
                    # ======================================================
                    # data loader
                    # ======================================================
                    input_data_numerical_array = d['input_data_numerical_array'].to(device)
                    input_data_mask_array      = d['input_data_mask_array'].to(device)
                    attention_mask             = d['attention_mask'].to(device)
                    y                          = d["y"].to(device)
                    
                    optimizer.zero_grad()
                    output = model(input_data_numerical_array, 
                                   input_data_mask_array,
                                   attention_mask)
                    loss = criterion(output[input_data_mask_array == 1], y[input_data_mask_array == 1])
                    loss.backward()
                    optimizer.step()
                    scheduler.step()
                    train_losses_batch.append(loss.item())
                train_loss = np.mean(train_losses_batch)
                
                
                # ======================================================
                # eval
                # ======================================================
                model.eval()       # switch model to the evaluation mode
                
                val_preds = np.ndarray((0, 1000, 3))
                tk0 = tqdm_nb(val_loader, total=len(val_loader), desc="val_loader  : ")
                with torch.no_grad():  # Do not calculate gradient since we are only predicting
                    # Predicting on validation set
                    for d in tk0:
                        input_data_numerical_array = d['input_data_numerical_array'].to(device)
                        input_data_mask_array      = d['input_data_mask_array'].to(device)
                        attention_mask             = d['attention_mask'].to(device)
                        
                        output = model(input_data_numerical_array, 
                                       input_data_mask_array,
                                       attention_mask)
                        val_preds = np.concatenate([val_preds, output.sigmoid().detach().cpu().numpy()], axis=0)
                
                pred_valid_index = (val_mask_array == 1) & (val_pred_array == 1)
                StartHesitation = average_precision_score(val_target_array[pred_valid_index][:, 0],
                                                          val_preds[pred_valid_index][:, 0])
                Turn = average_precision_score(val_target_array[pred_valid_index][:, 1],
                                               val_preds[pred_valid_index][:, 1])
                Walking = average_precision_score(val_target_array[pred_valid_index][:, 2],
                                                  val_preds[pred_valid_index][:, 2])
                map_score = np.mean([StartHesitation, Turn, Walking])
                
                # LOGGER.info(f"fold: {fold+1} epoch: {epoch+1},train loss {train_loss} map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                print(f"fold {fold+1} epoch {epoch+1}\ntrain loss {train_loss} map:{map_score}\nstart_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                
                if map_score >= best_val_score:
                    print("save weight")
                    best_val_score = map_score
                    best_val_preds = val_preds.copy()
                    torch.save(model.state_dict(), f"./output/exp/ex{ex}/ex{ex}_model/ex{ex}_fold{fold+1}_epoch{epoch+1}__map{best_val_score:.6f}.pth")
                print()
            y_oof[valid_idx] = best_val_preds
    
    np.save(f"./output/exp/ex{ex}/ex{ex}_oof.npy", y_oof)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 10 µs


<br>

#### oof score

In [35]:
if TRAIN_FLAG:
    val_pred_index = (mask_array == 1) & (pred_use_array == 1)
    StartHesitation = average_precision_score(target_array[val_pred_index][:,0], 
                                              y_oof[val_pred_index][:,0])
    Turn            = average_precision_score(target_array[val_pred_index][:,1], 
                                              y_oof[val_pred_index][:,1])
    Walking         = average_precision_score(target_array[val_pred_index][:,2],
                                              y_oof[val_pred_index][:,2])

    map_score = np.mean([StartHesitation, Turn, Walking])
    # LOGGER.info(f"cv map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
    print(f"cv map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")



    # kaggle_json = {"title": f"fog-ex{ex}",
    #                "id": f"takoihiraokazu/fog-ex{ex}",
    #                "licenses": [{"name": "CC0-1.0"}]}

    # with open(f"./output/exp/ex{ex}/ex{ex}_model/dataset-metadata.json", 'w') as f:
    #     json.dump(kaggle_json, f)


    # del LOGGER
    # gc.collect()
    # logging.shutdown()

<br>

### <font color=maroon>ex145_tdcsfog_gru_StartHesitation.ipynb</font>

<br>

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/exp/ex145_tdcsfog_gru_StartHesitation.ipynb" style="text-decoration:none">ex145_tdcsfog_gru_StartHesitation.ipynb</a>

In [36]:
debug = False

ex = "145_tdcsfog"
if not os.path.exists(f"./output/exp/ex{ex}"):
    os.makedirs(f"./output/exp/ex{ex}")
    os.makedirs(f"./output/exp/ex{ex}/ex{ex}_model")
    
# logger_path = f"./output/exp/ex{ex}/log_ex_{ex}.txt"
# LOGGER = init_logger(log_file=logger_path)

<br>

#### main

In [37]:
%%time

# with timer("gru"):
if TRAIN_FLAG:
    set_seed(seed)
    y_oof = np.empty([len(target_array), 1000, 3])      # Abrachan: out of fold
    gkf = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state = seed)
    for fold, (train_idx, valid_idx) in enumerate(gkf.split(numerical_array, 
                                                            y = df_id["group"].values,
                                                            groups=df_id["subject"].values)):
        # LOGGER.info(f"start fold:{fold+1}")
        print(f"start fold:{fold+1}")
        
        with timer(f"fold {fold+1}"):
            train_numerical_array = numerical_array[train_idx]
            train_target_array = target_array[train_idx]
            train_mask_array = mask_array[train_idx]

            val_numerical_array = numerical_array[valid_idx]
            val_target_array = target_array[valid_idx]
            val_mask_array = mask_array[valid_idx]
            val_pred_array = pred_use_array[valid_idx]
            
            train_ = FogDataset(train_numerical_array,
                                train_mask_array,
                                train=True,
                                y=train_target_array)
            val_ = FogDataset(val_numerical_array,
                              val_mask_array,
                              train=True,
                              y=val_target_array)
            
            train_loader = DataLoader(dataset=train_, 
                                      batch_size=batch_size, 
                                      shuffle = True, 
                                      # num_workers=8,
                                     )
            val_loader = DataLoader(dataset=val_, 
                                    batch_size=batch_size, 
                                    shuffle = False , 
                                    # num_workers=8
                                   )
            
            
            model = FogRnnModel()
            model = model.to(device)
            
            param_optimizer = list(model.named_parameters())
            no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
            optimizer_grouped_parameters = [
                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 
                 'weight_decay': weight_decay
                },
                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 
                 'weight_decay': 0.0
                }]
            optimizer = AdamW(optimizer_grouped_parameters,
                              lr=lr,
                              weight_decay=weight_decay,
                              )
            num_train_optimization_steps = int(len(train_loader) * n_epochs)
            scheduler = get_linear_schedule_with_warmup(optimizer,
                                                        num_warmup_steps=num_warmup_steps,
                                                        num_training_steps=num_train_optimization_steps)
            criterion = nn.BCEWithLogitsLoss()
            best_val_score = 0
            
            for epoch in range(n_epochs):
                model.train() 
                train_losses_batch = []
                val_losses_batch = []
                epoch_loss = 0
                train_preds = np.ndarray((0,3))   # array([], shape=(0, 3), dtype=float64)
                
                tk0 = tqdm_nb(train_loader, total=len(train_loader), desc="train_loader: ")
                for d in tk0:
                    # ======================================================
                    # data loader
                    # ======================================================
                    input_data_numerical_array = d['input_data_numerical_array'].to(device)
                    input_data_mask_array      = d['input_data_mask_array'].to(device)
                    attention_mask             = d['attention_mask'].to(device)
                    y                          = d["y"].to(device)
                    
                    optimizer.zero_grad()
                    output = model(input_data_numerical_array, 
                                   input_data_mask_array,
                                   attention_mask)
                    # loss = criterion(output[input_data_mask_array == 1], y[input_data_mask_array == 1])
                    
                    output1 = output[:, :, 0    ]
                    output2 = output[:, :, [1,2]]
                    y1 = y[:, :, 0    ]
                    y2 = y[:, :, [1,2]]
                    loss1 = criterion(output1[input_data_mask_array == 1], y1[input_data_mask_array == 1])
                    loss2 = criterion(output2[input_data_mask_array == 1], y2[input_data_mask_array == 1])
                    
                    loss = loss1*0.6 + loss2*0.4
                    loss.backward()
                    optimizer.step()
                    scheduler.step()
                    train_losses_batch.append(loss.item())
                train_loss = np.mean(train_losses_batch)
                
                
                # ======================================================
                # eval
                # ======================================================
                model.eval()       # switch model to the evaluation mode
                
                val_preds = np.ndarray((0, 1000, 3))
                tk0 = tqdm_nb(val_loader, total=len(val_loader), desc="val_loader  : ")
                with torch.no_grad():  # Do not calculate gradient since we are only predicting
                    # Predicting on validation set
                    for d in tk0:
                        input_data_numerical_array = d['input_data_numerical_array'].to(device)
                        input_data_mask_array      = d['input_data_mask_array'].to(device)
                        attention_mask             = d['attention_mask'].to(device)
                        
                        output = model(input_data_numerical_array, 
                                       input_data_mask_array,
                                       attention_mask)
                        val_preds = np.concatenate([val_preds, output.sigmoid().detach().cpu().numpy()], axis=0)
                
                pred_valid_index = (val_mask_array == 1) & (val_pred_array == 1)
                StartHesitation = average_precision_score(val_target_array[pred_valid_index][:, 0],
                                                          val_preds[pred_valid_index][:, 0])
                Turn = average_precision_score(val_target_array[pred_valid_index][:, 1],
                                               val_preds[pred_valid_index][:, 1])
                Walking = average_precision_score(val_target_array[pred_valid_index][:, 2],
                                                  val_preds[pred_valid_index][:, 2])
                map_score = np.mean([StartHesitation, Turn, Walking])
                
                # LOGGER.info(f"fold: {fold+1} epoch: {epoch+1},train loss {train_loss} map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                print(f"fold {fold+1} epoch {epoch+1}\ntrain loss {train_loss} map:{map_score}\nstart_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                
                if map_score >= best_val_score:
                    print("save weight")
                    best_val_score = map_score
                    best_val_preds = val_preds.copy()
                    torch.save(model.state_dict(), f"./output/exp/ex{ex}/ex{ex}_model/ex{ex}_fold{fold+1}_epoch{epoch+1}__map{best_val_score:.6f}.pth")
                print()
            y_oof[valid_idx] = best_val_preds
    
    np.save(f"./output/exp/ex{ex}/ex{ex}_oof.npy", y_oof)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.11 µs


<br>

#### oof score

In [38]:
if TRAIN_FLAG:
    val_pred_index = (mask_array == 1) & (pred_use_array == 1)
    StartHesitation = average_precision_score(target_array[val_pred_index][:,0], 
                                              y_oof[val_pred_index][:,0])
    Turn            = average_precision_score(target_array[val_pred_index][:,1], 
                                              y_oof[val_pred_index][:,1])
    Walking         = average_precision_score(target_array[val_pred_index][:,2],
                                              y_oof[val_pred_index][:,2])

    map_score = np.mean([StartHesitation, Turn, Walking])
    # LOGGER.info(f"cv map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
    print(f"cv map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")



    # kaggle_json = {"title": f"fog-ex{ex}",
    #                "id": f"takoihiraokazu/fog-ex{ex}",
    #                "licenses": [{"name": "CC0-1.0"}]}

    # with open(f"./output/exp/ex{ex}/ex{ex}_model/dataset-metadata.json", 'w') as f:
    #     json.dump(kaggle_json, f)


    # del LOGGER
    # gc.collect()

<br>

### <font color=maroon>ex146_tdcsfog_gru_Turn.ipynb</font>

<br>

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/exp/ex146_tdcsfog_gru_Turn.ipynb" style="text-decoration:none">ex146_tdcsfog_gru_Turn.ipynb</a>

In [39]:
debug = False
ex = "146_tdcsfog"
if not os.path.exists(f"./output/exp/ex{ex}"):
    os.makedirs(f"./output/exp/ex{ex}")
    os.makedirs(f"./output/exp/ex{ex}/ex{ex}_model")

# logger_path = f"./output/exp/ex{ex}/ex_{ex}.txt"
# LOGGER = init_logger(log_file=logger_path)

<br>

#### main

In [40]:
%%time

# with timer("gru"):
if TRAIN_FLAG:
    set_seed(seed)
    y_oof = np.empty([len(target_array), 1000, 3])      # Abrachan: out of fold
    gkf = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state = seed)
    for fold, (train_idx, valid_idx) in enumerate(gkf.split(numerical_array, 
                                                            y = df_id["group"].values,
                                                            groups=df_id["subject"].values)):
        # LOGGER.info(f"start fold:{fold+1}")
        print(f"start fold: {fold+1}")
        
        with timer(f"fold {fold+1}"):
            train_numerical_array = numerical_array[train_idx]
            train_target_array = target_array[train_idx]
            train_mask_array = mask_array[train_idx]

            val_numerical_array = numerical_array[valid_idx]
            val_target_array = target_array[valid_idx]
            val_mask_array = mask_array[valid_idx]
            val_pred_array = pred_use_array[valid_idx]
            
            train_ = FogDataset(train_numerical_array,
                                train_mask_array,
                                train=True,
                                y=train_target_array)
            val_ = FogDataset(val_numerical_array,
                              val_mask_array,
                              train=True,
                              y=val_target_array)
            
            train_loader = DataLoader(dataset=train_, 
                                      batch_size=batch_size, 
                                      shuffle = True, 
                                      # num_workers=8,
                                     )
            val_loader = DataLoader(dataset=val_, 
                                    batch_size=batch_size, 
                                    shuffle = False , 
                                    # num_workers=8
                                   )
            
            
            model = FogRnnModel()
            model = model.to(device)
            
            param_optimizer = list(model.named_parameters())
            no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
            optimizer_grouped_parameters = [
                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 
                 'weight_decay': weight_decay
                },
                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 
                 'weight_decay': 0.0
                }]
            optimizer = AdamW(optimizer_grouped_parameters,
                              lr=lr,
                              weight_decay=weight_decay,
                              )
            num_train_optimization_steps = int(len(train_loader) * n_epochs)
            scheduler = get_linear_schedule_with_warmup(optimizer,
                                                        num_warmup_steps=num_warmup_steps,
                                                        num_training_steps=num_train_optimization_steps)
            criterion = nn.BCEWithLogitsLoss()
            best_val_score = 0
            
            for epoch in range(n_epochs):
                model.train() 
                train_losses_batch = []
                val_losses_batch = []
                epoch_loss = 0
                train_preds = np.ndarray((0,3))   # array([], shape=(0, 3), dtype=float64)
                
                tk0 = tqdm_nb(train_loader, total=len(train_loader), desc="train_loader: ")
                for d in tk0:
                    # ======================================================
                    # data loader
                    # ======================================================
                    input_data_numerical_array = d['input_data_numerical_array'].to(device)
                    input_data_mask_array      = d['input_data_mask_array'].to(device)
                    attention_mask             = d['attention_mask'].to(device)
                    y                          = d["y"].to(device)
                    
                    optimizer.zero_grad()
                    output = model(input_data_numerical_array, 
                                   input_data_mask_array,
                                   attention_mask)
                    # loss = criterion(output[input_data_mask_array == 1], y[input_data_mask_array == 1])
                    
                    output1 = output[:, :, 1    ]
                    output2 = output[:, :, [0,2]]
                    y1 = y[:, :, 1    ]
                    y2 = y[:, :, [0,2]]
                    loss1 = criterion(output1[input_data_mask_array == 1], y1[input_data_mask_array == 1])
                    loss2 = criterion(output2[input_data_mask_array == 1], y2[input_data_mask_array == 1])
                    
                    loss = loss1*0.6 + loss2*0.4
                    loss.backward()
                    optimizer.step()
                    scheduler.step()
                    train_losses_batch.append(loss.item())
                train_loss = np.mean(train_losses_batch)
                
                
                # ======================================================
                # eval
                # ======================================================
                model.eval()       # switch model to the evaluation mode
                
                val_preds = np.ndarray((0, 1000, 3))
                tk0 = tqdm_nb(val_loader, total=len(val_loader), desc="val_loader  : ")
                with torch.no_grad():  # Do not calculate gradient since we are only predicting
                    # Predicting on validation set
                    for d in tk0:
                        input_data_numerical_array = d['input_data_numerical_array'].to(device)
                        input_data_mask_array      = d['input_data_mask_array'].to(device)
                        attention_mask             = d['attention_mask'].to(device)
                        
                        output = model(input_data_numerical_array, 
                                       input_data_mask_array,
                                       attention_mask)
                        val_preds = np.concatenate([val_preds, output.sigmoid().detach().cpu().numpy()], axis=0)
                
                pred_valid_index = (val_mask_array == 1) & (val_pred_array == 1)
                StartHesitation = average_precision_score(val_target_array[pred_valid_index][:, 0],
                                                          val_preds[pred_valid_index][:, 0])
                Turn = average_precision_score(val_target_array[pred_valid_index][:, 1],
                                               val_preds[pred_valid_index][:, 1])
                Walking = average_precision_score(val_target_array[pred_valid_index][:, 2],
                                                  val_preds[pred_valid_index][:, 2])
                map_score = np.mean([StartHesitation, Turn, Walking])
                
                # LOGGER.info(f"fold: {fold+1} epoch: {epoch+1},train loss {train_loss} map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                print(f"fold {fold+1} epoch {epoch+1}\ntrain loss {train_loss} map:{map_score}\nstart_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                
                if map_score >= best_val_score:
                    print("save weight")
                    best_val_score = map_score
                    best_val_preds = val_preds.copy()
                    torch.save(model.state_dict(), f"./output/exp/ex{ex}/ex{ex}_model/ex{ex}_fold{fold+1}_epoch{epoch+1}__map{best_val_score:.6f}.pth")
                print()
            y_oof[valid_idx] = best_val_preds
    
    np.save(f"./output/exp/ex{ex}/ex{ex}_oof.npy", y_oof)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 10.7 µs


<br>

#### oof score

In [41]:
if TRAIN_FLAG:
    val_pred_index = (mask_array == 1) & (pred_use_array == 1)
    StartHesitation = average_precision_score(target_array[val_pred_index][:,0], 
                                              y_oof[val_pred_index][:,0])
    Turn            = average_precision_score(target_array[val_pred_index][:,1], 
                                              y_oof[val_pred_index][:,1])
    Walking         = average_precision_score(target_array[val_pred_index][:,2],
                                              y_oof[val_pred_index][:,2])

    map_score = np.mean([StartHesitation, Turn, Walking])
    # LOGGER.info(f"cv map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
    print(f"cv map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")

<br>

### <font color=maroon>ex147_tdcsfog_gru_Walking.ipynb</font>

<br>

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/exp/ex147_tdcsfog_gru_Walking.ipynb" style="text-decoration:none">ex147_tdcsfog_gru_Walking.ipynb</a>

In [42]:
debug = False
ex = "147_tdcsfog"
if not os.path.exists(f"./output/exp/ex{ex}"):
    os.makedirs(f"./output/exp/ex{ex}")
    os.makedirs(f"./output/exp/ex{ex}/ex{ex}_model")

# logger_path = f"./output/exp/ex{ex}/ex_{ex}.txt"
# LOGGER = init_logger(log_file=logger_path)

<br>

#### main

In [43]:
%%time

# with timer("gru"):
if TRAIN_FLAG:
    set_seed(seed)
    y_oof = np.empty([len(target_array), 1000, 3])      # Abrachan: out of fold
    gkf = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state = seed)
    for fold, (train_idx, valid_idx) in enumerate(gkf.split(numerical_array, 
                                                            y = df_id["group"].values,
                                                            groups=df_id["subject"].values)):
        # LOGGER.info(f"start fold:{fold+1}")
        print(f"start fold: {fold+1}")
        
        with timer(f"fold {fold+1}"):
            train_numerical_array = numerical_array[train_idx]
            train_target_array = target_array[train_idx]
            train_mask_array = mask_array[train_idx]

            val_numerical_array = numerical_array[valid_idx]
            val_target_array = target_array[valid_idx]
            val_mask_array = mask_array[valid_idx]
            val_pred_array = pred_use_array[valid_idx]
            
            train_ = FogDataset(train_numerical_array,
                                train_mask_array,
                                train=True,
                                y=train_target_array)
            val_ = FogDataset(val_numerical_array,
                              val_mask_array,
                              train=True,
                              y=val_target_array)
            
            train_loader = DataLoader(dataset=train_, 
                                      batch_size=batch_size, 
                                      shuffle = True, 
                                      # num_workers=8,
                                     )
            val_loader = DataLoader(dataset=val_, 
                                    batch_size=batch_size, 
                                    shuffle = False , 
                                    # num_workers=8
                                   )
            
            
            model = FogRnnModel()
            model = model.to(device)
            
            param_optimizer = list(model.named_parameters())
            no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
            optimizer_grouped_parameters = [
                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 
                 'weight_decay': weight_decay
                },
                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 
                 'weight_decay': 0.0
                }]
            optimizer = AdamW(optimizer_grouped_parameters,
                              lr=lr,
                              weight_decay=weight_decay,
                              )
            num_train_optimization_steps = int(len(train_loader) * n_epochs)
            scheduler = get_linear_schedule_with_warmup(optimizer,
                                                        num_warmup_steps=num_warmup_steps,
                                                        num_training_steps=num_train_optimization_steps)
            criterion = nn.BCEWithLogitsLoss()
            best_val_score = 0
            
            for epoch in range(n_epochs):
                model.train() 
                train_losses_batch = []
                val_losses_batch = []
                epoch_loss = 0
                train_preds = np.ndarray((0,3))   # array([], shape=(0, 3), dtype=float64)
                
                tk0 = tqdm_nb(train_loader, total=len(train_loader), desc="train_loader: ")
                for d in tk0:
                    # ======================================================
                    # data loader
                    # ======================================================
                    input_data_numerical_array = d['input_data_numerical_array'].to(device)
                    input_data_mask_array      = d['input_data_mask_array'].to(device)
                    attention_mask             = d['attention_mask'].to(device)
                    y                          = d["y"].to(device)
                    
                    optimizer.zero_grad()
                    output = model(input_data_numerical_array, 
                                   input_data_mask_array,
                                   attention_mask)
                    # loss = criterion(output[input_data_mask_array == 1], y[input_data_mask_array == 1])
                    
                    output1 = output[:, :, 2    ]
                    output2 = output[:, :, [0,1]]
                    y1 = y[:, :, 2    ]
                    y2 = y[:, :, [0,1]]
                    loss1 = criterion(output1[input_data_mask_array == 1], y1[input_data_mask_array == 1])
                    loss2 = criterion(output2[input_data_mask_array == 1], y2[input_data_mask_array == 1])
                    
                    loss = loss1*0.6 + loss2*0.4
                    loss.backward()
                    optimizer.step()
                    scheduler.step()
                    train_losses_batch.append(loss.item())
                train_loss = np.mean(train_losses_batch)
                
                
                # ======================================================
                # eval
                # ======================================================
                model.eval()       # switch model to the evaluation mode
                
                val_preds = np.ndarray((0, 1000, 3))
                tk0 = tqdm_nb(val_loader, total=len(val_loader), desc="val_loader  : ")
                with torch.no_grad():  # Do not calculate gradient since we are only predicting
                    # Predicting on validation set
                    for d in tk0:
                        input_data_numerical_array = d['input_data_numerical_array'].to(device)
                        input_data_mask_array      = d['input_data_mask_array'].to(device)
                        attention_mask             = d['attention_mask'].to(device)
                        
                        output = model(input_data_numerical_array, 
                                       input_data_mask_array,
                                       attention_mask)
                        val_preds = np.concatenate([val_preds, output.sigmoid().detach().cpu().numpy()], axis=0)
                
                pred_valid_index = (val_mask_array == 1) & (val_pred_array == 1)
                StartHesitation = average_precision_score(val_target_array[pred_valid_index][:, 0],
                                                          val_preds[pred_valid_index][:, 0])
                Turn = average_precision_score(val_target_array[pred_valid_index][:, 1],
                                               val_preds[pred_valid_index][:, 1])
                Walking = average_precision_score(val_target_array[pred_valid_index][:, 2],
                                                  val_preds[pred_valid_index][:, 2])
                map_score = np.mean([StartHesitation, Turn, Walking])
                
                # LOGGER.info(f"fold: {fold+1} epoch: {epoch+1},train loss {train_loss} map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                print(f"fold {fold+1} epoch {epoch+1}\ntrain loss {train_loss} map:{map_score}\nstart_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                
                if map_score >= best_val_score:
                    print("save weight")
                    best_val_score = map_score
                    best_val_preds = val_preds.copy()
                    torch.save(model.state_dict(), f"./output/exp/ex{ex}/ex{ex}_model/ex{ex}_fold{fold+1}_epoch{epoch+1}__map{best_val_score:.6f}.pth")
                print()
            y_oof[valid_idx] = best_val_preds
    
    np.save(f"./output/exp/ex{ex}/ex{ex}_oof.npy", y_oof)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.58 µs


<br>

#### oof score

In [44]:
if TRAIN_FLAG:
    val_pred_index = (mask_array == 1) & (pred_use_array == 1)
    StartHesitation = average_precision_score(target_array[val_pred_index][:,0], 
                                              y_oof[val_pred_index][:,0])
    Turn            = average_precision_score(target_array[val_pred_index][:,1], 
                                              y_oof[val_pred_index][:,1])
    Walking         = average_precision_score(target_array[val_pred_index][:,2],
                                              y_oof[val_pred_index][:,2])

    map_score = np.mean([StartHesitation, Turn, Walking])
    # LOGGER.info(f"cv map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
    print(f"cv map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")


<br>

### <font color=maroon>ex182_tdcsfog_gru_StartHesitation_Turn.ipynb</font>

<br>

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/exp/ex182_tdcsfog_gru_StartHesitation_Turn.ipynb" style="text-decoration:none">ex182_tdcsfog_gru_StartHesitation_Turn.ipynb</a>

In [45]:
debug = False
ex = "182_tdcsfog"
if not os.path.exists(f"./output/exp/ex{ex}"):
    os.makedirs(f"./output/exp/ex{ex}")
    os.makedirs(f"./output/exp/ex{ex}/ex{ex}_model")

# logger_path = f"./output/exp/ex{ex}/ex_{ex}.txt"
# LOGGER = init_logger(log_file=logger_path)

<br>

#### main

In [46]:
%%time

# with timer("gru"):
if TRAIN_FLAG:
    set_seed(seed)
    y_oof = np.empty([len(target_array), 1000, 3])      # Abrachan: out of fold
    gkf = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state = seed)
    for fold, (train_idx, valid_idx) in enumerate(gkf.split(numerical_array, 
                                                            y = df_id["group"].values,
                                                            groups=df_id["subject"].values)):
        # LOGGER.info(f"start fold:{fold+1}")
        print(f"start fold: {fold+1}")
        
        with timer(f"fold {fold+1}"):
            train_numerical_array = numerical_array[train_idx]
            train_target_array = target_array[train_idx]
            train_mask_array = mask_array[train_idx]

            val_numerical_array = numerical_array[valid_idx]
            val_target_array = target_array[valid_idx]
            val_mask_array = mask_array[valid_idx]
            val_pred_array = pred_use_array[valid_idx]
            
            train_ = FogDataset(train_numerical_array,
                                train_mask_array,
                                train=True,
                                y=train_target_array)
            val_ = FogDataset(val_numerical_array,
                              val_mask_array,
                              train=True,
                              y=val_target_array)
            
            train_loader = DataLoader(dataset=train_, 
                                      batch_size=batch_size, 
                                      shuffle = True, 
                                      # num_workers=8,
                                     )
            val_loader = DataLoader(dataset=val_, 
                                    batch_size=batch_size, 
                                    shuffle = False , 
                                    # num_workers=8
                                   )
            
            
            model = FogRnnModel()
            model = model.to(device)
            
            param_optimizer = list(model.named_parameters())
            no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
            optimizer_grouped_parameters = [
                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 
                 'weight_decay': weight_decay
                },
                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 
                 'weight_decay': 0.0
                }]
            optimizer = AdamW(optimizer_grouped_parameters,
                              lr=lr,
                              weight_decay=weight_decay,
                              )
            num_train_optimization_steps = int(len(train_loader) * n_epochs)
            scheduler = get_linear_schedule_with_warmup(optimizer,
                                                        num_warmup_steps=num_warmup_steps,
                                                        num_training_steps=num_train_optimization_steps)
            criterion = nn.BCEWithLogitsLoss()
            best_val_score = 0
            
            for epoch in range(n_epochs):
                model.train() 
                train_losses_batch = []
                val_losses_batch = []
                epoch_loss = 0
                train_preds = np.ndarray((0,3))   # array([], shape=(0, 3), dtype=float64)
                
                tk0 = tqdm_nb(train_loader, total=len(train_loader), desc="train_loader: ")
                for d in tk0:
                    # ======================================================
                    # data loader
                    # ======================================================
                    input_data_numerical_array = d['input_data_numerical_array'].to(device)
                    input_data_mask_array      = d['input_data_mask_array'].to(device)
                    attention_mask             = d['attention_mask'].to(device)
                    y                          = d["y"].to(device)
                    
                    optimizer.zero_grad()
                    output = model(input_data_numerical_array, 
                                   input_data_mask_array,
                                   attention_mask)
                    # loss = criterion(output[input_data_mask_array == 1], y[input_data_mask_array == 1])
                    
                    output1 = output[:, :, [0,1]]
                    output2 = output[:, :, 2]
                    y1 = y[:, :, [0,1]]
                    y2 = y[:, :, 2]
                    loss1 = criterion(output1[input_data_mask_array == 1], y1[input_data_mask_array == 1])
                    loss2 = criterion(output2[input_data_mask_array == 1], y2[input_data_mask_array == 1])
                    
                    loss = loss1*0.8 + loss2*0.2
                    loss.backward()
                    optimizer.step()
                    scheduler.step()
                    train_losses_batch.append(loss.item())
                train_loss = np.mean(train_losses_batch)
                
                
                # ======================================================
                # eval
                # ======================================================
                model.eval()       # switch model to the evaluation mode
                
                val_preds = np.ndarray((0, 1000, 3))
                tk0 = tqdm_nb(val_loader, total=len(val_loader), desc="val_loader  : ")
                with torch.no_grad():  # Do not calculate gradient since we are only predicting
                    # Predicting on validation set
                    for d in tk0:
                        input_data_numerical_array = d['input_data_numerical_array'].to(device)
                        input_data_mask_array      = d['input_data_mask_array'].to(device)
                        attention_mask             = d['attention_mask'].to(device)
                        
                        output = model(input_data_numerical_array, 
                                       input_data_mask_array,
                                       attention_mask)
                        val_preds = np.concatenate([val_preds, output.sigmoid().detach().cpu().numpy()], axis=0)
                
                pred_valid_index = (val_mask_array == 1) & (val_pred_array == 1)
                StartHesitation = average_precision_score(val_target_array[pred_valid_index][:, 0],
                                                          val_preds[pred_valid_index][:, 0])
                Turn = average_precision_score(val_target_array[pred_valid_index][:, 1],
                                               val_preds[pred_valid_index][:, 1])
                Walking = average_precision_score(val_target_array[pred_valid_index][:, 2],
                                                  val_preds[pred_valid_index][:, 2])
                map_score = np.mean([StartHesitation, Turn, Walking])
                
                # LOGGER.info(f"fold: {fold+1} epoch: {epoch+1},train loss {train_loss} map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                print(f"fold {fold+1} epoch {epoch+1}\ntrain loss {train_loss} map:{map_score}\nstart_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                
                if map_score >= best_val_score:
                    print("save weight")
                    best_val_score = map_score
                    best_val_preds = val_preds.copy()
                    torch.save(model.state_dict(), f"./output/exp/ex{ex}/ex{ex}_model/ex{ex}_fold{fold+1}_epoch{epoch+1}__map{best_val_score:.6f}.pth")
                print()
            y_oof[valid_idx] = best_val_preds
    
    np.save(f"./output/exp/ex{ex}/ex{ex}_oof.npy", y_oof)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.3 µs


<br>

#### oof score

In [47]:
if TRAIN_FLAG:
    val_pred_index = (mask_array == 1) & (pred_use_array == 1)
    StartHesitation = average_precision_score(target_array[val_pred_index][:,0], 
                                              y_oof[val_pred_index][:,0])
    Turn            = average_precision_score(target_array[val_pred_index][:,1], 
                                              y_oof[val_pred_index][:,1])
    Walking         = average_precision_score(target_array[val_pred_index][:,2],
                                              y_oof[val_pred_index][:,2])

    map_score = np.mean([StartHesitation, Turn, Walking])
    # LOGGER.info(f"cv map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
    print(f"cv map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")

<br>

### <font color=maroon>ex183_tdcsfog_gru_StartHesitation_Walking.ipynb</font>

<br>

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/exp/ex183_tdcsfog_gru_StartHesitation_Walking.ipynb" style="text-decoration:none">ex183_tdcsfog_gru_StartHesitation_Walking.ipynb</a>

In [48]:
debug = False
ex = "183_tdcsfog"
if not os.path.exists(f"./output/exp/ex{ex}"):
    os.makedirs(f"./output/exp/ex{ex}")
    os.makedirs(f"./output/exp/ex{ex}/ex{ex}_model")

# logger_path = f"./output/exp/ex{ex}/ex_{ex}.txt"
# LOGGER = init_logger(log_file=logger_path)

<br>

#### main

In [49]:
%%time

# with timer("gru"):
if TRAIN_FLAG:
    set_seed(seed)
    y_oof = np.empty([len(target_array), 1000, 3])      # Abrachan: out of fold
    gkf = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state = seed)
    for fold, (train_idx, valid_idx) in enumerate(gkf.split(numerical_array, 
                                                            y = df_id["group"].values,
                                                            groups=df_id["subject"].values)):
        # LOGGER.info(f"start fold:{fold+1}")
        print(f"start fold: {fold+1}")
        
        with timer(f"fold {fold+1}"):
            train_numerical_array = numerical_array[train_idx]
            train_target_array = target_array[train_idx]
            train_mask_array = mask_array[train_idx]

            val_numerical_array = numerical_array[valid_idx]
            val_target_array = target_array[valid_idx]
            val_mask_array = mask_array[valid_idx]
            val_pred_array = pred_use_array[valid_idx]
            
            train_ = FogDataset(train_numerical_array,
                                train_mask_array,
                                train=True,
                                y=train_target_array)
            val_ = FogDataset(val_numerical_array,
                              val_mask_array,
                              train=True,
                              y=val_target_array)
            
            train_loader = DataLoader(dataset=train_, 
                                      batch_size=batch_size, 
                                      shuffle = True, 
                                      # num_workers=8,
                                     )
            val_loader = DataLoader(dataset=val_, 
                                    batch_size=batch_size, 
                                    shuffle = False , 
                                    # num_workers=8
                                   )
            
            
            model = FogRnnModel()
            model = model.to(device)
            
            param_optimizer = list(model.named_parameters())
            no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
            optimizer_grouped_parameters = [
                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 
                 'weight_decay': weight_decay
                },
                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 
                 'weight_decay': 0.0
                }]
            optimizer = AdamW(optimizer_grouped_parameters,
                              lr=lr,
                              weight_decay=weight_decay,
                              )
            num_train_optimization_steps = int(len(train_loader) * n_epochs)
            scheduler = get_linear_schedule_with_warmup(optimizer,
                                                        num_warmup_steps=num_warmup_steps,
                                                        num_training_steps=num_train_optimization_steps)
            criterion = nn.BCEWithLogitsLoss()
            best_val_score = 0
            
            for epoch in range(n_epochs):
                model.train() 
                train_losses_batch = []
                val_losses_batch = []
                epoch_loss = 0
                train_preds = np.ndarray((0,3))   # array([], shape=(0, 3), dtype=float64)
                
                tk0 = tqdm_nb(train_loader, total=len(train_loader), desc="train_loader: ")
                for d in tk0:
                    # ======================================================
                    # data loader
                    # ======================================================
                    input_data_numerical_array = d['input_data_numerical_array'].to(device)
                    input_data_mask_array      = d['input_data_mask_array'].to(device)
                    attention_mask             = d['attention_mask'].to(device)
                    y                          = d["y"].to(device)
                    
                    optimizer.zero_grad()
                    output = model(input_data_numerical_array, 
                                   input_data_mask_array,
                                   attention_mask)
                    # loss = criterion(output[input_data_mask_array == 1], y[input_data_mask_array == 1])
                    
                    output1 = output[:, :, [0,2]]
                    output2 = output[:, :, 1]
                    y1 = y[:, :, [0,2]]
                    y2 = y[:, :, 1]
                    loss1 = criterion(output1[input_data_mask_array == 1], y1[input_data_mask_array == 1])
                    loss2 = criterion(output2[input_data_mask_array == 1], y2[input_data_mask_array == 1])
                    
                    loss = loss1*0.8 + loss2*0.2
                    loss.backward()
                    optimizer.step()
                    scheduler.step()
                    train_losses_batch.append(loss.item())
                train_loss = np.mean(train_losses_batch)
                
                
                # ======================================================
                # eval
                # ======================================================
                model.eval()       # switch model to the evaluation mode
                
                val_preds = np.ndarray((0, 1000, 3))
                tk0 = tqdm_nb(val_loader, total=len(val_loader), desc="val_loader  : ")
                with torch.no_grad():  # Do not calculate gradient since we are only predicting
                    # Predicting on validation set
                    for d in tk0:
                        input_data_numerical_array = d['input_data_numerical_array'].to(device)
                        input_data_mask_array      = d['input_data_mask_array'].to(device)
                        attention_mask             = d['attention_mask'].to(device)
                        
                        output = model(input_data_numerical_array, 
                                       input_data_mask_array,
                                       attention_mask)
                        val_preds = np.concatenate([val_preds, output.sigmoid().detach().cpu().numpy()], axis=0)
                
                pred_valid_index = (val_mask_array == 1) & (val_pred_array == 1)
                StartHesitation = average_precision_score(val_target_array[pred_valid_index][:, 0],
                                                          val_preds[pred_valid_index][:, 0])
                Turn = average_precision_score(val_target_array[pred_valid_index][:, 1],
                                               val_preds[pred_valid_index][:, 1])
                Walking = average_precision_score(val_target_array[pred_valid_index][:, 2],
                                                  val_preds[pred_valid_index][:, 2])
                map_score = np.mean([StartHesitation, Turn, Walking])
                
                # LOGGER.info(f"fold: {fold+1} epoch: {epoch+1},train loss {train_loss} map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                print(f"fold {fold+1} epoch {epoch+1}\ntrain loss {train_loss} map:{map_score}\nstart_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                
                if map_score >= best_val_score:
                    print("save weight")
                    best_val_score = map_score
                    best_val_preds = val_preds.copy()
                    torch.save(model.state_dict(), f"./output/exp/ex{ex}/ex{ex}_model/ex{ex}_fold{fold+1}_epoch{epoch+1}__map{best_val_score:.6f}.pth")
                print()
            y_oof[valid_idx] = best_val_preds
    
    np.save(f"./output/exp/ex{ex}/ex{ex}_oof.npy", y_oof)

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 9.78 µs


<br>

#### oof score

In [50]:
if TRAIN_FLAG:
    val_pred_index = (mask_array == 1) & (pred_use_array == 1)
    StartHesitation = average_precision_score(target_array[val_pred_index][:,0], 
                                              y_oof[val_pred_index][:,0])
    Turn            = average_precision_score(target_array[val_pred_index][:,1], 
                                              y_oof[val_pred_index][:,1])
    Walking         = average_precision_score(target_array[val_pred_index][:,2],
                                              y_oof[val_pred_index][:,2])

    map_score = np.mean([StartHesitation, Turn, Walking])
    # LOGGER.info(f"cv map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
    print(f"cv map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")

<br>

### <font color=maroon>ex184_tdcsfog_gru_Turn_Walking.ipynb</font>

<br>

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/exp/ex184_tdcsfog_gru_Turn_Walking.ipynb" style="text-decoration:none">ex184_tdcsfog_gru_Turn_Walking.ipynb</a>

In [51]:
debug = False
ex = "184_tdcsfog"
if not os.path.exists(f"./output/exp/ex{ex}"):
    os.makedirs(f"./output/exp/ex{ex}")
    os.makedirs(f"./output/exp/ex{ex}/ex{ex}_model")

# logger_path = f"./output/exp/ex{ex}/ex_{ex}.txt"
# LOGGER = init_logger(log_file=logger_path)

<br>

#### main

In [52]:
%%time

# with timer("gru"):
if TRAIN_FLAG:
    set_seed(seed)
    y_oof = np.empty([len(target_array), 1000, 3])      # Abrachan: out of fold
    gkf = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state = seed)
    for fold, (train_idx, valid_idx) in enumerate(gkf.split(numerical_array, 
                                                            y = df_id["group"].values,
                                                            groups=df_id["subject"].values)):
        # LOGGER.info(f"start fold:{fold+1}")
        print(f"start fold: {fold+1}")
        
        with timer(f"fold {fold+1}"):
            train_numerical_array = numerical_array[train_idx]
            train_target_array = target_array[train_idx]
            train_mask_array = mask_array[train_idx]

            val_numerical_array = numerical_array[valid_idx]
            val_target_array = target_array[valid_idx]
            val_mask_array = mask_array[valid_idx]
            val_pred_array = pred_use_array[valid_idx]
            
            train_ = FogDataset(train_numerical_array,
                                train_mask_array,
                                train=True,
                                y=train_target_array)
            val_ = FogDataset(val_numerical_array,
                              val_mask_array,
                              train=True,
                              y=val_target_array)
            
            train_loader = DataLoader(dataset=train_, 
                                      batch_size=batch_size, 
                                      shuffle = True, 
                                      # num_workers=8,
                                     )
            val_loader = DataLoader(dataset=val_, 
                                    batch_size=batch_size, 
                                    shuffle = False , 
                                    # num_workers=8
                                   )
            
            
            model = FogRnnModel()
            model = model.to(device)
            
            param_optimizer = list(model.named_parameters())
            no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
            optimizer_grouped_parameters = [
                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 
                 'weight_decay': weight_decay
                },
                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 
                 'weight_decay': 0.0
                }]
            optimizer = AdamW(optimizer_grouped_parameters,
                              lr=lr,
                              weight_decay=weight_decay,
                              )
            num_train_optimization_steps = int(len(train_loader) * n_epochs)
            scheduler = get_linear_schedule_with_warmup(optimizer,
                                                        num_warmup_steps=num_warmup_steps,
                                                        num_training_steps=num_train_optimization_steps)
            criterion = nn.BCEWithLogitsLoss()
            best_val_score = 0
            
            for epoch in range(n_epochs):
                model.train() 
                train_losses_batch = []
                val_losses_batch = []
                epoch_loss = 0
                train_preds = np.ndarray((0,3))   # array([], shape=(0, 3), dtype=float64)
                
                tk0 = tqdm_nb(train_loader, total=len(train_loader), desc="train_loader: ")
                for d in tk0:
                    # ======================================================
                    # data loader
                    # ======================================================
                    input_data_numerical_array = d['input_data_numerical_array'].to(device)
                    input_data_mask_array      = d['input_data_mask_array'].to(device)
                    attention_mask             = d['attention_mask'].to(device)
                    y                          = d["y"].to(device)
                    
                    optimizer.zero_grad()
                    output = model(input_data_numerical_array, 
                                   input_data_mask_array,
                                   attention_mask)
                    # loss = criterion(output[input_data_mask_array == 1], y[input_data_mask_array == 1])
                    
                    output1 = output[:, :, [1,2]]
                    output2 = output[:, :, 0]
                    y1 = y[:, :, [1,2]]
                    y2 = y[:, :, 0]
                    loss1 = criterion(output1[input_data_mask_array == 1], y1[input_data_mask_array == 1])
                    loss2 = criterion(output2[input_data_mask_array == 1], y2[input_data_mask_array == 1])
                    
                    loss = loss1*0.8 + loss2*0.2
                    loss.backward()
                    optimizer.step()
                    scheduler.step()
                    train_losses_batch.append(loss.item())
                train_loss = np.mean(train_losses_batch)
                
                
                # ======================================================
                # eval
                # ======================================================
                model.eval()       # switch model to the evaluation mode
                
                val_preds = np.ndarray((0, 1000, 3))
                tk0 = tqdm_nb(val_loader, total=len(val_loader), desc="val_loader  : ")
                with torch.no_grad():  # Do not calculate gradient since we are only predicting
                    # Predicting on validation set
                    for d in tk0:
                        input_data_numerical_array = d['input_data_numerical_array'].to(device)
                        input_data_mask_array      = d['input_data_mask_array'].to(device)
                        attention_mask             = d['attention_mask'].to(device)
                        
                        output = model(input_data_numerical_array, 
                                       input_data_mask_array,
                                       attention_mask)
                        val_preds = np.concatenate([val_preds, output.sigmoid().detach().cpu().numpy()], axis=0)
                
                pred_valid_index = (val_mask_array == 1) & (val_pred_array == 1)
                StartHesitation = average_precision_score(val_target_array[pred_valid_index][:, 0],
                                                          val_preds[pred_valid_index][:, 0])
                Turn = average_precision_score(val_target_array[pred_valid_index][:, 1],
                                               val_preds[pred_valid_index][:, 1])
                Walking = average_precision_score(val_target_array[pred_valid_index][:, 2],
                                                  val_preds[pred_valid_index][:, 2])
                map_score = np.mean([StartHesitation, Turn, Walking])
                
                # LOGGER.info(f"fold: {fold+1} epoch: {epoch+1},train loss {train_loss} map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                print(f"fold {fold+1} epoch {epoch+1}\ntrain loss {train_loss} map:{map_score}\nstart_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                
                if map_score >= best_val_score:
                    print("save weight")
                    best_val_score = map_score
                    best_val_preds = val_preds.copy()
                    torch.save(model.state_dict(), f"./output/exp/ex{ex}/ex{ex}_model/ex{ex}_fold{fold+1}_epoch{epoch+1}__map{best_val_score:.6f}.pth")
                print()
            y_oof[valid_idx] = best_val_preds
    
    np.save(f"./output/exp/ex{ex}/ex{ex}_oof.npy", y_oof)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.58 µs


<br>

#### oof score

In [53]:
if TRAIN_FLAG:
    val_pred_index = (mask_array == 1) & (pred_use_array == 1)
    StartHesitation = average_precision_score(target_array[val_pred_index][:,0], 
                                              y_oof[val_pred_index][:,0])
    Turn            = average_precision_score(target_array[val_pred_index][:,1], 
                                              y_oof[val_pred_index][:,1])
    Walking         = average_precision_score(target_array[val_pred_index][:,2],
                                              y_oof[val_pred_index][:,2])

    map_score = np.mean([StartHesitation, Turn, Walking])
    # LOGGER.info(f"cv map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
    print(f"cv map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")

<br>

## **models choosed**

<br>

<font color=maroon size=5>All models above were used for final submission.</font>

<br>
<br>
<br>

# **defog** 【Training】

<br>

## **Feature Engineering**


### fe039_defog_base_feature

<br>

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/fe/fe039_defog_base_feature.ipynb" style="text-decoration:none">fe039_defog_base_feature.ipynb</a>

In [54]:
fe = "039"
if not os.path.exists(f"./output/fe/fe{fe}"):
    os.makedirs(f"./output/fe/fe{fe}")
    os.makedirs(f"./output/fe/fe{fe}/save")

In [55]:
sub_dict = {}
for n,i in enumerate(defog_metadata["Subject"].unique()):
    sub_dict[i] = n

In [56]:
defog_metadata["sub_id"] = defog_metadata["Subject"].map(sub_dict)
defog_metadata

,Id,Subject,Visit,Medication,sub_id
0,02ab235146,e1f62e,2,on,0
1,02ea782681,ae2d35,2,on,1
2,06414383cf,8c1f5e,2,off,2
3,092b4c1819,2874c5,1,off,3
4,0a900ed8a2,0e3d49,2,on,4
...,...,...,...,...,...
132,f3a921edee,1a778d,1,off,8
133,f40e8c6ebe,575c60,1,off,38
134,f8ddbdd98d,107712,1,on,39
135,f9efef91fb,5d9cae,2,off,44


In [57]:
with open(f'./output/fe/fe{fe}/fe{fe}_sub_id.pkl', 'wb') as p:
    pickle.dump(sub_dict, p)

In [58]:
# DEFOG_META_PATH = "../data/defog_metadata.csv"
# DEFOG_FOLDER = "../data/train/defog/*.csv"

# meta = pd.read_csv(DEFOG_META_PATH)
# data_list = glob.glob(DEFOG_FOLDER)

In [59]:
df_all = []
for i in tqdm(train_defog):
    df = pd.read_csv(i)
    df_all.append(df)
df_all = pd.concat(df_all).reset_index(drop=True)

# len(df_all)

100%|██████████| 91/91 [00:21<00:00,  4.24it/s]


In [60]:
df_all

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Valid,Task
0,0,-1.002697,0.022371,0.068304,0,0,0,False,False
1,1,-1.002641,0.019173,0.066162,0,0,0,False,False
2,2,-0.999820,0.019142,0.067536,0,0,0,False,False
3,3,-0.998023,0.018378,0.068409,0,0,0,False,False
4,4,-0.998359,0.016726,0.066448,0,0,0,False,False
...,...,...,...,...,...,...,...,...,...
13525697,109120,-0.939241,0.031564,-0.394737,0,0,0,False,False
13525698,109121,-0.941096,0.031582,-0.392626,0,0,0,False,False
13525699,109122,-0.940131,0.029092,-0.394385,0,0,0,False,False
13525700,109123,-0.939872,0.028058,-0.398664,0,0,0,False,False


In [61]:
mean_std_dict = {}
for c in ["AccV", "AccML", "AccAP"]:
    mean = df_all[c].mean()
    std = df_all[c].std()
    mean_std_dict[c] = [mean,std]
    print(c, mean, std)

AccV -0.9401728865187444 0.0860651354523073
AccML 0.0011727847170219124 0.12000798550797488
AccAP -0.13061518435798145 0.28238873112037965


In [62]:
with open(f'./output/fe/fe{fe}/save/fe{fe}_sc.pkl', 'wb') as p:
    pickle.dump(mean_std_dict, p)

In [63]:
d_list = []
num_array = []
target_array = []
valid_array = []
subject_list = []
id_list = []
mask_array = []
num_cols = ["AccV", "AccML", "AccAP"]
target_cols = ["StartHesitation", "Turn", "Walking"]
seq_len = 1000

for i,s in tqdm(zip(defog_metadata["Id"].values, defog_metadata["sub_id"].values), desc="defog_metadata: "):
    path = root_data + f"train/defog/{i}.csv"
    if path in [x.replace("\\", "/") for x in train_defog]:
    # if path in data_list:
        d_list.append(1)
        
        df = pd.read_csv(path)
        df["valid"] = df["Valid"] & df["Task"]
        df["valid"] = df["valid"].astype(int)
        
        batch = (len(df) // seq_len) + 1
        
        for c in num_cols:
            df[c] = (df[c] - mean_std_dict[c][0]) / mean_std_dict[c][1]
        num = df[num_cols].values
        target = df[target_cols].values
        valid = df["valid"].values
        
        num_array_    = np.zeros([batch, seq_len, 3])
        target_array_ = np.zeros([batch, seq_len, 3])
        mask_array_   = np.zeros([batch, seq_len])
        valid_array_  = np.zeros([batch, seq_len])
        
        for n,b in enumerate(range(batch)):
            if b == (batch - 1):
                num_ = num[b*seq_len : ]
                num_array_[b, :len(num_), :] = num_
                target_ = target[b*seq_len : ]
                target_array_[b,:len(target_), :] = target_
                valid_ = valid[b*seq_len : ]
                valid_array_[b, :len(valid_)] = valid_
                mask_array_[b, :len(target_)] = 1
            else:
                num_ = num[b*seq_len : (b+1)*seq_len]
                num_array_[b, :, :] = num_
                target_ = target[b*seq_len : (b+1)*seq_len]
                target_array_[b, :, :] = target_
                valid_ = valid[b*seq_len : (b+1)*seq_len]
                valid_array_[b, :] = valid_
                mask_array_[b,:] = 1
        num_array.append(num_array_)
        target_array.append(target_array_)
        mask_array.append(mask_array_)
        valid_array.append(valid_array_)
        subject_list += [s for _ in range(batch)]
        id_list      += [i for _ in range(batch)] 
    else:
        d_list.append(0)

defog_metadata: : 137it [00:15,  9.00it/s]


In [64]:
num_array    = np.concatenate(num_array, axis=0)
target_array = np.concatenate(target_array, axis=0)
mask_array   =  np.concatenate(mask_array, axis=0)
valid_array = np.concatenate(valid_array, axis=0)

np.save(f"./output/fe/fe{fe}/fe{fe}_num_array.npy", num_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_target_array.npy", target_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_mask_array.npy", mask_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_valid_array.npy", valid_array)

In [65]:
defog_metadata["data_is"] = d_list

defog_metadata.to_parquet(f"./output/fe/fe{fe}/fe{fe}_defog_meta.parquet")

In [66]:
df_id = pd.DataFrame()
df_id["Id"] = id_list
df_id["subject"] = subject_list

df_id.to_parquet(f"./output/fe/fe{fe}/fe{fe}_id.parquet")

<br>

### fe047_defog_5000

<br>

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/fe/fe047_defog_5000.ipynb" style="text-decoration:none">fe047_defog_5000.ipynb</a>

In [67]:
fe = "047"
if not os.path.exists(f"./output/fe/fe{fe}"):
    os.makedirs(f"./output/fe/fe{fe}")
    os.makedirs(f"./output/fe/fe{fe}/save")

In [68]:
cols = ["AccV", "AccML", "AccAP"]
num_cols = ["AccV", "AccML", "AccAP",
            'AccV_lag_diff',  'AccV_lead_diff', 
            'AccML_lag_diff', 'AccML_lead_diff', 
            'AccAP_lag_diff', 'AccAP_lead_diff']
target_cols = ["StartHesitation", "Turn", "Walking"]

seq_len = 5000
shift = 2500
offset = 1250

In [69]:
num_array = []
target_array = []
valid_array = []
mask_array = []
pred_use_array = []
time_array = []

subject_list = []
id_list = []
d_list = []

In [70]:
# data_list = glob.glob(DEFOG_FOLDER)

from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler

for i,s in tqdm(zip(defog_metadata["Id"].values, defog_metadata["sub_id"].values), desc="defog_metadata: "):
    path = root_data + f"train/defog/{i}.csv"
    if path in [x.replace("\\", "/") for x in train_defog]:
    # if path in data_list:
        d_list.append(1)
        
        df = pd.read_csv(path)
        df["valid"] = df["Valid"] & df["Task"]
        df["valid"] = df["valid"].astype(int)
        
        batch = (len(df)-1) // shift
        
        for c in cols:
            df[f"{c}_lag_diff"] = df[c].diff()
            df[f"{c}_lead_diff"] = df[c].diff(-1)
        
        sc = StandardScaler()
        df[num_cols] = sc.fit_transform(df[num_cols].values)
        df[num_cols] = df[num_cols].fillna(0)

        num = df[num_cols].values
        target = df[target_cols].values
        valid = df["valid"].values
        time_values = df["Time"].values
        
        num_array_ = np.zeros([batch,seq_len, 9])
        target_array_ = np.zeros([batch, seq_len, 3])
        valid_array_ = np.zeros([batch, seq_len], dtype=int)
        time_array_ = np.zeros([batch, seq_len], dtype=int)
        mask_array_ = np.zeros([batch, seq_len], dtype=int)
        pred_use_array_ = np.zeros([batch, seq_len], dtype=int)
        for n,b in enumerate(range(batch)):
            if b == (batch - 1):
                num_ = num[b*shift : ]
                num_array_[b,:len(num_), :] = num_
                target_ = target[b*shift : ]
                target_array_[b, :len(target_), :] = target_
                mask_array_[b, :len(target_)] = 1
                pred_use_array_[b, offset:len(target_)] = 1
                time_ = time_values[b*shift : ]
                time_array_[b, :len(time_)] = time_
                valid_ = valid[b*shift : ]
                valid_array_[b, :len(valid_)] = valid_
            elif b == 0:
                num_ = num[b*shift : b*shift+seq_len]
                num_array_[b, :, :] = num_
                target_ = target[b*shift : b*shift + seq_len]
                target_array_[b, :, :] = target_
                mask_array_[b, :] = 1
                pred_use_array_[b, :shift + offset] = 1
                time_ = time_values[b*shift : b*shift + seq_len]
                time_array_[b, :] = time_
                valid_ = valid[b*shift : b*shift + seq_len]
                valid_array_[b, :] = valid_
            else:
                num_ = num[b*shift : b*shift+seq_len]
                num_array_[b, :, :] = num_
                target_ = target[b*shift : b*shift + seq_len]
                target_array_[b, :, :] = target_
                mask_array_[b, :] = 1
                pred_use_array_[b, offset:shift + offset] = 1
                time_ = time_values[b*shift : b*shift + seq_len]
                time_array_[b, :] = time_
                valid_ = valid[b*shift : b*shift + seq_len]
                valid_array_[b, :] = valid_

        num_array.append(num_array_)
        target_array.append(target_array_)
        mask_array.append(mask_array_)
        pred_use_array.append(pred_use_array_)
        time_array.append(time_array_)
        valid_array.append(valid_array_)
        subject_list += [s for _ in range(batch)]
        id_list      += [i for _ in range(batch)] 
    else:
        d_list.append(0)

defog_metadata: : 137it [00:23,  5.84it/s]


In [71]:
num_array = np.concatenate(num_array, axis=0)
target_array =np.concatenate(target_array, axis=0)
mask_array =  np.concatenate(mask_array, axis=0)
pred_use_array = np.concatenate(pred_use_array, axis=0)
time_array = np.concatenate(time_array, axis=0)
valid_array = np.concatenate(valid_array, axis=0)

np.save(f"./output/fe/fe{fe}/fe{fe}_num_array.npy", num_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_target_array.npy", target_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_mask_array.npy", mask_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_time_array.npy", time_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_pred_use_array.npy", pred_use_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_valid_array.npy", valid_array)

In [72]:
df_id = pd.DataFrame()
df_id["Id"] = id_list
df_id["subject"] = subject_list

df_id.to_parquet(f"./output/fe/fe{fe}/fe{fe}_id.parquet")

<br>

## <font color=red>**Training**</font>


<br>

### helpers

In [73]:
TRAIN_FLAG_DEFOG = True

<br>

#### def `set_seed()`

In [74]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

<br>

#### def `timer()`

In [75]:
@contextmanager
def timer(name):
    t0 = time.time()
    
    yield 
    # LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s')
    print(f'[{name}] done in {time.time() - t0:.0f} s')
    print("="*66)
    print("\n"*2)
    
# setup_logger(out_file=logger_path)

<br>

#### def `preprocess()`

In [76]:
def preprocess(numerical_array, mask_array, valid_array,):
    
    attention_mask = mask_array == 0

    return {'input_data_numerical_array': numerical_array,
            'input_data_mask_array': mask_array,
            'input_data_valid_array': valid_array,
            'attention_mask': attention_mask,
           }

<br>

#### class `FogDataset()`

In [77]:
class FogDataset(Dataset):
    def __init__(self, numerical_array, mask_array, valid_array, train = True, y = None):
        self.numerical_array = numerical_array
        self.mask_array = mask_array
        self.valid_array = valid_array
        self.train = train
        self.y = y
    
    
    def __len__(self):
        return len(self.numerical_array)
    
    
    def __getitem__(self, item):
        data = preprocess(self.numerical_array[item], self.mask_array[item], self.valid_array[item],)

        # Return the processed data where the lists are converted to `torch.tensor`s
        if self.train : 
            return {
              'input_data_numerical_array': torch.tensor(data['input_data_numerical_array'], dtype=torch.float32),
              'input_data_mask_array': torch.tensor(data['input_data_mask_array'],  dtype=torch.long),  
              'input_data_valid_array': torch.tensor(data['input_data_valid_array'], dtype=torch.long),   
              'attention_mask': torch.tensor(data["attention_mask"], dtype=torch.bool),
              "y": torch.tensor(self.y[item], dtype=torch.float32)
            }
        else:
            return {
              'input_data_numerical_array': torch.tensor(data['input_data_numerical_array'], dtype=torch.float32),
              'input_data_mask_array': torch.tensor(data['input_data_mask_array'], dtype=torch.long),  
              'input_data_valid_array': torch.tensor(data['input_data_valid_array'], dtype=torch.long),  
              'attention_mask': torch.tensor(data["attention_mask"], dtype=torch.bool),
               }

<br>

#### class `FogRnnModel()`

In [78]:
class FogRnnModel(nn.Module):
    def __init__(self, 
                 dropout=0.2,
                 input_numerical_size=9,
                 numeraical_linear_size = 64,
                 model_size = 128,
                 linear_out = 128,
                 out_size=3):
        
        super(FogRnnModel, self).__init__()
        
        self.numerical_linear  = nn.Sequential(nn.Linear(input_numerical_size, numeraical_linear_size),
                                               nn.LayerNorm(numeraical_linear_size))
        
        self.rnn = nn.GRU(numeraical_linear_size, 
                          model_size,
                          num_layers = 2, 
                          batch_first=True,
                          bidirectional=True)
        
        self.linear_out  = nn.Sequential(nn.Linear(model_size*2, linear_out),
                                         nn.LayerNorm(linear_out),
                                         nn.ReLU(),
                                         nn.Dropout(dropout),
                                         nn.Linear(linear_out, out_size))
        self._reinitialize()
        
        
        
    def _reinitialize(self):
        """Tensorflow/Keras-like initialization"""
        for name, p in self.named_parameters():
            if 'rnn' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
        
        
        
    def forward(self, numerical_array, mask_array, attention_mask):
        numerical_embedding = self.numerical_linear(numerical_array)
        output, _ = self.rnn(numerical_embedding)
        output = self.linear_out(output)
        return output

<br>

### load data & preprocessing

In [79]:
id_path        = f"./output/fe/fe047/fe047_id.parquet"
numerical_path = f"./output/fe/fe047/fe047_num_array.npy"
target_path    = f"./output/fe/fe047/fe047_target_array.npy"
mask_path      = f"./output/fe/fe047/fe047_mask_array.npy"
valid_path     = f"./output/fe/fe047/fe047_valid_array.npy"
pred_use_path  = f"./output/fe/fe047/fe047_pred_use_array.npy"

In [80]:
df_id = pd.read_parquet(id_path)
numerical_array = np.load(numerical_path)
target_array = np.load(target_path)
mask_array = np.load(mask_path)
valid_array = np.load(valid_path)
pred_use_array = np.load(pred_use_path)

target1 = []
target2 = []
target3 = []
for i in range(len(target_array)):
    target1.append(np.sum(target_array[i,:,0]))
    target2.append(np.sum(target_array[i,:,1]))
    target3.append(np.sum(target_array[i,:,2]))


df_id["target1"] = target1
df_id["target2"] = target2
df_id["target3"] = target3
df_id["target1_1"] = df_id["target1"] > 0
df_id["target2_1"] = df_id["target2"] > 0
df_id["target3_1"] = df_id["target3"] > 0
df_id["target1_1"] = df_id["target1_1"].astype(np.int)
df_id["target2_1"] = df_id["target2_1"].astype(np.int)
df_id["target3_1"] = df_id["target3_1"].astype(np.int)

df_id["group"] = 0
df_id.loc[df_id["target1_1"] > 0,"group"] = 1
df_id.loc[df_id["target2_1"] > 0,"group"] = 2
df_id.loc[df_id["target3_1"] > 0,"group"] = 3


df_id["group"].value_counts()

0    4107
2     897
3     354
1      11
Name: group, dtype: int64

<br>

### config

In [81]:
# config
seed = 0
shuffle = True
n_splits = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model config
batch_size = 24
n_epochs = 15
lr = 1e-3
weight_decay = 0.05
num_warmup_steps = 10

<br>

### <font color=maroon>ex153_defog_gru.ipynb</font>

<br>

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/exp/ex153_defog_gru.ipynb" style="text-decoration:none">ex153_defog_gru.ipynb</a>

In [82]:
debug = False
ex = "153_defog"
if not os.path.exists(f"./output/exp/ex{ex}"):
    os.makedirs(f"./output/exp/ex{ex}")
    os.makedirs(f"./output/exp/ex{ex}/ex{ex}_model")

<br>

#### main

In [83]:
%%time

# with timer("gru"):
if TRAIN_FLAG:
    set_seed(seed)
    y_oof = np.empty([len(target_array), 5000, 3])      # Abrachan: out of fold
    gkf = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state = seed)
    for fold, (train_idx, valid_idx) in enumerate(gkf.split(numerical_array, 
                                                            y = df_id["group"].values,
                                                            groups=df_id["subject"].values)):
        # LOGGER.info(f"start fold:{fold+1}")
        print(f"start fold: {fold+1}")
        
        with timer(f"fold {fold+1}"):
            train_numerical_array = numerical_array[train_idx]
            train_target_array = target_array[train_idx]
            train_mask_array = mask_array[train_idx]
            train_valid_array = valid_array[train_idx]

            val_numerical_array = numerical_array[valid_idx]
            val_target_array = target_array[valid_idx]
            val_mask_array = mask_array[valid_idx]
            val_valid_array = valid_array[valid_idx]
            val_pred_array = pred_use_array[valid_idx]
            
            train_ = FogDataset(train_numerical_array,
                                train_mask_array,
                                train_valid_array,
                                train=True,
                                y=train_target_array)
            val_ = FogDataset(val_numerical_array,
                              val_mask_array,
                              val_valid_array,
                              train=True,
                              y=val_target_array)
            
            train_loader = DataLoader(dataset=train_, 
                                      batch_size=batch_size, 
                                      shuffle = True, 
                                      # num_workers=8,
                                     )
            val_loader = DataLoader(dataset=val_, 
                                    batch_size=batch_size, 
                                    shuffle = False , 
                                    # num_workers=8
                                   )
            
            
            model = FogRnnModel()
            model = model.to(device)
            
            param_optimizer = list(model.named_parameters())
            no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
            optimizer_grouped_parameters = [
                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 
                 'weight_decay': weight_decay
                },
                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 
                 'weight_decay': 0.0
                }]
            optimizer = AdamW(optimizer_grouped_parameters,
                              lr=lr,
                              weight_decay=weight_decay,
                              )
            num_train_optimization_steps = int(len(train_loader) * n_epochs)
            scheduler = get_linear_schedule_with_warmup(optimizer,
                                                        num_warmup_steps=num_warmup_steps,
                                                        num_training_steps=num_train_optimization_steps)
            criterion = nn.BCEWithLogitsLoss()
            best_val_score = 0
            
            for epoch in range(n_epochs):
                model.train() 
                train_losses_batch = []
                val_losses_batch = []
                epoch_loss = 0
                train_preds = np.ndarray((0,3))   # array([], shape=(0, 3), dtype=float64)
                
                tk0 = tqdm_nb(train_loader, total=len(train_loader), desc="train_loader: ")
                for d in tk0:
                    # ======================================================
                    # data loader
                    # ======================================================
                    input_data_numerical_array = d['input_data_numerical_array'].to(device)
                    input_data_mask_array      = d['input_data_mask_array'].to(device)
                    input_data_valid_array     = d['input_data_valid_array'].to(device)
                    attention_mask             = d['attention_mask'].to(device)
                    y                          = d["y"].to(device)
                    
                    optimizer.zero_grad()
                    output = model(input_data_numerical_array, 
                                   input_data_mask_array,
                                   attention_mask)
                    loss = criterion(output[input_data_mask_array == 1], y[input_data_mask_array == 1])
                    
                    # output1 = output[:, :, [1,2]]
                    # output2 = output[:, :, 0]
                    # y1 = y[:, :, [1,2]]
                    # y2 = y[:, :, 0]
                    # loss1 = criterion(output1[input_data_mask_array == 1], y1[input_data_mask_array == 1])
                    # loss2 = criterion(output2[input_data_mask_array == 1], y2[input_data_mask_array == 1])
                    # loss = loss1*0.8 + loss2*0.2
                    
                    loss.backward()
                    optimizer.step()
                    scheduler.step()
                    train_losses_batch.append(loss.item())
                train_loss = np.mean(train_losses_batch)
                
                
                # ======================================================
                # eval
                # ======================================================
                model.eval()       # switch model to the evaluation mode
                
                val_preds = np.ndarray((0, 5000, 3))
                tk0 = tqdm_nb(val_loader, total=len(val_loader), desc="val_loader  : ")
                with torch.no_grad():  # Do not calculate gradient since we are only predicting
                    # Predicting on validation set
                    for d in tk0:
                        input_data_numerical_array = d['input_data_numerical_array'].to(device)
                        input_data_mask_array      = d['input_data_mask_array'].to(device)
                        attention_mask             = d['attention_mask'].to(device)
                        
                        output = model(input_data_numerical_array, 
                                       input_data_mask_array,
                                       attention_mask)
                        val_preds = np.concatenate([val_preds, output.sigmoid().detach().cpu().numpy()], axis=0)
                
                pred_valid_index = (val_mask_array == 1) & (val_pred_array == 1) & (val_valid_array == 1)
                StartHesitation = average_precision_score(val_target_array[pred_valid_index][:, 0],
                                                          val_preds[pred_valid_index][:, 0])
                Turn = average_precision_score(val_target_array[pred_valid_index][:, 1],
                                               val_preds[pred_valid_index][:, 1])
                Walking = average_precision_score(val_target_array[pred_valid_index][:, 2],
                                                  val_preds[pred_valid_index][:, 2])
                # map_score = np.mean([StartHesitation, Turn, Walking])
                map_score = np.mean([Turn, Walking])
                
                # LOGGER.info(f"fold: {fold+1} epoch: {epoch+1},train loss {train_loss} map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                print(f"fold {fold+1} epoch {epoch+1}\ntrain loss {train_loss} map:{map_score}\nstart_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                
                if map_score >= best_val_score:
                    print("save weight")
                    best_val_score = map_score
                    best_val_preds = val_preds.copy()
                    torch.save(model.state_dict(), f"./output/exp/ex{ex}/ex{ex}_model/ex{ex}_fold{fold+1}_epoch{epoch+1}__map{best_val_score:.6f}.pth")
                print()
            y_oof[valid_idx] = best_val_preds
    
    np.save(f"./output/exp/ex{ex}/ex{ex}_oof.npy", y_oof)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.06 µs


<br>

#### oof score

In [84]:
if TRAIN_FLAG:
    val_pred_index = (mask_array == 1) & (pred_use_array == 1) & (valid_array == 1)
    StartHesitation = average_precision_score(target_array[val_pred_index][:,0], 
                                              y_oof[val_pred_index][:,0])
    Turn            = average_precision_score(target_array[val_pred_index][:,1], 
                                              y_oof[val_pred_index][:,1])
    Walking         = average_precision_score(target_array[val_pred_index][:,2],
                                              y_oof[val_pred_index][:,2])

    map_score = np.mean([StartHesitation, Turn, Walking])
    # LOGGER.info(f"cv map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
    print(f"cv map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")

<br>

### <font color=maroon>ex154_defog_gru.ipynb</font>

<br>

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/exp/ex154_defog_gru.ipynb" style="text-decoration:none">ex154_defog_gru.ipynb</a>

In [85]:
debug = False
ex = "154_defog"
if not os.path.exists(f"./output/exp/ex{ex}"):
    os.makedirs(f"./output/exp/ex{ex}")
    os.makedirs(f"./output/exp/ex{ex}/ex{ex}_model")

<br>

#### class `FogRnnModel()`

In [86]:
class FogRnnModel(nn.Module):
    def __init__(self, 
                 dropout=0.2,
                 input_numerical_size=9,
                 numeraical_linear_size = 96,
                 model_size = 256,
                 linear_out = 256,
                 out_size=3):
        
        super(FogRnnModel, self).__init__()
        
        self.numerical_linear  = nn.Sequential(nn.Linear(input_numerical_size, numeraical_linear_size),
                                               nn.LayerNorm(numeraical_linear_size))
        
        self.rnn = nn.GRU(numeraical_linear_size, 
                          model_size,
                          num_layers = 2, 
                          batch_first=True,
                          bidirectional=True)
        
        self.linear_out  = nn.Sequential(nn.Linear(model_size*2, linear_out),
                                         nn.LayerNorm(linear_out),
                                         nn.ReLU(),
                                         nn.Dropout(dropout),
                                         nn.Linear(linear_out, out_size))
        self._reinitialize()
        
        
        
    def _reinitialize(self):
        """Tensorflow/Keras-like initialization"""
        for name, p in self.named_parameters():
            if 'rnn' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
        
        
        
    def forward(self, numerical_array, mask_array, attention_mask):
        numerical_embedding = self.numerical_linear(numerical_array)
        output, _ = self.rnn(numerical_embedding)
        output = self.linear_out(output)
        return output

<br>

#### main

In [87]:
%%time

# with timer("gru"):
if TRAIN_FLAG:
    set_seed(seed)
    y_oof = np.empty([len(target_array), 5000, 3])      # Abrachan: out of fold
    gkf = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state = seed)
    for fold, (train_idx, valid_idx) in enumerate(gkf.split(numerical_array, 
                                                            y = df_id["group"].values,
                                                            groups=df_id["subject"].values)):
        # LOGGER.info(f"start fold:{fold+1}")
        print(f"start fold: {fold+1}")
        
        with timer(f"fold {fold+1}"):
            train_numerical_array = numerical_array[train_idx]
            train_target_array = target_array[train_idx]
            train_mask_array = mask_array[train_idx]
            train_valid_array = valid_array[train_idx]

            val_numerical_array = numerical_array[valid_idx]
            val_target_array = target_array[valid_idx]
            val_mask_array = mask_array[valid_idx]
            val_valid_array = valid_array[valid_idx]
            val_pred_array = pred_use_array[valid_idx]
            
            train_ = FogDataset(train_numerical_array,
                                train_mask_array,
                                train_valid_array,
                                train=True,
                                y=train_target_array)
            val_ = FogDataset(val_numerical_array,
                              val_mask_array,
                              val_valid_array,
                              train=True,
                              y=val_target_array)
            
            train_loader = DataLoader(dataset=train_, 
                                      batch_size=batch_size, 
                                      shuffle = True, 
                                      # num_workers=8,
                                     )
            val_loader = DataLoader(dataset=val_, 
                                    batch_size=batch_size, 
                                    shuffle = False , 
                                    # num_workers=8
                                   )
            
            
            model = FogRnnModel()
            model = model.to(device)
            
            param_optimizer = list(model.named_parameters())
            no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
            optimizer_grouped_parameters = [
                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 
                 'weight_decay': weight_decay
                },
                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 
                 'weight_decay': 0.0
                }]
            optimizer = AdamW(optimizer_grouped_parameters,
                              lr=lr,
                              weight_decay=weight_decay,
                              )
            num_train_optimization_steps = int(len(train_loader) * n_epochs)
            scheduler = get_linear_schedule_with_warmup(optimizer,
                                                        num_warmup_steps=num_warmup_steps,
                                                        num_training_steps=num_train_optimization_steps)
            criterion = nn.BCEWithLogitsLoss()
            best_val_score = 0
            
            for epoch in range(n_epochs):
                model.train() 
                train_losses_batch = []
                val_losses_batch = []
                epoch_loss = 0
                train_preds = np.ndarray((0,3))   # array([], shape=(0, 3), dtype=float64)
                
                tk0 = tqdm_nb(train_loader, total=len(train_loader), desc="train_loader: ")
                for d in tk0:
                    # ======================================================
                    # data loader
                    # ======================================================
                    input_data_numerical_array = d['input_data_numerical_array'].to(device)
                    input_data_mask_array      = d['input_data_mask_array'].to(device)
                    input_data_valid_array     = d['input_data_valid_array'].to(device)
                    attention_mask             = d['attention_mask'].to(device)
                    y                          = d["y"].to(device)
                    
                    optimizer.zero_grad()
                    output = model(input_data_numerical_array, 
                                   input_data_mask_array,
                                   attention_mask)
                    loss = criterion(output[input_data_mask_array == 1], y[input_data_mask_array == 1])
                    
                    # output1 = output[:, :, [1,2]]
                    # output2 = output[:, :, 0]
                    # y1 = y[:, :, [1,2]]
                    # y2 = y[:, :, 0]
                    # loss1 = criterion(output1[input_data_mask_array == 1], y1[input_data_mask_array == 1])
                    # loss2 = criterion(output2[input_data_mask_array == 1], y2[input_data_mask_array == 1])
                    # loss = loss1*0.8 + loss2*0.2
                    
                    loss.backward()
                    optimizer.step()
                    scheduler.step()
                    train_losses_batch.append(loss.item())
                train_loss = np.mean(train_losses_batch)
                
                
                # ======================================================
                # eval
                # ======================================================
                model.eval()       # switch model to the evaluation mode
                
                val_preds = np.ndarray((0, 5000, 3))
                tk0 = tqdm_nb(val_loader, total=len(val_loader), desc="val_loader  : ")
                with torch.no_grad():  # Do not calculate gradient since we are only predicting
                    # Predicting on validation set
                    for d in tk0:
                        input_data_numerical_array = d['input_data_numerical_array'].to(device)
                        input_data_mask_array      = d['input_data_mask_array'].to(device)
                        attention_mask             = d['attention_mask'].to(device)
                        
                        output = model(input_data_numerical_array, 
                                       input_data_mask_array,
                                       attention_mask)
                        val_preds = np.concatenate([val_preds, output.sigmoid().detach().cpu().numpy()], axis=0)
                
                pred_valid_index = (val_mask_array == 1) & (val_pred_array == 1) & (val_valid_array == 1)
                StartHesitation = average_precision_score(val_target_array[pred_valid_index][:, 0],
                                                          val_preds[pred_valid_index][:, 0])
                Turn = average_precision_score(val_target_array[pred_valid_index][:, 1],
                                               val_preds[pred_valid_index][:, 1])
                Walking = average_precision_score(val_target_array[pred_valid_index][:, 2],
                                                  val_preds[pred_valid_index][:, 2])
                # map_score = np.mean([StartHesitation, Turn, Walking])
                map_score = np.mean([Turn, Walking])
                
                # LOGGER.info(f"fold: {fold+1} epoch: {epoch+1},train loss {train_loss} map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                print(f"fold {fold+1} epoch {epoch+1}\ntrain loss {train_loss} map:{map_score}\nstart_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                
                if map_score >= best_val_score:
                    print("save weight")
                    best_val_score = map_score
                    best_val_preds = val_preds.copy()
                    torch.save(model.state_dict(), f"./output/exp/ex{ex}/ex{ex}_model/ex{ex}_fold{fold+1}_epoch{epoch+1}__map{best_val_score:.6f}.pth")
                print()
            y_oof[valid_idx] = best_val_preds
    
    np.save(f"./output/exp/ex{ex}/ex{ex}_oof.npy", y_oof)

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 11.2 µs


<br>

#### oof score

In [88]:
if TRAIN_FLAG:
    val_pred_index = (mask_array == 1) & (pred_use_array == 1) & (valid_array == 1)
    StartHesitation = average_precision_score(target_array[val_pred_index][:,0], 
                                              y_oof[val_pred_index][:,0])
    Turn            = average_precision_score(target_array[val_pred_index][:,1], 
                                              y_oof[val_pred_index][:,1])
    Walking         = average_precision_score(target_array[val_pred_index][:,2],
                                              y_oof[val_pred_index][:,2])

    map_score = np.mean([StartHesitation, Turn, Walking])
    # LOGGER.info(f"cv map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
    print(f"cv map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")

<br>
<br>
<br>


# **notype** 【Training】


<br>

## **Feature Engineering**


<br>

### fe061_notype_5000

<br>

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/fe/fe061_notype_5000.ipynb" style="text-decoration:none">fe061_notype_5000.ipynb</a>

In [89]:
fe = "061_notype"
if not os.path.exists(f"./output/fe/fe{fe}"):
    os.makedirs(f"./output/fe/fe{fe}")
    os.makedirs(f"./output/fe/fe{fe}/save")

In [90]:
# DEFOG_META_PATH = "../data/defog_metadata.csv"
# DEFOG_FOLDER = "../data/train/notype/*.csv"

defog_meta = pd.read_parquet("./output/fe/fe039/fe039_defog_meta.parquet")

In [91]:
cols = ["AccV", "AccML", "AccAP"]
num_cols = ["AccV", "AccML",  "AccAP",
            'AccV_lag_diff',  'AccV_lead_diff',  
            'AccML_lag_diff', 'AccML_lead_diff', 
            'AccAP_lag_diff', 'AccAP_lead_diff']
target_cols = ["Event"]
seq_len = 5000
shift = 2500
offset = 1250

In [92]:
num_array = []
target_array = []
valid_array = []
mask_array = []
pred_use_array = []
time_array = []

subject_list = []
id_list = []
d_list = []

In [93]:
# data_list = glob.glob(DEFOG_FOLDER)

from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler

for i,s in tqdm(zip(defog_meta["Id"].values, defog_meta["sub_id"].values), desc="defog_meta: "):
    path = root_data + f"train/notype/{i}.csv"
    if path in [x.replace("\\", "/") for x in train_notype]:
    # if path in data_list:
        d_list.append(1)
        
        df = pd.read_csv(path)
        df["valid"] = df["Valid"] & df["Task"]
        df["valid"] = df["valid"].astype(int)
        
        batch = (len(df)-1) // shift
        
        for c in cols:
            df[f"{c}_lag_diff"] = df[c].diff()
            df[f"{c}_lead_diff"] = df[c].diff(-1)
        
        sc = StandardScaler()
        df[num_cols] = sc.fit_transform(df[num_cols].values)
        df[num_cols] = df[num_cols].fillna(0)

        num = df[num_cols].values
        target = df[target_cols].values
        valid = df["valid"].values
        time_values = df["Time"].values
        
        num_array_ = np.zeros([batch,seq_len, 9])
        target_array_ = np.zeros([batch, seq_len, 1])
        valid_array_ = np.zeros([batch, seq_len], dtype=int)
        time_array_ = np.zeros([batch, seq_len], dtype=int)
        mask_array_ = np.zeros([batch, seq_len], dtype=int)
        pred_use_array_ = np.zeros([batch, seq_len], dtype=int)
        for n,b in enumerate(range(batch)):
            if b == (batch - 1):
                num_ = num[b*shift : ]
                num_array_[b,:len(num_), :] = num_
                target_ = target[b*shift : ]
                target_array_[b, :len(target_), :] = target_
                mask_array_[b, :len(target_)] = 1
                pred_use_array_[b, offset:len(target_)] = 1
                time_ = time_values[b*shift : ]
                time_array_[b, :len(time_)] = time_
                valid_ = valid[b*shift : ]
                valid_array_[b, :len(valid_)] = valid_
            elif b == 0:
                num_ = num[b*shift : b*shift+seq_len]
                num_array_[b, :, :] = num_
                target_ = target[b*shift : b*shift + seq_len]
                target_array_[b, :, :] = target_
                mask_array_[b, :] = 1
                pred_use_array_[b, :shift + offset] = 1
                time_ = time_values[b*shift : b*shift + seq_len]
                time_array_[b, :] = time_
                valid_ = valid[b*shift : b*shift + seq_len]
                valid_array_[b, :] = valid_
            else:
                num_ = num[b*shift : b*shift+seq_len]
                num_array_[b, :, :] = num_
                target_ = target[b*shift : b*shift + seq_len]
                target_array_[b, :, :] = target_
                mask_array_[b, :] = 1
                pred_use_array_[b, offset:shift + offset] = 1
                time_ = time_values[b*shift : b*shift + seq_len]
                time_array_[b, :] = time_
                valid_ = valid[b*shift : b*shift + seq_len]
                valid_array_[b, :] = valid_

        num_array.append(num_array_)
        target_array.append(target_array_)
        mask_array.append(mask_array_)
        pred_use_array.append(pred_use_array_)
        time_array.append(time_array_)
        valid_array.append(valid_array_)
        subject_list += [s for _ in range(batch)]
        id_list      += [i for _ in range(batch)] 
    else:
        d_list.append(0)

defog_meta: : 137it [00:25,  5.46it/s]


In [94]:
num_array = np.concatenate(num_array, axis=0)
target_array =np.concatenate(target_array, axis=0)
mask_array =  np.concatenate(mask_array, axis=0)
pred_use_array = np.concatenate(pred_use_array, axis=0)
time_array = np.concatenate(time_array, axis=0)
valid_array = np.concatenate(valid_array, axis=0)

np.save(f"./output/fe/fe{fe}/fe{fe}_num_array.npy", num_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_target_array.npy", target_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_mask_array.npy", mask_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_time_array.npy", time_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_pred_use_array.npy", pred_use_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_valid_array.npy", valid_array)

In [95]:
df_id = pd.DataFrame()
df_id["Id"] = id_list
df_id["subject"] = subject_list

df_id["Id"].nunique()

46

In [96]:
df_id.to_parquet(f"./output/fe/fe{fe}/fe{fe}_id.parquet")

<br>

### fe064_notype_10000

<br>

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/fe/fe064_notype_10000.ipynb" style="text-decoration:none">fe064_notype_10000.ipynb</a>

In [97]:
fe = "064_notype"
if not os.path.exists(f"./output/fe/fe{fe}"):
    os.makedirs(f"./output/fe/fe{fe}")
    os.makedirs(f"./output/fe/fe{fe}/save")

In [98]:
# DEFOG_META_PATH = "../data/defog_metadata.csv"
# DEFOG_FOLDER = "../data/train/notype/*.csv"

defog_meta = pd.read_parquet("./output/fe/fe039/fe039_defog_meta.parquet")

In [99]:
cols = ["AccV","AccML","AccAP"]
num_cols = ["AccV", "AccML", "AccAP",
            'AccV_lag_diff', 'AccV_lead_diff', 
            'AccML_lag_diff', 'AccML_lead_diff',
            'AccAP_lag_diff', 'AccAP_lead_diff']
target_cols = ["Event"]
seq_len = 10000
shift = 5000
offset = 2500

In [100]:
num_array = []
target_array = []
valid_array = []
mask_array = []
pred_use_array = []
time_array = []

subject_list = []
id_list = []
d_list = []

In [101]:
# data_list = glob.glob(DEFOG_FOLDER)

from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler

for i,s in tqdm(zip(defog_meta["Id"].values, defog_meta["sub_id"].values), desc="defog_meta: "):
    path = root_data + f"train/notype/{i}.csv"
    if path in [x.replace("\\", "/") for x in train_notype]:
    # if path in data_list:
        d_list.append(1)
        
        df = pd.read_csv(path)
        df["valid"] = df["Valid"] & df["Task"]
        df["valid"] = df["valid"].astype(int)
        
        batch = (len(df)-1) // shift
        
        for c in cols:
            df[f"{c}_lag_diff"] = df[c].diff()
            df[f"{c}_lead_diff"] = df[c].diff(-1)
        
        sc = StandardScaler()
        df[num_cols] = sc.fit_transform(df[num_cols].values)
        df[num_cols] = df[num_cols].fillna(0)

        num = df[num_cols].values
        target = df[target_cols].values
        valid = df["valid"].values
        time_values = df["Time"].values
        
        num_array_ = np.zeros([batch,seq_len, 9])
        target_array_ = np.zeros([batch, seq_len, 1])
        valid_array_ = np.zeros([batch, seq_len], dtype=int)
        time_array_ = np.zeros([batch, seq_len], dtype=int)
        mask_array_ = np.zeros([batch, seq_len], dtype=int)
        pred_use_array_ = np.zeros([batch, seq_len], dtype=int)
        for n,b in enumerate(range(batch)):
            if b == (batch - 1):
                num_ = num[b*shift : ]
                num_array_[b,:len(num_), :] = num_
                target_ = target[b*shift : ]
                target_array_[b, :len(target_), :] = target_
                mask_array_[b, :len(target_)] = 1
                pred_use_array_[b, offset:len(target_)] = 1
                time_ = time_values[b*shift : ]
                time_array_[b, :len(time_)] = time_
                valid_ = valid[b*shift : ]
                valid_array_[b, :len(valid_)] = valid_
            elif b == 0:
                num_ = num[b*shift : b*shift+seq_len]
                num_array_[b, :, :] = num_
                target_ = target[b*shift : b*shift + seq_len]
                target_array_[b, :, :] = target_
                mask_array_[b, :] = 1
                pred_use_array_[b, :shift + offset] = 1
                time_ = time_values[b*shift : b*shift + seq_len]
                time_array_[b, :] = time_
                valid_ = valid[b*shift : b*shift + seq_len]
                valid_array_[b, :] = valid_
            else:
                num_ = num[b*shift : b*shift+seq_len]
                num_array_[b, :, :] = num_
                target_ = target[b*shift : b*shift + seq_len]
                target_array_[b, :, :] = target_
                mask_array_[b, :] = 1
                pred_use_array_[b, offset:shift + offset] = 1
                time_ = time_values[b*shift : b*shift + seq_len]
                time_array_[b, :] = time_
                valid_ = valid[b*shift : b*shift + seq_len]
                valid_array_[b, :] = valid_

        num_array.append(num_array_)
        target_array.append(target_array_)
        mask_array.append(mask_array_)
        pred_use_array.append(pred_use_array_)
        time_array.append(time_array_)
        valid_array.append(valid_array_)
        subject_list += [s for _ in range(batch)]
        id_list      += [i for _ in range(batch)] 
    else:
        d_list.append(0)

defog_meta: : 137it [00:22,  6.18it/s]


In [102]:
num_array = np.concatenate(num_array, axis=0)
target_array =np.concatenate(target_array, axis=0)
mask_array =  np.concatenate(mask_array, axis=0)
pred_use_array = np.concatenate(pred_use_array, axis=0)
time_array = np.concatenate(time_array, axis=0)
valid_array = np.concatenate(valid_array, axis=0)

np.save(f"./output/fe/fe{fe}/fe{fe}_num_array.npy", num_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_target_array.npy", target_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_mask_array.npy", mask_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_time_array.npy", time_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_pred_use_array.npy", pred_use_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_valid_array.npy", valid_array)

In [103]:
df_id = pd.DataFrame()
df_id["Id"] = id_list
df_id["subject"] = subject_list

df_id["Id"].nunique()

46

In [104]:
df_id.to_parquet(f"./output/fe/fe{fe}/fe{fe}_id.parquet")

<br>

### fe074_notype_15000

<br>

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/fe/fe074_notype_15000.ipynb" style="text-decoration:none">fe074_notype_15000.ipynb</a>

In [105]:
fe = "074_notype"
if not os.path.exists(f"./output/fe/fe{fe}"):
    os.makedirs(f"./output/fe/fe{fe}")
    os.makedirs(f"./output/fe/fe{fe}/save")

In [106]:
# DEFOG_META_PATH = "../data/defog_metadata.csv"
# DEFOG_FOLDER = "../data/train/notype/*.csv"

defog_meta = pd.read_parquet("./output/fe/fe039/fe039_defog_meta.parquet")

In [107]:
cols = ["AccV","AccML","AccAP"]
num_cols = ["AccV", "AccML", "AccAP",
            'AccV_lag_diff', 'AccV_lead_diff', 
            'AccML_lag_diff', 'AccML_lead_diff',
            'AccAP_lag_diff', 'AccAP_lead_diff']
target_cols = ["Event"]
seq_len = 15000
shift = 7500
offset = 3750

In [108]:
num_array = []
target_array = []
valid_array = []
mask_array = []
pred_use_array = []
time_array = []

subject_list = []
id_list = []
d_list = []

In [109]:
# data_list = glob.glob(DEFOG_FOLDER)

from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler

for i,s in tqdm(zip(defog_meta["Id"].values, defog_meta["sub_id"].values), desc="defog_meta: "):
    path = root_data + f"train/notype/{i}.csv"
    if path in [x.replace("\\", "/") for x in train_notype]:
    # if path in data_list:
        d_list.append(1)
        
        df = pd.read_csv(path)
        df["valid"] = df["Valid"] & df["Task"]
        df["valid"] = df["valid"].astype(int)
        
        batch = (len(df)-1) // shift
        
        for c in cols:
            df[f"{c}_lag_diff"] = df[c].diff()
            df[f"{c}_lead_diff"] = df[c].diff(-1)
        
        sc = StandardScaler()
        df[num_cols] = sc.fit_transform(df[num_cols].values)
        df[num_cols] = df[num_cols].fillna(0)

        num = df[num_cols].values
        target = df[target_cols].values
        valid = df["valid"].values
        time_values = df["Time"].values
        
        num_array_ = np.zeros([batch,seq_len, 9])
        target_array_ = np.zeros([batch, seq_len, 1])
        valid_array_ = np.zeros([batch, seq_len], dtype=int)
        time_array_ = np.zeros([batch, seq_len], dtype=int)
        mask_array_ = np.zeros([batch, seq_len], dtype=int)
        pred_use_array_ = np.zeros([batch, seq_len], dtype=int)
        for n,b in enumerate(range(batch)):
            if b == (batch - 1):
                num_ = num[b*shift : ]
                num_array_[b,:len(num_), :] = num_
                target_ = target[b*shift : ]
                target_array_[b, :len(target_), :] = target_
                mask_array_[b, :len(target_)] = 1
                pred_use_array_[b, offset:len(target_)] = 1
                time_ = time_values[b*shift : ]
                time_array_[b, :len(time_)] = time_
                valid_ = valid[b*shift : ]
                valid_array_[b, :len(valid_)] = valid_
            elif b == 0:
                num_ = num[b*shift : b*shift+seq_len]
                num_array_[b, :, :] = num_
                target_ = target[b*shift : b*shift + seq_len]
                target_array_[b, :, :] = target_
                mask_array_[b, :] = 1
                pred_use_array_[b, :shift + offset] = 1
                time_ = time_values[b*shift : b*shift + seq_len]
                time_array_[b, :] = time_
                valid_ = valid[b*shift : b*shift + seq_len]
                valid_array_[b, :] = valid_
            else:
                num_ = num[b*shift : b*shift+seq_len]
                num_array_[b, :, :] = num_
                target_ = target[b*shift : b*shift + seq_len]
                target_array_[b, :, :] = target_
                mask_array_[b, :] = 1
                pred_use_array_[b, offset:shift + offset] = 1
                time_ = time_values[b*shift : b*shift + seq_len]
                time_array_[b, :] = time_
                valid_ = valid[b*shift : b*shift + seq_len]
                valid_array_[b, :] = valid_

        num_array.append(num_array_)
        target_array.append(target_array_)
        mask_array.append(mask_array_)
        pred_use_array.append(pred_use_array_)
        time_array.append(time_array_)
        valid_array.append(valid_array_)
        subject_list += [s for _ in range(batch)]
        id_list      += [i for _ in range(batch)] 
    else:
        d_list.append(0)

defog_meta: : 137it [00:15,  8.63it/s]


In [110]:
num_array = np.concatenate(num_array, axis=0)
target_array =np.concatenate(target_array, axis=0)
mask_array =  np.concatenate(mask_array, axis=0)
pred_use_array = np.concatenate(pred_use_array, axis=0)
time_array = np.concatenate(time_array, axis=0)
valid_array = np.concatenate(valid_array, axis=0)

np.save(f"./output/fe/fe{fe}/fe{fe}_num_array.npy", num_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_target_array.npy", target_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_mask_array.npy", mask_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_time_array.npy", time_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_pred_use_array.npy", pred_use_array)
np.save(f"./output/fe/fe{fe}/fe{fe}_valid_array.npy", valid_array)

In [111]:
df_id = pd.DataFrame()
df_id["Id"] = id_list
df_id["subject"] = subject_list

df_id["Id"].nunique()

46

In [112]:
df_id.to_parquet(f"./output/fe/fe{fe}/fe{fe}_id.parquet")

<br>

## **Helpers**

<br>

### def `set_seed()`

In [113]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

<br>

### def `timer()`

In [114]:
@contextmanager
def timer(name):
    t0 = time.time()
    
    yield 
    # LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s')
    print(f'[{name}] done in {time.time() - t0:.0f} s')
    print("="*66)
    
# setup_logger(out_file=logger_path)

<br>

### def `preprocess()`

In [115]:
def preprocess(numerical_array, mask_array, valid_array,):
    
    attention_mask = mask_array == 0

    return {'input_data_numerical_array': numerical_array,
            'input_data_mask_array': mask_array,
            'input_data_valid_array': valid_array,
            'attention_mask': attention_mask,
           }

<br>

### class `FogDataset()`

In [116]:
class FogDataset(Dataset):
    def __init__(self, numerical_array, mask_array, valid_array, train = True, y = None):
        self.numerical_array = numerical_array
        self.mask_array = mask_array
        self.valid_array = valid_array
        self.train = train
        self.y = y
    
    
    def __len__(self):
        return len(self.numerical_array)
    
    
    def __getitem__(self, item):
        data = preprocess(self.numerical_array[item], self.mask_array[item], self.valid_array[item],)

        # Return the processed data where the lists are converted to `torch.tensor`s
        if self.train : 
            return {
              'input_data_numerical_array': torch.tensor(data['input_data_numerical_array'], dtype=torch.float32),
              'input_data_mask_array': torch.tensor(data['input_data_mask_array'],  dtype=torch.long),  
              'input_data_valid_array': torch.tensor(data['input_data_valid_array'], dtype=torch.long),   
              'attention_mask': torch.tensor(data["attention_mask"], dtype=torch.bool),
              "y": torch.tensor(self.y[item], dtype=torch.float32)
            }
        else:
            return {
              'input_data_numerical_array': torch.tensor(data['input_data_numerical_array'], dtype=torch.float32),
              'input_data_mask_array': torch.tensor(data['input_data_mask_array'], dtype=torch.long),  
              'input_data_valid_array': torch.tensor(data['input_data_valid_array'], dtype=torch.long),  
              'attention_mask': torch.tensor(data["attention_mask"], dtype=torch.bool),
               }

<br>

### class `FogRnnModel()`

In [117]:
class FogRnnModel(nn.Module):
    def __init__(self, 
                 dropout=0.2,
                 input_numerical_size=9,
                 numeraical_linear_size = 64,
                 model_size = 128,
                 linear_out = 128,
                 out_size=3):
        
        super(FogRnnModel, self).__init__()
        
        self.numerical_linear  = nn.Sequential(nn.Linear(input_numerical_size, numeraical_linear_size),
                                               nn.LayerNorm(numeraical_linear_size))
        
        self.rnn = nn.GRU(numeraical_linear_size, 
                          model_size,
                          num_layers = 2, 
                          batch_first=True,
                          bidirectional=True)
        
        self.linear_out  = nn.Sequential(nn.Linear(model_size*2, linear_out),
                                         nn.LayerNorm(linear_out),
                                         nn.ReLU(),
                                         nn.Dropout(dropout),
                                         nn.Linear(linear_out, out_size))
        self._reinitialize()
        
        
        
    def _reinitialize(self):
        """Tensorflow/Keras-like initialization"""
        for name, p in self.named_parameters():
            if 'rnn' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
        
        
        
    def forward(self, numerical_array, mask_array, attention_mask):
        numerical_embedding = self.numerical_linear(numerical_array)
        output, _ = self.rnn(numerical_embedding)
        output = self.linear_out(output)
        return output

<br>
<br>

## <font color=blue>**ROUND 1**</font>

<br>
<br>


## <font color=red>**Predicting**</font>: **Making** <font color=red>**pseudo**</font> **label** (round1: use models of ex153 & ex154)

<br>

### config

In [118]:
# config
seed = 0
shuffle = True
n_splits = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model config
batch_size = 24
n_epochs = 15
lr = 1e-3
weight_decay = 0.05
num_warmup_steps = 10

<br>

### <font color=maroon>ex153_defog_gru_inference_notype_10000.ipynb</font>

<br>

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/exp/ex153_defog_gru_inference_notype_10000.ipynb" style="text-decoration:none">ex153_defog_gru_inference_notype_10000.ipynb</a>

In [119]:
debug = False
ex = "153_defog"
if not os.path.exists(f"./output/exp/ex{ex}"):
    os.makedirs(f"./output/exp/ex{ex}")
    os.makedirs(f"./output/exp/ex{ex}/ex{ex}_model")

<br>

#### load data

In [120]:
seq_len = 10000

id_path        = f"./output/fe/fe064_notype/fe064_notype_id.parquet"
numerical_path = f"./output/fe/fe064_notype/fe064_notype_num_array.npy"
target_path    = f"./output/fe/fe064_notype/fe064_notype_target_array.npy"
mask_path      = f"./output/fe/fe064_notype/fe064_notype_mask_array.npy"
valid_path     = f"./output/fe/fe064_notype/fe064_notype_valid_array.npy"
pred_use_path  = f"./output/fe/fe064_notype/fe064_notype_pred_use_array.npy"
time_path      = f"./output/fe/fe064_notype/fe064_notype_time_array.npy"

df_id           = pd.read_parquet(id_path)
numerical_array = np.load(numerical_path)
target_array    = np.load(target_path)
mask_array      = np.load(mask_path)
valid_array     = np.load(valid_path)
pred_use_array  = np.load(pred_use_path)
time_array      = np.load(time_path)

<br>

#### main (predict)

In [121]:
models_list = os.listdir(f"./output/exp/ex{ex}/ex{ex}_model")
models_list

[]

In [122]:
if PREDICT_FLAG:
# with timer("gru"):
    set_seed(seed)
    for fold in range(1, 6):
        with timer(f"fold {fold}"):
            val_numerical_array = numerical_array.copy()
            val_target_array = target_array.copy()
            val_mask_array = mask_array.copy()
            val_valid_array = valid_array.copy()
            val_pred_array = pred_use_array.copy()

            val_ = FogDataset(val_numerical_array, 
                              val_mask_array,
                              val_valid_array, 
                              train=True,
                              y=val_target_array)

            val_loader = DataLoader(dataset=val_, 
                                    batch_size=batch_size, 
                                    shuffle = False,
                                    # num_workers=8
                                   )

            model = FogRnnModel()
            # model.load_state_dict(torch.load(f"./output/exp/ex{ex}/ex{ex}_model/ex{ex}_{fold}.pth"))
            model.load_state_dict(torch.load(f"./output/exp/ex{ex}/ex{ex}_model/" + models_list[fold-1]))
            model = model.to(device)
            model.eval()  # switch model to the evaluation mode
            
            val_preds = np.ndarray((0, 10000, 3))
            tk0 = tqdm(val_loader, total=len(val_loader))
            with torch.no_grad():  # Do not calculate gradient since we are only predicting
                # Predicting on validation set
                for d in tk0:
                    input_data_numerical_array = d['input_data_numerical_array'].to(device)
                    input_data_mask_array      = d['input_data_mask_array'].to(device)
                    attention_mask             = d['attention_mask'].to(device)
                    output = model(input_data_numerical_array, 
                                   input_data_mask_array,
                                   attention_mask)
                    val_preds = np.concatenate([val_preds, output.sigmoid().detach().cpu().numpy()], axis=0)
            np.save(f"./output/exp/ex{ex}/ex{ex}_notype_{fold}_oof_{seq_len}.npy",val_preds)

In [123]:
if PREDICT_FLAG:
    id_array = df_id["Id"].values
    for i in range(1, 6):
        pred_all = []
        pred = np.load(f"./output/exp/ex{ex}/ex{ex}_notype_{i}_oof_{seq_len}.npy")
        for v in tqdm(range(len(pred_use_array))):
            use_ = pred_use_array[v, :] == 1
            pred_ = pred[v, use_ == 1, :]
            time_ = time_array[v, use_ == 1]
            Id = id_array[v]
            pred_df = pd.DataFrame()
            pred_df["Time"] = time_
            pred_df["Id"] = Id
            pred_df["StartHesitation"] = pred_[:, 0]
            pred_df["Turn"] = pred_[:, 1]
            pred_df["Walking"] = pred_[:, 2]
            pred_all.append(pred_df)
        pred_all = pd.concat(pred_all).reset_index(drop=True)
        pred_all.to_parquet(f"./output/exp/ex{ex}/ex{ex}_notype_{i}_pred_{seq_len}.parquet")

<br>

### <font color=maroon>ex153_defog_gru_inference_notype_15000.ipynb</font>

<br>

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/exp/ex153_defog_gru_inference_notype_15000.ipynb" style="text-decoration:none">ex153_defog_gru_inference_notype_15000.ipynb</a>

In [124]:
debug = False
ex = "153_defog"
if not os.path.exists(f"./output/exp/ex{ex}"):
    os.makedirs(f"./output/exp/ex{ex}")
    os.makedirs(f"./output/exp/ex{ex}/ex{ex}_model")

<br>

#### load data

In [125]:
seq_len = 15000

id_path        = f"./output/fe/fe074_notype/fe074_notype_id.parquet"
numerical_path = f"./output/fe/fe074_notype/fe074_notype_num_array.npy"
target_path    = f"./output/fe/fe074_notype/fe074_notype_target_array.npy"
mask_path      = f"./output/fe/fe074_notype/fe074_notype_mask_array.npy"
valid_path     = f"./output/fe/fe074_notype/fe074_notype_valid_array.npy"
pred_use_path  = f"./output/fe/fe074_notype/fe074_notype_pred_use_array.npy"
time_path      = f"./output/fe/fe074_notype/fe074_notype_time_array.npy"

df_id           = pd.read_parquet(id_path)
numerical_array = np.load(numerical_path)
target_array    = np.load(target_path)
mask_array      = np.load(mask_path)
valid_array     = np.load(valid_path)
pred_use_array  = np.load(pred_use_path)
time_array      = np.load(time_path)

<br>

#### main (predict)

In [126]:
models_list = os.listdir(f"./output/exp/ex{ex}/ex{ex}_model")
models_list

[]

In [127]:
if PREDICT_FLAG:
# with timer("gru"):
    set_seed(seed)
    for fold in range(1, 6):
        with timer(f"fold {fold}"):
            val_numerical_array = numerical_array.copy()
            val_target_array = target_array.copy()
            val_mask_array = mask_array.copy()
            val_valid_array = valid_array.copy()
            val_pred_array = pred_use_array.copy()

            val_ = FogDataset(val_numerical_array, 
                              val_mask_array,
                              val_valid_array, 
                              train=True,
                              y=val_target_array)

            val_loader = DataLoader(dataset=val_, 
                                    batch_size=batch_size, 
                                    shuffle = False,
                                    # num_workers=8
                                   )

            model = FogRnnModel()
            # model.load_state_dict(torch.load(f"./output/exp/ex{ex}/ex{ex}_model/ex{ex}_{fold}.pth"))
            model.load_state_dict(torch.load(f"./output/exp/ex{ex}/ex{ex}_model/" + models_list[fold-1]))
            model = model.to(device)
            model.eval()  # switch model to the evaluation mode
            
            val_preds = np.ndarray((0, seq_len, 3))
            tk0 = tqdm(val_loader, total=len(val_loader))
            with torch.no_grad():  # Do not calculate gradient since we are only predicting
                # Predicting on validation set
                for d in tk0:
                    input_data_numerical_array = d['input_data_numerical_array'].to(device)
                    input_data_mask_array      = d['input_data_mask_array'].to(device)
                    attention_mask             = d['attention_mask'].to(device)
                    output = model(input_data_numerical_array, 
                                   input_data_mask_array,
                                   attention_mask)
                    val_preds = np.concatenate([val_preds, output.sigmoid().detach().cpu().numpy()], axis=0)
            np.save(f"./output/exp/ex{ex}/ex{ex}_notype_{fold}_oof_{seq_len}.npy",val_preds)

In [128]:
if PREDICT_FLAG:
    id_array = df_id["Id"].values
    for i in range(1, 6):
        pred_all = []
        pred = np.load(f"./output/exp/ex{ex}/ex{ex}_notype_{i}_oof_{seq_len}.npy")
        for v in tqdm(range(len(pred_use_array))):
            use_ = pred_use_array[v, :] == 1
            pred_ = pred[v, use_ == 1, :]
            time_ = time_array[v, use_ == 1]
            Id = id_array[v]
            pred_df = pd.DataFrame()
            pred_df["Time"] = time_
            pred_df["Id"] = Id
            pred_df["StartHesitation"] = pred_[:, 0]
            pred_df["Turn"] = pred_[:, 1]
            pred_df["Walking"] = pred_[:, 2]
            pred_all.append(pred_df)
        pred_all = pd.concat(pred_all).reset_index(drop=True)
        pred_all.to_parquet(f"./output/exp/ex{ex}/ex{ex}_notype_{i}_pred_{seq_len}.parquet")

<br>

### <font color=maroon>ex154_defog_gru_inference_notype_15000.ipynb</font>

<br>

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/exp/ex154_defog_gru_inference_notype_15000.ipynb" style="text-decoration:none">ex154_defog_gru_inference_notype_15000.ipynb</a>

In [129]:
debug = False
ex = "154_defog"
if not os.path.exists(f"./output/exp/ex{ex}"):
    os.makedirs(f"./output/exp/ex{ex}")
    os.makedirs(f"./output/exp/ex{ex}/ex{ex}_model")

<br>

#### load data

In [130]:
seq_len = 15000

id_path        = f"./output/fe/fe074_notype/fe074_notype_id.parquet"
numerical_path = f"./output/fe/fe074_notype/fe074_notype_num_array.npy"
target_path    = f"./output/fe/fe074_notype/fe074_notype_target_array.npy"
mask_path      = f"./output/fe/fe074_notype/fe074_notype_mask_array.npy"
valid_path     = f"./output/fe/fe074_notype/fe074_notype_valid_array.npy"
pred_use_path  = f"./output/fe/fe074_notype/fe074_notype_pred_use_array.npy"
time_path      = f"./output/fe/fe074_notype/fe074_notype_time_array.npy"

df_id           = pd.read_parquet(id_path)
numerical_array = np.load(numerical_path)
target_array    = np.load(target_path)
mask_array      = np.load(mask_path)
valid_array     = np.load(valid_path)
pred_use_array  = np.load(pred_use_path)
time_array      = np.load(time_path)

<br>

#### main (predict)

In [131]:
models_list = os.listdir(f"./output/exp/ex{ex}/ex{ex}_model")
models_list

[]

In [132]:
if PREDICT_FLAG:
# with timer("gru"):
    set_seed(seed)
    for fold in range(1, 6):
        with timer(f"fold {fold}"):
            val_numerical_array = numerical_array.copy()
            val_target_array = target_array.copy()
            val_mask_array = mask_array.copy()
            val_valid_array = valid_array.copy()
            val_pred_array = pred_use_array.copy()

            val_ = FogDataset(val_numerical_array, 
                              val_mask_array,
                              val_valid_array, 
                              train=True,
                              y=val_target_array)

            val_loader = DataLoader(dataset=val_, 
                                    batch_size=batch_size, 
                                    shuffle = False,
                                    # num_workers=8
                                   )

            # model = FogRnnModel()
            model = FogRnnModel(numeraical_linear_size = 96,
                                model_size = 256,
                                linear_out = 256,)
            # model.load_state_dict(torch.load(f"./output/exp/ex{ex}/ex{ex}_model/ex{ex}_{fold}.pth"))
            model.load_state_dict(torch.load(f"./output/exp/ex{ex}/ex{ex}_model/" + models_list[fold-1]))
            model = model.to(device)
            model.eval()  # switch model to the evaluation mode
            
            val_preds = np.ndarray((0, seq_len, 3))
            tk0 = tqdm(val_loader, total=len(val_loader))
            with torch.no_grad():  # Do not calculate gradient since we are only predicting
                # Predicting on validation set
                for d in tk0:
                    input_data_numerical_array = d['input_data_numerical_array'].to(device)
                    input_data_mask_array      = d['input_data_mask_array'].to(device)
                    attention_mask             = d['attention_mask'].to(device)
                    output = model(input_data_numerical_array, 
                                   input_data_mask_array,
                                   attention_mask)
                    val_preds = np.concatenate([val_preds, output.sigmoid().detach().cpu().numpy()], axis=0)
            np.save(f"./output/exp/ex{ex}/ex{ex}_notype_{fold}_oof_{seq_len}.npy",val_preds)

In [133]:
if PREDICT_FLAG:
    id_array = df_id["Id"].values
    for i in range(1, 6):
        pred_all = []
        pred = np.load(f"./output/exp/ex{ex}/ex{ex}_notype_{i}_oof_{seq_len}.npy")
        for v in tqdm(range(len(pred_use_array))):
            use_ = pred_use_array[v, :] == 1
            pred_ = pred[v, use_ == 1, :]
            time_ = time_array[v, use_ == 1]
            Id = id_array[v]
            pred_df = pd.DataFrame()
            pred_df["Time"] = time_
            pred_df["Id"] = Id
            pred_df["StartHesitation"] = pred_[:, 0]
            pred_df["Turn"] = pred_[:, 1]
            pred_df["Walking"] = pred_[:, 2]
            pred_all.append(pred_df)
        pred_all = pd.concat(pred_all).reset_index(drop=True)
        pred_all.to_parquet(f"./output/exp/ex{ex}/ex{ex}_notype_{i}_pred_{seq_len}.parquet")

<br>
<br>

## **Feature Engineering with** <font color=red>**pseudo**</font>  **label**

<br>

### fe073_notype_pseudo_ex153_10000.ipynb

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/fe/fe073_notype_pseudo_ex153_10000.ipynb" style="text-decoration:none">fe073_notype_pseudo_ex153_10000.ipynb</a>

In [134]:
fe = "073_notype_pseudo"
ex = "153_defog"
if not os.path.exists(f"./output/fe/fe{fe}"):
    os.makedirs(f"./output/fe/fe{fe}")
    os.makedirs(f"./output/fe/fe{fe}/save")

In [135]:
# DEFOG_META_PATH = "../data/defog_metadata.csv"
# DEFOG_FOLDER = "../data/train/notype/*.csv"

defog_meta = pd.read_parquet("./output/fe/fe039/fe039_defog_meta.parquet")

In [136]:
cols = ["AccV", "AccML", "AccAP"]
num_cols = ["AccV", "AccML", "AccAP",
            'AccV_lag_diff', 'AccV_lead_diff', 
            'AccML_lag_diff', 'AccML_lead_diff',
            'AccAP_lag_diff', 'AccAP_lead_diff']

target_use_cols = ["Event"]
target_cols = ["StartHesitation", "Turn", "Walking"]
seq_len = 5000
shift = 2500
offset = 1250

In [137]:
# data_list = glob.glob(DEFOG_FOLDER)

if TRAIN_FLAG:
    for fold in range(1, 6):
        print(fold)
        pred = pd.read_parquet(f"./output/exp/ex{ex}/ex{ex}_notype_{fold}_pred_10000.parquet")
        target_array = []
        for i,s in tqdm(zip(defog_meta["Id"].values, defog_meta["sub_id"].values)):
            path = root_data + f"train/notype/{i}.csv"
            if path in [x.replace("\\", "/") for x in train_notype]:
            # if path in data_list:
                df = pd.read_csv(path)
                df_ = pred[pred["Id"] == i].reset_index(drop=True)
                df = df.merge(df_, how="left", on="Time")
                df["target_max"] = np.argmax(df[["StartHesitation", "Turn", "Walking"]].values, axis=1)

                df.loc[df["target_max"] == 0, "StartHesitation"] = 1
                df.loc[df["target_max"] == 0, ["Turn","Walking"]] = 0

                df.loc[df["target_max"] == 1, "Turn"] = 1
                df.loc[df["target_max"] == 1, ["StartHesitation","Walking"]] = 0

                df.loc[df["target_max"] == 2, "Walking"] = 1
                df.loc[df["target_max"] == 2, ["StartHesitation","Turn"]] = 0

                df.loc[df["Event"] == 0, ["StartHesitation", "Turn", "Walking"]] = 0

                df["valid"] = df["Valid"] & df["Task"]
                df["valid"] = df["valid"].astype(int)
                batch = (len(df)-1) // shift
                target = df[target_cols].values
                target_array_ = np.zeros([batch, seq_len, 3])
                for n,b in enumerate(range(batch)):
                    if b == (batch - 1):
                        target_ = target[b*shift : ]
                        target_array_[b, :len(target_), :] = target_
                    elif b == 0:
                        target_ = target[b*shift : b*shift + seq_len]
                        target_array_[b, :, :] = target_
                    else:
                        target_ = target[b*shift : b*shift + seq_len]
                        target_array_[b, :, :] = target_

                target_array.append(target_array_)
        target_array = np.concatenate(target_array, axis=0)
        np.save(f"./output/fe/fe{fe}/fe{fe}_target_array_{fold}.npy", target_array)

In [138]:
# df

In [139]:
# pred

<br>

### fe075_notype_pseudo_ex153_15000.ipynb

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/fe/fe075_notype_pseudo_ex153_15000.ipynb" style="text-decoration:none">fe075_notype_pseudo_ex153_15000.ipynb</a>

In [140]:
fe = "075_notype_pseudo"
ex = "153_defog"
if not os.path.exists(f"./output/fe/fe{fe}"):
    os.makedirs(f"./output/fe/fe{fe}")
    os.makedirs(f"./output/fe/fe{fe}/save")

In [141]:
# DEFOG_META_PATH = "../data/defog_metadata.csv"
# DEFOG_FOLDER = "../data/train/notype/*.csv"

defog_meta = pd.read_parquet("./output/fe/fe039/fe039_defog_meta.parquet")

In [142]:
cols = ["AccV", "AccML", "AccAP"]
num_cols = ["AccV", "AccML", "AccAP",
            'AccV_lag_diff', 'AccV_lead_diff', 
            'AccML_lag_diff', 'AccML_lead_diff',
            'AccAP_lag_diff', 'AccAP_lead_diff']

target_use_cols = ["Event"]
target_cols = ["StartHesitation", "Turn", "Walking"]
seq_len = 5000
shift = 2500
offset = 1250

In [143]:
# data_list = glob.glob(DEFOG_FOLDER)

if TRAIN_FLAG:
    for fold in range(1, 6):
        print(fold)
        pred = pd.read_parquet(f"./output/exp/ex{ex}/ex{ex}_notype_{fold}_pred_15000.parquet")
        target_array = []
        for i,s in tqdm(zip(defog_meta["Id"].values, defog_meta["sub_id"].values)):
            path = root_data + f"train/notype/{i}.csv"
            if path in [x.replace("\\", "/") for x in train_notype]:
            # if path in data_list:
                df = pd.read_csv(path)
                df_ = pred[pred["Id"] == i].reset_index(drop=True)
                df = df.merge(df_, how="left", on="Time")
                df["target_max"] = np.argmax(df[["StartHesitation", "Turn", "Walking"]].values, axis=1)

                df.loc[df["target_max"] == 0, "StartHesitation"] = 1
                df.loc[df["target_max"] == 0, ["Turn","Walking"]] = 0

                df.loc[df["target_max"] == 1, "Turn"] = 1
                df.loc[df["target_max"] == 1, ["StartHesitation","Walking"]] = 0

                df.loc[df["target_max"] == 2, "Walking"] = 1
                df.loc[df["target_max"] == 2, ["StartHesitation","Turn"]] = 0

                df.loc[df["Event"] == 0, ["StartHesitation", "Turn", "Walking"]] = 0

                df["valid"] = df["Valid"] & df["Task"]
                df["valid"] = df["valid"].astype(int)
                batch = (len(df)-1) // shift
                target = df[target_cols].values
                target_array_ = np.zeros([batch, seq_len, 3])
                for n,b in enumerate(range(batch)):
                    if b == (batch - 1):
                        target_ = target[b*shift : ]
                        target_array_[b, :len(target_), :] = target_
                    elif b == 0:
                        target_ = target[b*shift : b*shift + seq_len]
                        target_array_[b, :, :] = target_
                    else:
                        target_ = target[b*shift : b*shift + seq_len]
                        target_array_[b, :, :] = target_

                target_array.append(target_array_)
        target_array = np.concatenate(target_array, axis=0)
        np.save(f"./output/fe/fe{fe}/fe{fe}_target_array_{fold}.npy", target_array)

In [144]:
# df

In [145]:
# pred

<br>

### fe086_notype_pseudo_ex154_15000.ipynb

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/fe/fe086_notype_pseudo_ex154_15000.ipynb" style="text-decoration:none">fe086_notype_pseudo_ex154_15000.ipynb</a>

In [146]:
fe = "086_notype_pseudo"
ex = "154_defog"
if not os.path.exists(f"./output/fe/fe{fe}"):
    os.makedirs(f"./output/fe/fe{fe}")
    os.makedirs(f"./output/fe/fe{fe}/save")

In [147]:
# DEFOG_META_PATH = "../data/defog_metadata.csv"
# DEFOG_FOLDER = "../data/train/notype/*.csv"

defog_meta = pd.read_parquet("./output/fe/fe039/fe039_defog_meta.parquet")

In [148]:
cols = ["AccV", "AccML", "AccAP"]
num_cols = ["AccV", "AccML", "AccAP",
            'AccV_lag_diff', 'AccV_lead_diff', 
            'AccML_lag_diff', 'AccML_lead_diff',
            'AccAP_lag_diff', 'AccAP_lead_diff']

target_use_cols = ["Event"]
target_cols = ["StartHesitation", "Turn", "Walking"]
seq_len = 5000
shift = 2500
offset = 1250

In [149]:
# data_list = glob.glob(DEFOG_FOLDER)
if TRAIN_FLAG:
    for fold in range(1, 6):
        print(fold)
        pred = pd.read_parquet(f"./output/exp/ex{ex}/ex{ex}_notype_{fold}_pred_15000.parquet")
        target_array = []
        for i,s in tqdm(zip(defog_meta["Id"].values, defog_meta["sub_id"].values)):
            path = root_data + f"train/notype/{i}.csv"
            if path in [x.replace("\\", "/") for x in train_notype]:
            # if path in data_list:
                df = pd.read_csv(path)
                df_ = pred[pred["Id"] == i].reset_index(drop=True)
                df = df.merge(df_, how="left", on="Time")
                df["target_max"] = np.argmax(df[["StartHesitation", "Turn", "Walking"]].values, axis=1)

                df.loc[df["target_max"] == 0, "StartHesitation"] = 1
                df.loc[df["target_max"] == 0, ["Turn","Walking"]] = 0

                df.loc[df["target_max"] == 1, "Turn"] = 1
                df.loc[df["target_max"] == 1, ["StartHesitation","Walking"]] = 0

                df.loc[df["target_max"] == 2, "Walking"] = 1
                df.loc[df["target_max"] == 2, ["StartHesitation","Turn"]] = 0

                df.loc[df["Event"] == 0, ["StartHesitation", "Turn", "Walking"]] = 0

                df["valid"] = df["Valid"] & df["Task"]
                df["valid"] = df["valid"].astype(int)
                batch = (len(df)-1) // shift
                target = df[target_cols].values
                target_array_ = np.zeros([batch, seq_len, 3])
                for n,b in enumerate(range(batch)):
                    if b == (batch - 1):
                        target_ = target[b*shift : ]
                        target_array_[b, :len(target_), :] = target_
                    elif b == 0:
                        target_ = target[b*shift : b*shift + seq_len]
                        target_array_[b, :, :] = target_
                    else:
                        target_ = target[b*shift : b*shift + seq_len]
                        target_array_[b, :, :] = target_

                target_array.append(target_array_)
        target_array = np.concatenate(target_array, axis=0)
        np.save(f"./output/fe/fe{fe}/fe{fe}_target_array_{fold}.npy", target_array)

In [150]:
# df

In [151]:
# pred

<br>
<br>

##  <font color=red>**Training**</font> **with** <font color=red>**pseudo**</font> **label** (round1)



<br>

### load data & preprocessing

In [152]:
id_path        = f"./output/fe/fe047/fe047_id.parquet"
numerical_path = f"./output/fe/fe047/fe047_num_array.npy"
target_path    = f"./output/fe/fe047/fe047_target_array.npy"
mask_path      = f"./output/fe/fe047/fe047_mask_array.npy"
valid_path     = f"./output/fe/fe047/fe047_valid_array.npy"
pred_use_path  = f"./output/fe/fe047/fe047_pred_use_array.npy"

df_id           = pd.read_parquet(id_path)
numerical_array = np.load(numerical_path)
target_array    = np.load(target_path)
mask_array      = np.load(mask_path)
valid_array     = np.load(valid_path)
pred_use_array  = np.load(pred_use_path)

In [153]:
target1 = []
target2 = []
target3 = []
for i in range(len(target_array)):
    target1.append(np.sum(target_array[i,:,0]))
    target2.append(np.sum(target_array[i,:,1]))
    target3.append(np.sum(target_array[i,:,2]))

df_id["target1"] = target1
df_id["target2"] = target2
df_id["target3"] = target3
df_id["target1_1"] = df_id["target1"] > 0
df_id["target2_1"] = df_id["target2"] > 0
df_id["target3_1"] = df_id["target3"] > 0
df_id["target1_1"] = df_id["target1_1"].astype(np.int)
df_id["target2_1"] = df_id["target2_1"].astype(np.int)
df_id["target3_1"] = df_id["target3_1"].astype(np.int)

df_id["group"] = 0
df_id.loc[df_id["target1_1"] > 0,"group"] = 1
df_id.loc[df_id["target2_1"] > 0,"group"] = 2
df_id.loc[df_id["target3_1"] > 0,"group"] = 3

df_id["group"].value_counts()

0    4107
2     897
3     354
1      11
Name: group, dtype: int64

In [154]:
pseudo_id_path        = f"./output/fe/fe061_notype/fe061_notype_id.parquet"
pseudo_numerical_path = f"./output/fe/fe061_notype/fe061_notype_num_array.npy"
pseudo_mask_path      = f"./output/fe/fe061_notype/fe061_notype_mask_array.npy"
pseudo_valid_path     = f"./output/fe/fe061_notype/fe061_notype_valid_array.npy"
pseudo_pred_use_path  = f"./output/fe/fe061_notype/fe061_notype_pred_use_array.npy"

pseudo_numerical_array = np.load(pseudo_numerical_path)
pseudo_mask_array      = np.load(pseudo_mask_path)
pseudo_valid_array     = np.load(pseudo_valid_path)
pseudo_pred_use_array  = np.load(pseudo_pred_use_path)

<br>

### config

In [155]:
# config
seed = 0
shuffle = True
n_splits = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model config
batch_size = 24
n_epochs = 10
lr = 1e-3
weight_decay = 0.05
num_warmup_steps = 10

<br>

### <font color=maroon>ex175_defog_gru_pseudo.ipynb</font>

<br>

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/exp/ex175_defog_gru_pseudo.ipynb" style="text-decoration:none">ex175_defog_gru_pseudo.ipynb</a>

In [156]:
debug = False
ex = "175_notype_pseudo_target"
if not os.path.exists(f"./output/exp/ex{ex}"):
    os.makedirs(f"./output/exp/ex{ex}")
    os.makedirs(f"./output/exp/ex{ex}/ex{ex}_model")

In [157]:
pseudo_target = "073_notype_pseudo"

<br>

#### main

In [158]:
%%time

# with timer("gru"):
if TRAIN_FLAG:
    set_seed(seed)
    y_oof = np.empty([len(target_array), 5000, 3])      # Abrachan: out of fold
    gkf = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state = seed)
    for fold, (train_idx, valid_idx) in enumerate(gkf.split(numerical_array, 
                                                            y = df_id["group"].values,
                                                            groups=df_id["subject"].values)):
        # LOGGER.info(f"start fold:{fold+1}")
        print(f"start fold: {fold+1}")
        
        with timer(f"fold {fold+1}"):
            train_numerical_array = numerical_array[train_idx]
            train_target_array = target_array[train_idx]
            train_mask_array = mask_array[train_idx]
            train_valid_array = valid_array[train_idx]
            
            pseudo_target_array = np.load(f"./output/fe/fe{pseudo_target}/fe{pseudo_target}_target_array_{fold+1}.npy")
            train_numerical_array = np.concatenate([train_numerical_array, pseudo_numerical_array], axis=0)
            train_target_array = np.concatenate([train_target_array, pseudo_target_array], axis=0)
            train_mask_array = np.concatenate([train_mask_array, pseudo_mask_array], axis=0)
            train_valid_array = np.concatenate([train_valid_array, pseudo_valid_array], axis=0)

            val_numerical_array = numerical_array[valid_idx]
            val_target_array = target_array[valid_idx]
            val_mask_array = mask_array[valid_idx]
            val_valid_array = valid_array[valid_idx]
            val_pred_array = pred_use_array[valid_idx]
            
            train_ = FogDataset(train_numerical_array,
                                train_mask_array,
                                train_valid_array,
                                train=True,
                                y=train_target_array)
            val_ = FogDataset(val_numerical_array,
                              val_mask_array,
                              val_valid_array,
                              train=True,
                              y=val_target_array)
            
            train_loader = DataLoader(dataset=train_, 
                                      batch_size=batch_size, 
                                      shuffle = True, 
                                      # num_workers=8,
                                     )
            val_loader = DataLoader(dataset=val_, 
                                    batch_size=batch_size, 
                                    shuffle = False , 
                                    # num_workers=8
                                   )
            
            
            model = FogRnnModel()
            model = model.to(device)
            
            param_optimizer = list(model.named_parameters())
            no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
            optimizer_grouped_parameters = [
                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 
                 'weight_decay': weight_decay
                },
                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 
                 'weight_decay': 0.0
                }]
            optimizer = AdamW(optimizer_grouped_parameters,
                              lr=lr,
                              weight_decay=weight_decay,
                              )
            num_train_optimization_steps = int(len(train_loader) * n_epochs)
            scheduler = get_linear_schedule_with_warmup(optimizer,
                                                        num_warmup_steps=num_warmup_steps,
                                                        num_training_steps=num_train_optimization_steps)
            criterion = nn.BCEWithLogitsLoss()
            best_val_score = 0
            
            for epoch in range(n_epochs):
                model.train() 
                train_losses_batch = []
                val_losses_batch = []
                epoch_loss = 0
                train_preds = np.ndarray((0,3))   # array([], shape=(0, 3), dtype=float64)
                
                tk0 = tqdm_nb(train_loader, total=len(train_loader), desc="train_loader: ")
                for d in tk0:
                    # ======================================================
                    # data loader
                    # ======================================================
                    input_data_numerical_array = d['input_data_numerical_array'].to(device)
                    input_data_mask_array      = d['input_data_mask_array'].to(device)
                    input_data_valid_array     = d['input_data_valid_array'].to(device)
                    attention_mask             = d['attention_mask'].to(device)
                    y                          = d["y"].to(device)
                    
                    optimizer.zero_grad()
                    output = model(input_data_numerical_array, 
                                   input_data_mask_array,
                                   attention_mask)
                    loss = criterion(output[input_data_mask_array == 1], y[input_data_mask_array == 1])
                    
                    # output1 = output[:, :, [1,2]]
                    # output2 = output[:, :, 0]
                    # y1 = y[:, :, [1,2]]
                    # y2 = y[:, :, 0]
                    # loss1 = criterion(output1[input_data_mask_array == 1], y1[input_data_mask_array == 1])
                    # loss2 = criterion(output2[input_data_mask_array == 1], y2[input_data_mask_array == 1])
                    # loss = loss1*0.8 + loss2*0.2
                    
                    loss.backward()
                    optimizer.step()
                    scheduler.step()
                    train_losses_batch.append(loss.item())
                train_loss = np.mean(train_losses_batch)
                
                
                # ======================================================
                # eval
                # ======================================================
                model.eval()       # switch model to the evaluation mode
                
                val_preds = np.ndarray((0, 5000, 3))
                tk0 = tqdm_nb(val_loader, total=len(val_loader), desc="val_loader  : ")
                with torch.no_grad():  # Do not calculate gradient since we are only predicting
                    # Predicting on validation set
                    for d in tk0:
                        input_data_numerical_array = d['input_data_numerical_array'].to(device)
                        input_data_mask_array      = d['input_data_mask_array'].to(device)
                        attention_mask             = d['attention_mask'].to(device)
                        
                        output = model(input_data_numerical_array, 
                                       input_data_mask_array,
                                       attention_mask)
                        val_preds = np.concatenate([val_preds, output.sigmoid().detach().cpu().numpy()], axis=0)
                
                pred_valid_index = (val_mask_array == 1) & (val_pred_array == 1) & (val_valid_array == 1)
                StartHesitation = average_precision_score(val_target_array[pred_valid_index][:, 0],
                                                          val_preds[pred_valid_index][:, 0])
                Turn = average_precision_score(val_target_array[pred_valid_index][:, 1],
                                               val_preds[pred_valid_index][:, 1])
                Walking = average_precision_score(val_target_array[pred_valid_index][:, 2],
                                                  val_preds[pred_valid_index][:, 2])
                # map_score = np.mean([StartHesitation, Turn, Walking])
                map_score = np.mean([Turn, Walking])
                
                # LOGGER.info(f"fold: {fold+1} epoch: {epoch+1},train loss {train_loss} map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                print(f"fold {fold+1} epoch {epoch+1}\ntrain loss {train_loss} map:{map_score}\nstart_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                
                if map_score >= best_val_score:
                    print("save weight")
                    best_val_score = map_score
                    best_val_preds = val_preds.copy()
                    torch.save(model.state_dict(), f"./output/exp/ex{ex}/ex{ex}_model/ex{ex}_fold{fold+1}_epoch{epoch+1}__map{best_val_score:.6f}.pth")
                print()
            y_oof[valid_idx] = best_val_preds
    
    np.save(f"./output/exp/ex{ex}/ex{ex}_oof.npy", y_oof)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.54 µs


<br>

#### oof score

In [159]:
if TRAIN_FLAG:
    val_pred_index = (mask_array == 1) & (pred_use_array == 1) & (valid_array == 1)
    StartHesitation = average_precision_score(target_array[val_pred_index][:,0], 
                                              y_oof[val_pred_index][:,0])
    Turn            = average_precision_score(target_array[val_pred_index][:,1], 
                                              y_oof[val_pred_index][:,1])
    Walking         = average_precision_score(target_array[val_pred_index][:,2],
                                              y_oof[val_pred_index][:,2])

    map_score = np.mean([StartHesitation, Turn, Walking])
    # LOGGER.info(f"cv map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
    print(f"cv map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")

<br>

### <font color=maroon>ex179_defog_gru_pseudo.ipynb</font>

<br>

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/exp/ex179_defog_gru_pseudo.ipynb" style="text-decoration:none">ex179_defog_gru_pseudo.ipynb</a>

In [160]:
debug = False
ex = "179_notype_pseudo_target"
if not os.path.exists(f"./output/exp/ex{ex}"):
    os.makedirs(f"./output/exp/ex{ex}")
    os.makedirs(f"./output/exp/ex{ex}/ex{ex}_model")

In [161]:
seq_len = 5000

pseudo_target = "075_notype_pseudo"

<br>

#### main

In [162]:
%%time

# with timer("gru"):
if TRAIN_FLAG:
    set_seed(seed)
    y_oof = np.empty([len(target_array), seq_len, 3])      # Abrachan: out of fold
    gkf = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state = seed)
    for fold, (train_idx, valid_idx) in enumerate(gkf.split(numerical_array, 
                                                            y = df_id["group"].values,
                                                            groups=df_id["subject"].values)):
        # LOGGER.info(f"start fold:{fold+1}")
        print(f"start fold: {fold+1}")
        
        with timer(f"fold {fold+1}"):
            train_numerical_array = numerical_array[train_idx]
            train_target_array = target_array[train_idx]
            train_mask_array = mask_array[train_idx]
            train_valid_array = valid_array[train_idx]
            
            pseudo_target_array = np.load(f"./output/fe/fe{pseudo_target}/fe{pseudo_target}_target_array_{fold+1}.npy")
            train_numerical_array = np.concatenate([train_numerical_array, pseudo_numerical_array], axis=0)
            train_target_array = np.concatenate([train_target_array, pseudo_target_array], axis=0)
            train_mask_array = np.concatenate([train_mask_array, pseudo_mask_array], axis=0)
            train_valid_array = np.concatenate([train_valid_array, pseudo_valid_array], axis=0)

            val_numerical_array = numerical_array[valid_idx]
            val_target_array = target_array[valid_idx]
            val_mask_array = mask_array[valid_idx]
            val_valid_array = valid_array[valid_idx]
            val_pred_array = pred_use_array[valid_idx]
            
            train_ = FogDataset(train_numerical_array,
                                train_mask_array,
                                train_valid_array,
                                train=True,
                                y=train_target_array)
            val_ = FogDataset(val_numerical_array,
                              val_mask_array,
                              val_valid_array,
                              train=True,
                              y=val_target_array)
            
            train_loader = DataLoader(dataset=train_, 
                                      batch_size=batch_size, 
                                      shuffle = True, 
                                      # num_workers=8,
                                     )
            val_loader = DataLoader(dataset=val_, 
                                    batch_size=batch_size, 
                                    shuffle = False , 
                                    # num_workers=8
                                   )
            
            
            model = FogRnnModel()
            model = model.to(device)
            
            param_optimizer = list(model.named_parameters())
            no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
            optimizer_grouped_parameters = [
                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 
                 'weight_decay': weight_decay
                },
                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 
                 'weight_decay': 0.0
                }]
            optimizer = AdamW(optimizer_grouped_parameters,
                              lr=lr,
                              weight_decay=weight_decay,
                              )
            num_train_optimization_steps = int(len(train_loader) * n_epochs)
            scheduler = get_linear_schedule_with_warmup(optimizer,
                                                        num_warmup_steps=num_warmup_steps,
                                                        num_training_steps=num_train_optimization_steps)
            criterion = nn.BCEWithLogitsLoss()
            best_val_score = 0
            
            for epoch in range(n_epochs):
                model.train() 
                train_losses_batch = []
                val_losses_batch = []
                epoch_loss = 0
                train_preds = np.ndarray((0,3))   # array([], shape=(0, 3), dtype=float64)
                
                tk0 = tqdm_nb(train_loader, total=len(train_loader), desc="train_loader: ")
                for d in tk0:
                    # ======================================================
                    # data loader
                    # ======================================================
                    input_data_numerical_array = d['input_data_numerical_array'].to(device)
                    input_data_mask_array      = d['input_data_mask_array'].to(device)
                    input_data_valid_array     = d['input_data_valid_array'].to(device)
                    attention_mask             = d['attention_mask'].to(device)
                    y                          = d["y"].to(device)
                    
                    optimizer.zero_grad()
                    output = model(input_data_numerical_array, 
                                   input_data_mask_array,
                                   attention_mask)
                    loss = criterion(output[input_data_mask_array == 1], y[input_data_mask_array == 1])
                    
                    # output1 = output[:, :, [1,2]]
                    # output2 = output[:, :, 0]
                    # y1 = y[:, :, [1,2]]
                    # y2 = y[:, :, 0]
                    # loss1 = criterion(output1[input_data_mask_array == 1], y1[input_data_mask_array == 1])
                    # loss2 = criterion(output2[input_data_mask_array == 1], y2[input_data_mask_array == 1])
                    # loss = loss1*0.8 + loss2*0.2
                    
                    loss.backward()
                    optimizer.step()
                    scheduler.step()
                    train_losses_batch.append(loss.item())
                train_loss = np.mean(train_losses_batch)
                
                
                # ======================================================
                # eval
                # ======================================================
                model.eval()       # switch model to the evaluation mode
                
                val_preds = np.ndarray((0, seq_len, 3))
                tk0 = tqdm_nb(val_loader, total=len(val_loader), desc="val_loader  : ")
                with torch.no_grad():  # Do not calculate gradient since we are only predicting
                    # Predicting on validation set
                    for d in tk0:
                        input_data_numerical_array = d['input_data_numerical_array'].to(device)
                        input_data_mask_array      = d['input_data_mask_array'].to(device)
                        attention_mask             = d['attention_mask'].to(device)
                        
                        output = model(input_data_numerical_array, 
                                       input_data_mask_array,
                                       attention_mask)
                        val_preds = np.concatenate([val_preds, output.sigmoid().detach().cpu().numpy()], axis=0)
                
                pred_valid_index = (val_mask_array == 1) & (val_pred_array == 1) & (val_valid_array == 1)
                StartHesitation = average_precision_score(val_target_array[pred_valid_index][:, 0],
                                                          val_preds[pred_valid_index][:, 0])
                Turn = average_precision_score(val_target_array[pred_valid_index][:, 1],
                                               val_preds[pred_valid_index][:, 1])
                Walking = average_precision_score(val_target_array[pred_valid_index][:, 2],
                                                  val_preds[pred_valid_index][:, 2])
                # map_score = np.mean([StartHesitation, Turn, Walking])
                map_score = np.mean([Turn, Walking])
                
                # LOGGER.info(f"fold: {fold+1} epoch: {epoch+1},train loss {train_loss} map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                print(f"fold {fold+1} epoch {epoch+1}\ntrain loss {train_loss} map:{map_score}\nstart_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                
                if map_score >= best_val_score:
                    print("save weight")
                    best_val_score = map_score
                    best_val_preds = val_preds.copy()
                    torch.save(model.state_dict(), f"./output/exp/ex{ex}/ex{ex}_model/ex{ex}_fold{fold+1}_epoch{epoch+1}__map{best_val_score:.6f}.pth")
                print()
            y_oof[valid_idx] = best_val_preds
    
    np.save(f"./output/exp/ex{ex}/ex{ex}_oof.npy", y_oof)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 10.5 µs


<br>

#### oof score

In [163]:
if TRAIN_FLAG:
    val_pred_index = (mask_array == 1) & (pred_use_array == 1) & (valid_array == 1)
    StartHesitation = average_precision_score(target_array[val_pred_index][:,0], 
                                              y_oof[val_pred_index][:,0])
    Turn            = average_precision_score(target_array[val_pred_index][:,1], 
                                              y_oof[val_pred_index][:,1])
    Walking         = average_precision_score(target_array[val_pred_index][:,2],
                                              y_oof[val_pred_index][:,2])

    map_score = np.mean([StartHesitation, Turn, Walking])
    # LOGGER.info(f"cv map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
    print(f"cv map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")

<br>

### <font color=maroon>ex204_defog_gru_pseudo.ipynb</font>

<br>

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/exp/ex204_defog_gru_pseudo.ipynb" style="text-decoration:none">ex204_defog_gru_pseudo.ipynb</a>

In [164]:
debug = False
ex = "204_notype_pseudo_target"
if not os.path.exists(f"./output/exp/ex{ex}"):
    os.makedirs(f"./output/exp/ex{ex}")
    os.makedirs(f"./output/exp/ex{ex}/ex{ex}_model")

In [165]:
pseudo_target = "086_notype_pseudo"

<br>

#### main

In [166]:
%%time

# with timer("gru"):
if TRAIN_FLAG:
    set_seed(seed)
    y_oof = np.empty([len(target_array), 5000, 3])      # Abrachan: out of fold
    gkf = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state = seed)
    for fold, (train_idx, valid_idx) in enumerate(gkf.split(numerical_array, 
                                                            y = df_id["group"].values,
                                                            groups=df_id["subject"].values)):
        # LOGGER.info(f"start fold:{fold+1}")
        print(f"start fold: {fold+1}")
        
        with timer(f"fold {fold+1}"):
            train_numerical_array = numerical_array[train_idx]
            train_target_array = target_array[train_idx]
            train_mask_array = mask_array[train_idx]
            train_valid_array = valid_array[train_idx]
            
            pseudo_target_array = np.load(f"./output/fe/fe{pseudo_target}/fe{pseudo_target}_target_array_{fold+1}.npy")
            train_numerical_array = np.concatenate([train_numerical_array, pseudo_numerical_array], axis=0)
            train_target_array = np.concatenate([train_target_array, pseudo_target_array], axis=0)
            train_mask_array = np.concatenate([train_mask_array, pseudo_mask_array], axis=0)
            train_valid_array = np.concatenate([train_valid_array, pseudo_valid_array], axis=0)

            val_numerical_array = numerical_array[valid_idx]
            val_target_array = target_array[valid_idx]
            val_mask_array = mask_array[valid_idx]
            val_valid_array = valid_array[valid_idx]
            val_pred_array = pred_use_array[valid_idx]
            
            train_ = FogDataset(train_numerical_array,
                                train_mask_array,
                                train_valid_array,
                                train=True,
                                y=train_target_array)
            val_ = FogDataset(val_numerical_array,
                              val_mask_array,
                              val_valid_array,
                              train=True,
                              y=val_target_array)
            
            train_loader = DataLoader(dataset=train_, 
                                      batch_size=batch_size, 
                                      shuffle = True, 
                                      # num_workers=8,
                                     )
            val_loader = DataLoader(dataset=val_, 
                                    batch_size=batch_size, 
                                    shuffle = False , 
                                    # num_workers=8
                                   )
            
            
            # model = FogRnnModel()
            model = FogRnnModel(numeraical_linear_size = 96,
                                model_size = 256,
                                linear_out = 256,)            
            model = model.to(device)
            
            param_optimizer = list(model.named_parameters())
            no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
            optimizer_grouped_parameters = [
                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 
                 'weight_decay': weight_decay
                },
                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 
                 'weight_decay': 0.0
                }]
            optimizer = AdamW(optimizer_grouped_parameters,
                              lr=lr,
                              weight_decay=weight_decay,
                              )
            num_train_optimization_steps = int(len(train_loader) * n_epochs)
            scheduler = get_linear_schedule_with_warmup(optimizer,
                                                        num_warmup_steps=num_warmup_steps,
                                                        num_training_steps=num_train_optimization_steps)
            criterion = nn.BCEWithLogitsLoss()
            best_val_score = 0
            
            for epoch in range(n_epochs):
                model.train() 
                train_losses_batch = []
                val_losses_batch = []
                epoch_loss = 0
                train_preds = np.ndarray((0,3))   # array([], shape=(0, 3), dtype=float64)
                
                tk0 = tqdm_nb(train_loader, total=len(train_loader), desc="train_loader: ")
                for d in tk0:
                    # ======================================================
                    # data loader
                    # ======================================================
                    input_data_numerical_array = d['input_data_numerical_array'].to(device)
                    input_data_mask_array      = d['input_data_mask_array'].to(device)
                    input_data_valid_array     = d['input_data_valid_array'].to(device)
                    attention_mask             = d['attention_mask'].to(device)
                    y                          = d["y"].to(device)
                    
                    optimizer.zero_grad()
                    output = model(input_data_numerical_array, 
                                   input_data_mask_array,
                                   attention_mask)
                    loss = criterion(output[input_data_mask_array == 1], y[input_data_mask_array == 1])
                    
                    # output1 = output[:, :, [1,2]]
                    # output2 = output[:, :, 0]
                    # y1 = y[:, :, [1,2]]
                    # y2 = y[:, :, 0]
                    # loss1 = criterion(output1[input_data_mask_array == 1], y1[input_data_mask_array == 1])
                    # loss2 = criterion(output2[input_data_mask_array == 1], y2[input_data_mask_array == 1])
                    # loss = loss1*0.8 + loss2*0.2
                    
                    loss.backward()
                    optimizer.step()
                    scheduler.step()
                    train_losses_batch.append(loss.item())
                train_loss = np.mean(train_losses_batch)
                
                
                # ======================================================
                # eval
                # ======================================================
                model.eval()       # switch model to the evaluation mode
                
                val_preds = np.ndarray((0, 5000, 3))
                tk0 = tqdm_nb(val_loader, total=len(val_loader), desc="val_loader  : ")
                with torch.no_grad():  # Do not calculate gradient since we are only predicting
                    # Predicting on validation set
                    for d in tk0:
                        input_data_numerical_array = d['input_data_numerical_array'].to(device)
                        input_data_mask_array      = d['input_data_mask_array'].to(device)
                        attention_mask             = d['attention_mask'].to(device)
                        
                        output = model(input_data_numerical_array, 
                                       input_data_mask_array,
                                       attention_mask)
                        val_preds = np.concatenate([val_preds, output.sigmoid().detach().cpu().numpy()], axis=0)
                
                pred_valid_index = (val_mask_array == 1) & (val_pred_array == 1) & (val_valid_array == 1)
                StartHesitation = average_precision_score(val_target_array[pred_valid_index][:, 0],
                                                          val_preds[pred_valid_index][:, 0])
                Turn = average_precision_score(val_target_array[pred_valid_index][:, 1],
                                               val_preds[pred_valid_index][:, 1])
                Walking = average_precision_score(val_target_array[pred_valid_index][:, 2],
                                                  val_preds[pred_valid_index][:, 2])
                # map_score = np.mean([StartHesitation, Turn, Walking])
                map_score = np.mean([Turn, Walking])
                
                # LOGGER.info(f"fold: {fold+1} epoch: {epoch+1},train loss {train_loss} map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                print(f"fold {fold+1} epoch {epoch+1}\ntrain loss {train_loss} map:{map_score}\nstart_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                
                if map_score >= best_val_score:
                    print("save weight")
                    best_val_score = map_score
                    best_val_preds = val_preds.copy()
                    torch.save(model.state_dict(), f"./output/exp/ex{ex}/ex{ex}_model/ex{ex}_fold{fold+1}_epoch{epoch+1}__map{best_val_score:.6f}.pth")
                print()
            y_oof[valid_idx] = best_val_preds
    
    np.save(f"./output/exp/ex{ex}/ex{ex}_oof.npy", y_oof)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.82 µs


<br>

#### oof score

In [167]:
if TRAIN_FLAG:
    val_pred_index = (mask_array == 1) & (pred_use_array == 1) & (valid_array == 1)
    StartHesitation = average_precision_score(target_array[val_pred_index][:,0], 
                                              y_oof[val_pred_index][:,0])
    Turn            = average_precision_score(target_array[val_pred_index][:,1], 
                                              y_oof[val_pred_index][:,1])
    Walking         = average_precision_score(target_array[val_pred_index][:,2],
                                              y_oof[val_pred_index][:,2])

    map_score = np.mean([StartHesitation, Turn, Walking])
    # LOGGER.info(f"cv map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
    print(f"cv map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")

<br>
<br>

## <font color=blue>**ROUND 2**</font>

<br>
<br>

## <font color=red>**Predicting**</font>: **Making** <font color=red>**pseudo**</font> **label** (round2: use models of ex175 in round 1)


<br>

### <font color=maroon>ex175_defog_gru_inference_notype_15000.ipynbb</font>

<br>

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/exp/ex175_defog_gru_inference_notype_15000.ipynb" style="text-decoration:none">ex175_defog_gru_inference_notype_15000.ipynb</a>

In [168]:
debug = False
ex = "175_notype_pseudo_target"
if not os.path.exists(f"./output/exp/ex{ex}"):
    os.makedirs(f"./output/exp/ex{ex}")
    os.makedirs(f"./output/exp/ex{ex}/ex{ex}_model")

<br>

#### load data & preprocessing

In [169]:
seq_len = 15000

id_path        = f"./output/fe/fe074_notype/fe074_notype_id.parquet"
numerical_path = f"./output/fe/fe074_notype/fe074_notype_num_array.npy"
target_path    = f"./output/fe/fe074_notype/fe074_notype_target_array.npy"
mask_path      = f"./output/fe/fe074_notype/fe074_notype_mask_array.npy"
valid_path     = f"./output/fe/fe074_notype/fe074_notype_valid_array.npy"
pred_use_path  = f"./output/fe/fe074_notype/fe074_notype_pred_use_array.npy"
time_path      = f"./output/fe/fe074_notype/fe074_notype_time_array.npy"

df_id           = pd.read_parquet(id_path)
numerical_array = np.load(numerical_path)
target_array    = np.load(target_path)
mask_array      = np.load(mask_path)
valid_array     = np.load(valid_path)
pred_use_array  = np.load(pred_use_path)
time_array      = np.load(time_path)

<br>

#### config

In [170]:
# config
seed = 0
shuffle = True
n_splits = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model config
batch_size = 24
n_epochs = 15
lr = 1e-3
weight_decay = 0.05
num_warmup_steps = 10

<br>

#### main (predict)

In [171]:
models_list = os.listdir(f"./output/exp/ex{ex}/ex{ex}_model")
models_list

[]

In [172]:
if PREDICT_FLAG:
# with timer("gru"):
    set_seed(seed)
    for fold in range(1, 6):
        with timer(f"fold {fold}"):
            val_numerical_array = numerical_array.copy()
            val_target_array = target_array.copy()
            val_mask_array = mask_array.copy()
            val_valid_array = valid_array.copy()
            val_pred_array = pred_use_array.copy()

            val_ = FogDataset(val_numerical_array, 
                              val_mask_array,
                              val_valid_array, 
                              train=True,
                              y=val_target_array)

            val_loader = DataLoader(dataset=val_, 
                                    batch_size=batch_size, 
                                    shuffle = False,
                                    # num_workers=8
                                   )

            model = FogRnnModel()
            # model.load_state_dict(torch.load(f"./output/exp/ex{ex}/ex{ex}_model/ex{ex}_{fold}.pth"))
            model.load_state_dict(torch.load(f"./output/exp/ex{ex}/ex{ex}_model/" + models_list[fold-1]))
            model = model.to(device)
            model.eval()  # switch model to the evaluation mode
            
            val_preds = np.ndarray((0, seq_len, 3))
            tk0 = tqdm(val_loader, total=len(val_loader))
            with torch.no_grad():  # Do not calculate gradient since we are only predicting
                # Predicting on validation set
                for d in tk0:
                    input_data_numerical_array = d['input_data_numerical_array'].to(device)
                    input_data_mask_array      = d['input_data_mask_array'].to(device)
                    attention_mask             = d['attention_mask'].to(device)
                    output = model(input_data_numerical_array, 
                                   input_data_mask_array,
                                   attention_mask)
                    val_preds = np.concatenate([val_preds, output.sigmoid().detach().cpu().numpy()], axis=0)
            # np.save(f"./output/exp/ex{ex}/ex{ex}_notype_{fold}_oof_{seq_len}.npy",val_preds)
            np.save(f"./output/exp/ex{ex}/ex{ex}_R2_{fold}_oof_{seq_len}.npy",val_preds)            

In [173]:
if PREDICT_FLAG:
    id_array = df_id["Id"].values
    for i in range(1, 6):
        pred_all = []
        # pred = np.load(f"./output/exp/ex{ex}/ex{ex}_notype_{i}_oof_{seq_len}.npy")
        pred = np.load(f"./output/exp/ex{ex}/ex{ex}_R2_{i}_oof_{seq_len}.npy")
        for v in tqdm(range(len(pred_use_array))):
            use_ = pred_use_array[v, :] == 1
            pred_ = pred[v, use_ == 1, :]
            time_ = time_array[v, use_ == 1]
            Id = id_array[v]
            pred_df = pd.DataFrame()
            pred_df["Time"] = time_
            pred_df["Id"] = Id
            pred_df["StartHesitation"] = pred_[:, 0]
            pred_df["Turn"] = pred_[:, 1]
            pred_df["Walking"] = pred_[:, 2]
            pred_all.append(pred_df)
        pred_all = pd.concat(pred_all).reset_index(drop=True)
        # pred_all.to_parquet(f"./output/exp/ex{ex}/ex{ex}_notype_{i}_pred_{seq_len}.parquet")
        pred_all.to_parquet(f"./output/exp/ex{ex}/ex{ex}_R2_{i}_pred_{seq_len}.parquet")    

<br>
<br>

## **Feature Engineering with** <font color=red>**pseudo**</font>  **label** 

<br>


### <font color=maroon>fe078_notype_pseudo_ex175_15000.ipynb</font>

<br>

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/fe/fe078_notype_pseudo_ex175_15000.ipynb" style="text-decoration:none">fe078_notype_pseudo_ex175_15000.ipynb</a>

In [174]:
fe = "078_notype_pseudo"
ex = "175_notype_pseudo_target"
if not os.path.exists(f"./output/fe/fe{fe}"):
    os.makedirs(f"./output/fe/fe{fe}")
    os.makedirs(f"./output/fe/fe{fe}/save")

In [175]:
# DEFOG_META_PATH = "../data/defog_metadata.csv"
# DEFOG_FOLDER = "../data/train/notype/*.csv"

defog_meta = pd.read_parquet("./output/fe/fe039/fe039_defog_meta.parquet")

In [176]:
cols = ["AccV", "AccML", "AccAP"]
num_cols = ["AccV", "AccML", "AccAP",
            'AccV_lag_diff', 'AccV_lead_diff', 
            'AccML_lag_diff', 'AccML_lead_diff',
            'AccAP_lag_diff', 'AccAP_lead_diff']

target_use_cols = ["Event"]
target_cols = ["StartHesitation", "Turn", "Walking"]
seq_len = 5000
shift = 2500
offset = 1250

In [177]:
# data_list = glob.glob(DEFOG_FOLDER)
if TRAIN_FLAG:
    for fold in range(1, 6):
        print(fold)
        # pred = pd.read_parquet(f"./output/exp/ex{ex}/ex{ex}_notype_{fold}_pred_15000.parquet")
        pred = pd.read_parquet(f"./output/exp/ex{ex}/ex{ex}_R2_{fold}_pred_15000.parquet")
        target_array = []
        for i,s in tqdm(zip(defog_meta["Id"].values, defog_meta["sub_id"].values)):
            path = root_data + f"train/notype/{i}.csv"
            if path in [x.replace("\\", "/") for x in train_notype]:
            # if path in data_list:
                df = pd.read_csv(path)
                df_ = pred[pred["Id"] == i].reset_index(drop=True)
                df = df.merge(df_, how="left", on="Time")
                df["target_max"] = np.argmax(df[["StartHesitation", "Turn", "Walking"]].values, axis=1)

                df.loc[df["target_max"] == 0, "StartHesitation"] = 1
                df.loc[df["target_max"] == 0, ["Turn","Walking"]] = 0

                df.loc[df["target_max"] == 1, "Turn"] = 1
                df.loc[df["target_max"] == 1, ["StartHesitation","Walking"]] = 0

                df.loc[df["target_max"] == 2, "Walking"] = 1
                df.loc[df["target_max"] == 2, ["StartHesitation","Turn"]] = 0

                df.loc[df["Event"] == 0, ["StartHesitation", "Turn", "Walking"]] = 0

                df["valid"] = df["Valid"] & df["Task"]
                df["valid"] = df["valid"].astype(int)
                batch = (len(df)-1) // shift
                target = df[target_cols].values
                target_array_ = np.zeros([batch, seq_len, 3])
                for n,b in enumerate(range(batch)):
                    if b == (batch - 1):
                        target_ = target[b*shift : ]
                        target_array_[b, :len(target_), :] = target_
                    elif b == 0:
                        target_ = target[b*shift : b*shift + seq_len]
                        target_array_[b, :, :] = target_
                    else:
                        target_ = target[b*shift : b*shift + seq_len]
                        target_array_[b, :, :] = target_

                target_array.append(target_array_)
        target_array = np.concatenate(target_array, axis=0)
        np.save(f"./output/fe/fe{fe}/fe{fe}_target_array_{fold}.npy", target_array)

<br>
<br>

##  <font color=red>**Training**</font> **with** <font color=red>**pseudo**</font> **label** (round2)

<br>

### load data & preprocessing

In [178]:
id_path        = f"./output/fe/fe047/fe047_id.parquet"
numerical_path = f"./output/fe/fe047/fe047_num_array.npy"
target_path    = f"./output/fe/fe047/fe047_target_array.npy"
mask_path      = f"./output/fe/fe047/fe047_mask_array.npy"
valid_path     = f"./output/fe/fe047/fe047_valid_array.npy"
pred_use_path  = f"./output/fe/fe047/fe047_pred_use_array.npy"

df_id           = pd.read_parquet(id_path)
numerical_array = np.load(numerical_path)
target_array    = np.load(target_path)
mask_array      = np.load(mask_path)
valid_array     = np.load(valid_path)
pred_use_array  = np.load(pred_use_path)

In [179]:
target1 = []
target2 = []
target3 = []
for i in range(len(target_array)):
    target1.append(np.sum(target_array[i,:,0]))
    target2.append(np.sum(target_array[i,:,1]))
    target3.append(np.sum(target_array[i,:,2]))

df_id["target1"] = target1
df_id["target2"] = target2
df_id["target3"] = target3
df_id["target1_1"] = df_id["target1"] > 0
df_id["target2_1"] = df_id["target2"] > 0
df_id["target3_1"] = df_id["target3"] > 0
df_id["target1_1"] = df_id["target1_1"].astype(np.int)
df_id["target2_1"] = df_id["target2_1"].astype(np.int)
df_id["target3_1"] = df_id["target3_1"].astype(np.int)

df_id["group"] = 0
df_id.loc[df_id["target1_1"] > 0,"group"] = 1
df_id.loc[df_id["target2_1"] > 0,"group"] = 2
df_id.loc[df_id["target3_1"] > 0,"group"] = 3

df_id["group"].value_counts()

0    4107
2     897
3     354
1      11
Name: group, dtype: int64

In [180]:
pseudo_id_path        = f"./output/fe/fe061_notype/fe061_notype_id.parquet"
pseudo_numerical_path = f"./output/fe/fe061_notype/fe061_notype_num_array.npy"
pseudo_mask_path      = f"./output/fe/fe061_notype/fe061_notype_mask_array.npy"
pseudo_valid_path     = f"./output/fe/fe061_notype/fe061_notype_valid_array.npy"
pseudo_pred_use_path  = f"./output/fe/fe061_notype/fe061_notype_pred_use_array.npy"

pseudo_numerical_array = np.load(pseudo_numerical_path)
pseudo_mask_array      = np.load(pseudo_mask_path)
pseudo_valid_array     = np.load(pseudo_valid_path)
pseudo_pred_use_array  = np.load(pseudo_pred_use_path)

<br>

### config

In [181]:
seq_len = 5000

# config
seed = 0
shuffle = True
n_splits = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model config
batch_size = 24
n_epochs = 10
lr = 1e-3
weight_decay = 0.05
num_warmup_steps = 10

<br>

### <font color=maroon>ex185_defog_gru_pseudo2.ipynb</font>

<br>

Code below originates from: 
* <a href="https://github.com/TakoiHirokazu/Kaggle-Parkinsons-Freezing-of-Gait-Prediction/blob/main/takoi/exp/ex185_defog_gru_pseudo2.ipynb" style="text-decoration:none">ex185_defog_gru_pseudo2.ipynb</a>

In [182]:
debug = False
ex = "185_notype_pseudo_target"
if not os.path.exists(f"./output/exp/ex{ex}"):
    os.makedirs(f"./output/exp/ex{ex}")
    os.makedirs(f"./output/exp/ex{ex}/ex{ex}_model")

In [183]:
pseudo_target = "078_notype_pseudo"

<br>

#### main

In [184]:
%%time

# with timer("gru"):
if TRAIN_FLAG:
    set_seed(seed)
    y_oof = np.empty([len(target_array), seq_len, 3])      # Abrachan: out of fold
    gkf = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state = seed)
    for fold, (train_idx, valid_idx) in enumerate(gkf.split(numerical_array, 
                                                            y = df_id["group"].values,
                                                            groups=df_id["subject"].values)):
        # LOGGER.info(f"start fold:{fold+1}")
        print(f"start fold: {fold+1}")
        
        with timer(f"fold {fold+1}"):
            train_numerical_array = numerical_array[train_idx]
            train_target_array = target_array[train_idx]
            train_mask_array = mask_array[train_idx]
            train_valid_array = valid_array[train_idx]
            
            pseudo_target_array = np.load(f"./output/fe/fe{pseudo_target}/fe{pseudo_target}_target_array_{fold+1}.npy")
            train_numerical_array = np.concatenate([train_numerical_array, pseudo_numerical_array], axis=0)
            train_target_array = np.concatenate([train_target_array, pseudo_target_array], axis=0)
            train_mask_array = np.concatenate([train_mask_array, pseudo_mask_array], axis=0)
            train_valid_array = np.concatenate([train_valid_array, pseudo_valid_array], axis=0)

            val_numerical_array = numerical_array[valid_idx]
            val_target_array = target_array[valid_idx]
            val_mask_array = mask_array[valid_idx]
            val_valid_array = valid_array[valid_idx]
            val_pred_array = pred_use_array[valid_idx]
            
            train_ = FogDataset(train_numerical_array,
                                train_mask_array,
                                train_valid_array,
                                train=True,
                                y=train_target_array)
            val_ = FogDataset(val_numerical_array,
                              val_mask_array,
                              val_valid_array,
                              train=True,
                              y=val_target_array)
            
            train_loader = DataLoader(dataset=train_, 
                                      batch_size=batch_size, 
                                      shuffle = True, 
                                      # num_workers=8,
                                     )
            val_loader = DataLoader(dataset=val_, 
                                    batch_size=batch_size, 
                                    shuffle = False , 
                                    # num_workers=8
                                   )
            
            
            model = FogRnnModel()
            model = model.to(device)
            
            param_optimizer = list(model.named_parameters())
            no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
            optimizer_grouped_parameters = [
                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 
                 'weight_decay': weight_decay
                },
                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 
                 'weight_decay': 0.0
                }]
            optimizer = AdamW(optimizer_grouped_parameters,
                              lr=lr,
                              weight_decay=weight_decay,
                              )
            num_train_optimization_steps = int(len(train_loader) * n_epochs)
            scheduler = get_linear_schedule_with_warmup(optimizer,
                                                        num_warmup_steps=num_warmup_steps,
                                                        num_training_steps=num_train_optimization_steps)
            criterion = nn.BCEWithLogitsLoss()
            best_val_score = 0
            
            for epoch in range(n_epochs):
                model.train() 
                train_losses_batch = []
                val_losses_batch = []
                epoch_loss = 0
                train_preds = np.ndarray((0,3))   # array([], shape=(0, 3), dtype=float64)
                
                tk0 = tqdm_nb(train_loader, total=len(train_loader), desc="train_loader: ")
                for d in tk0:
                    # ======================================================
                    # data loader
                    # ======================================================
                    input_data_numerical_array = d['input_data_numerical_array'].to(device)
                    input_data_mask_array      = d['input_data_mask_array'].to(device)
                    input_data_valid_array     = d['input_data_valid_array'].to(device)
                    attention_mask             = d['attention_mask'].to(device)
                    y                          = d["y"].to(device)
                    
                    optimizer.zero_grad()
                    output = model(input_data_numerical_array, 
                                   input_data_mask_array,
                                   attention_mask)
                    loss = criterion(output[input_data_mask_array == 1], y[input_data_mask_array == 1])
                    
                    # output1 = output[:, :, [1,2]]
                    # output2 = output[:, :, 0]
                    # y1 = y[:, :, [1,2]]
                    # y2 = y[:, :, 0]
                    # loss1 = criterion(output1[input_data_mask_array == 1], y1[input_data_mask_array == 1])
                    # loss2 = criterion(output2[input_data_mask_array == 1], y2[input_data_mask_array == 1])
                    # loss = loss1*0.8 + loss2*0.2
                    
                    loss.backward()
                    optimizer.step()
                    scheduler.step()
                    train_losses_batch.append(loss.item())
                train_loss = np.mean(train_losses_batch)
                
                
                # ======================================================
                # eval
                # ======================================================
                model.eval()       # switch model to the evaluation mode
                
                val_preds = np.ndarray((0, seq_len, 3))
                tk0 = tqdm_nb(val_loader, total=len(val_loader), desc="val_loader  : ")
                with torch.no_grad():  # Do not calculate gradient since we are only predicting
                    # Predicting on validation set
                    for d in tk0:
                        input_data_numerical_array = d['input_data_numerical_array'].to(device)
                        input_data_mask_array      = d['input_data_mask_array'].to(device)
                        attention_mask             = d['attention_mask'].to(device)
                        
                        output = model(input_data_numerical_array, 
                                       input_data_mask_array,
                                       attention_mask)
                        val_preds = np.concatenate([val_preds, output.sigmoid().detach().cpu().numpy()], axis=0)
                
                pred_valid_index = (val_mask_array == 1) & (val_pred_array == 1) & (val_valid_array == 1)
                StartHesitation = average_precision_score(val_target_array[pred_valid_index][:, 0],
                                                          val_preds[pred_valid_index][:, 0])
                Turn = average_precision_score(val_target_array[pred_valid_index][:, 1],
                                               val_preds[pred_valid_index][:, 1])
                Walking = average_precision_score(val_target_array[pred_valid_index][:, 2],
                                                  val_preds[pred_valid_index][:, 2])
                # map_score = np.mean([StartHesitation, Turn, Walking])
                map_score = np.mean([Turn, Walking])
                
                # LOGGER.info(f"fold: {fold+1} epoch: {epoch+1},train loss {train_loss} map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                print(f"fold {fold+1} epoch {epoch+1}\ntrain loss {train_loss} map:{map_score}\nstart_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
                
                if map_score >= best_val_score:
                    print("save weight")
                    best_val_score = map_score
                    best_val_preds = val_preds.copy()
                    torch.save(model.state_dict(), f"./output/exp/ex{ex}/ex{ex}_model/ex{ex}_fold{fold+1}_epoch{epoch+1}__map{best_val_score:.6f}.pth")
                print()
            y_oof[valid_idx] = best_val_preds
    
    np.save(f"./output/exp/ex{ex}/ex{ex}_oof.npy", y_oof)

CPU times: user 0 ns, sys: 5 µs, total: 5 µs
Wall time: 8.82 µs


<br>

#### oof score

In [185]:
if TRAIN_FLAG:
    val_pred_index = (mask_array == 1) & (pred_use_array == 1) & (valid_array == 1)
    StartHesitation = average_precision_score(target_array[val_pred_index][:,0], 
                                              y_oof[val_pred_index][:,0])
    Turn            = average_precision_score(target_array[val_pred_index][:,1], 
                                              y_oof[val_pred_index][:,1])
    Walking         = average_precision_score(target_array[val_pred_index][:,2],
                                              y_oof[val_pred_index][:,2])

    map_score = np.mean([StartHesitation, Turn, Walking])
    # LOGGER.info(f"cv map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")
    print(f"cv map:{map_score} start_hesi:{StartHesitation} turn:{Turn} walking :{Walking}")

<br>

## **models choosed**

<br>

<font color=maroon size=4>The following numbered models were used for the final submission:</font>
* <font color=maroon size=4>ex153</font>
* <font color=maroon size=4>ex179</font>
* <font color=maroon size=4>ex185</font>
* <font color=maroon size=4>ex204</font>

<br>
<br>
<br>


# 【**Inference**】

<br>

Code below originates from: 
* <a href="https://www.kaggle.com/code/takoihiraokazu/cv-ensemble-sub-0607-1" style="text-decoration:none">[cv]ensemble_sub_0607_1</a>

<br>

## Config

In [186]:
# SUB_PATH          = "/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/sample_submission.csv"
# DEFOG_DATA_PATH   = "/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/test/defog/*.csv"
# TDCSFOG_DATA_PATH = "/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/test/tdcsfog/*.csv"

In [187]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bs = 32

In [188]:
# sub = pd.read_csv(SUB_PATH)

# # sub = pd.read_csv(root_data + 'sample_submission.csv')

In [189]:
df_all = []

<br>

## Helpers

<br>

### def `preprocess()`

In [190]:
def preprocess(numerical_array, mask_array):
    
    attention_mask = mask_array == 0

    return {'input_data_numerical_array': numerical_array,
            'input_data_mask_array': mask_array,
            'attention_mask': attention_mask,
           }

<br>

### class `FogDataset()`

In [191]:
class FogDataset(Dataset):
    def __init__(self, numerical_array, mask_array, train = True, y = None):
        self.numerical_array = numerical_array
        self.mask_array = mask_array
        self.train = train
        self.y = y
    
    
    def __len__(self):
        return len(self.numerical_array)
    
    
    def __getitem__(self, item):
        data = preprocess(self.numerical_array[item], self.mask_array[item])

        # Return the processed data where the lists are converted to `torch.tensor`s
        if self.train : 
            return {
              'input_data_numerical_array': torch.tensor(data['input_data_numerical_array'], dtype=torch.float32),
              'input_data_mask_array': torch.tensor(data['input_data_mask_array'],  dtype=torch.long),              
              'attention_mask': torch.tensor(data["attention_mask"], dtype=torch.bool),
              "y": torch.tensor(self.y[item], dtype=torch.float32)
            }
        else:
            return {
              'input_data_numerical_array': torch.tensor(data['input_data_numerical_array'], dtype=torch.float32),
              'input_data_mask_array': torch.tensor(data['input_data_mask_array'], dtype=torch.long),
              'attention_mask': torch.tensor(data["attention_mask"], dtype=torch.bool),
               }

<br>

### class `TdcsfogRnnModel()`

In [192]:
class TdcsfogRnnModel(nn.Module):
    def __init__(self, 
                 dropout=0.2,
                 input_numerical_size=12,
                 numeraical_linear_size = 64,
                 model_size = 128,
                 linear_out = 128,
                 out_size=3):
        
        super(TdcsfogRnnModel, self).__init__()
        
        self.numerical_linear  = nn.Sequential(nn.Linear(input_numerical_size, numeraical_linear_size),
                                               nn.LayerNorm(numeraical_linear_size))
        
        self.rnn = nn.GRU(numeraical_linear_size, 
                          model_size,
                          num_layers = 2, 
                          batch_first=True,
                          bidirectional=True)
        
        self.linear_out  = nn.Sequential(nn.Linear(model_size*2, linear_out),
                                         nn.LayerNorm(linear_out),
                                         nn.ReLU(),
                                         nn.Dropout(dropout),
                                         nn.Linear(linear_out, out_size))
        self._reinitialize()
        
        
        
    def _reinitialize(self):
        """Tensorflow/Keras-like initialization"""
        for name, p in self.named_parameters():
            if 'rnn' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
        
        
        
    def forward(self, numerical_array, mask_array, attention_mask):
        numerical_embedding = self.numerical_linear(numerical_array)
        output, _ = self.rnn(numerical_embedding)
        output = self.linear_out(output)
        return output

<br>

### class `TdcsfogRnnModel2()`

In [193]:
class TdcsfogRnnModel2(nn.Module):
    def __init__(self, 
                 dropout=0.2,
                 input_numerical_size=12,
                 numeraical_linear_size = 64,
                 model_size = 128,
                 linear_out = 128,
                 out_size=3):
        
        super(TdcsfogRnnModel2, self).__init__()
        
        self.numerical_linear  = nn.Sequential(nn.Linear(input_numerical_size, numeraical_linear_size),
                                               nn.LayerNorm(numeraical_linear_size))
        
        self.rnn = nn.GRU(numeraical_linear_size, 
                          model_size,
                          num_layers = 2, 
                          batch_first=True,
                          bidirectional=True)
        
        self.linear_out  = nn.Sequential(nn.Linear(model_size*2, linear_out),
                                         nn.LayerNorm(linear_out),
                                         nn.ReLU(),
                                         nn.Dropout(dropout),
                                         # nn.Linear(linear_out, out_size)
                                         )
        self.out1 = nn.Linear(linear_out, out_size)
        self.out2 = nn.Linear(linear_out, out_size)
        
        
        self._reinitialize()
        
        
        
    def _reinitialize(self):
        """Tensorflow/Keras-like initialization"""
        for name, p in self.named_parameters():
            if 'rnn' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
        
        
        
    def forward(self, numerical_array, mask_array, attention_mask):
        numerical_embedding = self.numerical_linear(numerical_array)
        output, _ = self.rnn(numerical_embedding)
        output = self.linear_out(output)
        # return output
        
        output1 = self.out1(output)
        output2 = self.out2(output)
        return output1, output2

<br>

### class `DefogRnnModel()`

In [194]:
class DefogRnnModel(nn.Module):
    def __init__(self, 
                 dropout=0.2,
                 input_numerical_size=9,
                 numeraical_linear_size = 64,
                 model_size = 128,
                 linear_out = 128,
                 out_size=3):
        
        super(DefogRnnModel, self).__init__()
        
        self.numerical_linear  = nn.Sequential(nn.Linear(input_numerical_size, numeraical_linear_size),
                                               nn.LayerNorm(numeraical_linear_size))
        
        self.rnn = nn.GRU(numeraical_linear_size, 
                          model_size,
                          num_layers = 2, 
                          batch_first=True,
                          bidirectional=True)
        
        self.linear_out  = nn.Sequential(nn.Linear(model_size*2, linear_out),
                                         nn.LayerNorm(linear_out),
                                         nn.ReLU(),
                                         nn.Dropout(dropout),
                                         nn.Linear(linear_out, out_size))
        self._reinitialize()
        
        
        
    def _reinitialize(self):
        """Tensorflow/Keras-like initialization"""
        for name, p in self.named_parameters():
            if 'rnn' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
        
        
        
    def forward(self, numerical_array, mask_array, attention_mask):
        numerical_embedding = self.numerical_linear(numerical_array)
        output, _ = self.rnn(numerical_embedding)
        output = self.linear_out(output)
        return output

<br>

### class `DefogRnnModel2()`

In [195]:
class DefogRnnModel2(nn.Module):
    def __init__(self, 
                 dropout=0.2,
                 input_numerical_size=9,
                 numeraical_linear_size = 96,
                 model_size = 256,
                 linear_out = 256,
                 out_size=3):
        
        super(DefogRnnModel2, self).__init__()
        
        self.numerical_linear  = nn.Sequential(nn.Linear(input_numerical_size, numeraical_linear_size),
                                               nn.LayerNorm(numeraical_linear_size))
        
        self.rnn = nn.GRU(numeraical_linear_size, 
                          model_size,
                          num_layers = 2, 
                          batch_first=True,
                          bidirectional=True)
        
        self.linear_out  = nn.Sequential(nn.Linear(model_size*2, linear_out),
                                         nn.LayerNorm(linear_out),
                                         nn.ReLU(),
                                         nn.Dropout(dropout),
                                         nn.Linear(linear_out, out_size))
        self._reinitialize()
        
        
        
    def _reinitialize(self):
        """Tensorflow/Keras-like initialization"""
        for name, p in self.named_parameters():
            if 'rnn' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
        
        
        
    def forward(self, numerical_array, mask_array, attention_mask):
        numerical_embedding = self.numerical_linear(numerical_array)
        output, _ = self.rnn(numerical_embedding)
        output = self.linear_out(output)
        return output

<br>

### class `Defog3Model()`

In [196]:
class Defog3Model(nn.Module):
    def __init__(self, 
                 dropout=0.2,
                 input_numerical_size=9,
                 numeraical_linear_size = 64,
                 model_size = 128,
                 linear_out = 128,
                 out_size=3):
        
        super(Defog3Model, self).__init__()
        
        self.numerical_linear  = nn.Sequential(nn.Linear(input_numerical_size, numeraical_linear_size),
                                               nn.LayerNorm(numeraical_linear_size))
        
        self.lstm = nn.GRU(numeraical_linear_size, 
                          model_size,
                          num_layers = 2, 
                          batch_first=True,
                          bidirectional=True)
        
        self.linear_out  = nn.Sequential(nn.Linear(model_size*2, linear_out),
                                         nn.LayerNorm(linear_out),
                                         nn.ReLU(),
                                         nn.Dropout(dropout),
                                         nn.Linear(linear_out, out_size))
        self._reinitialize()
        
        
        
    def _reinitialize(self):
        """Tensorflow/Keras-like initialization"""
        for name, p in self.named_parameters():
            if 'lstm' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
        
        
        
    def forward(self, numerical_array, mask_array, attention_mask):
        numerical_embedding = self.numerical_linear(numerical_array)
        output, _ = self.rnn(numerical_embedding)
        output = self.linear_out(output)
        return output

<br>

### def `make_pred()`

In [197]:
def make_pred(test_loader, model):
    test_preds = []
    # tk0 = tqdm(test_loader, total=len(test_loader), desc="test_loader: ")
    with torch.no_grad():  # Do not calculate gradient since we are only predicting
        # Predicting on validation set
        # for d in tk0:
        for d in test_loader:
            input_data_numerical_array = d['input_data_numerical_array'].to(device)
            input_data_mask_array      = d['input_data_mask_array'].to(device)
            attention_mask             = d['attention_mask'].to(device)
            output = model(input_data_numerical_array, 
                           input_data_mask_array,
                           attention_mask)
            test_preds.append(output.sigmoid().cpu().numpy())
    test_preds = np.concatenate(test_preds, axis=0)
    return test_preds

<br>

### def `make_pred2()`

In [198]:
def make_pred2(test_loader, model):
    test_preds = []
    # tk0 = tqdm(test_loader, total=len(test_loader), desc="test_loader: ")
    with torch.no_grad():  # Do not calculate gradient since we are only predicting
        # Predicting on validation set
        # for d in tk0:
        for d in test_loader:
            input_data_numerical_array = d['input_data_numerical_array'].to(device)
            input_data_mask_array      = d['input_data_mask_array'].to(device)
            attention_mask             = d['attention_mask'].to(device)
            
            output, _ = model(input_data_numerical_array, 
                             input_data_mask_array,
                             attention_mask)
            test_preds.append(output.sigmoid().cpu().numpy())
    test_preds = np.concatenate(test_preds, axis=0)
    return test_preds

<br>

## <font color=red>tdcsfog models</font>

### checkpoint paths

In [199]:
# tdcsfog_path1   = [f"/kaggle/input/fog-ex143/ex143_{i}.pth" for i in range(5)] # len 3000 cv TdcsfogRnnModel
# tdcsfog_path3_1 = [f"/kaggle/input/fog-ex145/ex145_{i}.pth" for i in range(5)] # len 3000 TdcsfogRnnModel  
# tdcsfog_path3_2 = [f"/kaggle/input/fog-ex146/ex146_{i}.pth" for i in range(5)] # len 3000 TdcsfogRnnModel 
# tdcsfog_path3_3 = [f"/kaggle/input/fog-ex147/ex147_{i}.pth" for i in range(5)] # len 3000 TdcsfogRnnModel
# tdcsfog_path4_1 = [f"/kaggle/input/fog-ex182/ex182_{i}.pth" for i in range(5)] # len 3000 TdcsfogRnnModel  
# tdcsfog_path4_2 = [f"/kaggle/input/fog-ex183/ex183_{i}.pth" for i in range(5)] # len 3000 TdcsfogRnnModel 
# tdcsfog_path4_3 = [f"/kaggle/input/fog-ex184/ex184_{i}.pth" for i in range(5)] # len 3000 TdcsfogRnnModel

In [200]:
tdcsfog_path1     = glob.glob("/kaggle/input/parkinson-fog-prediction/ex143*")
tdcsfog_path3_1   = glob.glob("/kaggle/input/parkinson-fog-prediction/ex145*")
tdcsfog_path3_2   = glob.glob("/kaggle/input/parkinson-fog-prediction/ex146*")
tdcsfog_path3_3   = glob.glob("/kaggle/input/parkinson-fog-prediction/ex147*")
tdcsfog_path4_1   = glob.glob("/kaggle/input/parkinson-fog-prediction/ex182*")
tdcsfog_path4_2   = glob.glob("/kaggle/input/parkinson-fog-prediction/ex183*")
tdcsfog_path4_3   = glob.glob("/kaggle/input/parkinson-fog-prediction/ex184*")

<br>

### **ex143** (tdcsfog1: 0.2)


#### load checkpoint

In [201]:
# TdcsfogRnnModel()

tdcsfog_model_list1 = []
for i in tdcsfog_path1:  # ex143
    model = TdcsfogRnnModel()
    model.load_state_dict(torch.load(i))
    model = model.to(device)
    model.eval()
    tdcsfog_model_list1.append(model)

<br>

#### predict

In [202]:
%%time

# =========================
# tdcsfog1
# =========================
th_len = 5000
w = 0.20

cols = ["AccV","AccML","AccAP"]
num_cols = ['AccV', 'AccML', 'AccAP', 
            'AccV_lag_diff',  'AccV_lead_diff', 'AccV_cumsum', 
            'AccML_lag_diff', 'AccML_lead_diff', 'AccML_cumsum', 
            'AccAP_lag_diff', 'AccAP_lead_diff', 'AccAP_cumsum']

# tdcsfog_list = glob.glob(TDCSFOG_DATA_PATH)
# for p in tqdm(tdcsfog_list):
for p in tqdm(test_tdcsfog, desc="test_tdcsfog: "):
    # id_values = p.split("/")[-1].split(".")[0]
    id_values = p.split("/")[-1].split(".")[0].split("\\")[-1]
    df = pd.read_csv(p)
    
    if len(df) > th_len:
        seq_len = 5000
        shift = 2500
        offset = 1250
    else:
        seq_len = 3000
        shift = 1500
        offset = 750
        
    batch = (len(df)-1) // shift
    if batch == 0:
        batch = 1
    for c in cols:
        df[f"{c}_lag_diff"] = df[c].diff()
        df[f"{c}_lead_diff"] = df[c].diff(-1)
        df[f"{c}_cumsum"] = df[c].cumsum()
    sc = RobustScaler()
    df[num_cols] = sc.fit_transform(df[num_cols].values)
    df[num_cols] = df[num_cols].fillna(0)
    num = df[num_cols].values
    time = df["Time"].values
    
    num_array = np.zeros([batch,seq_len, 12])
    mask_array = np.zeros([batch,seq_len], dtype=int)
    time_array = np.zeros([batch,seq_len], dtype=int)
    pred_use_array = np.zeros([batch,seq_len], dtype=int)
    
    if len(df) <= seq_len:
        b = 0
        num_ = num.copy()
        time_ = time.copy()
        num_len = len(num_)

        num_array[b,:num_len,:] = num_
        time_array[b,:num_len] = time_
        mask_array[b,:num_len] = 1
        pred_use_array[b,:num_len] = 1
    else:
        for n,b in enumerate(range(batch)):
            if b == (batch - 1):
                num_ = num[b*shift : ]
                time_ = time[b*shift : ]
                num_len = len(num_)

                num_array[b, :num_len, :] = num_
                time_array[b,:num_len] = time_
                mask_array[b,:num_len] = 1
                pred_use_array[b, offset:num_len] = 1
            elif b == 0:
                num_ = num[b*shift : b*shift+seq_len]
                time_ = time[b*shift : b*shift + seq_len]

                num_array[b, :, :] = num_
                time_array[b, :] = time_
                mask_array[b, :] = 1
                pred_use_array[b, :shift+offset] = 1
            else:
                num_ = num[b*shift : b*shift+seq_len]
                time_ = time[b*shift : b*shift + seq_len]

                num_array[b, :, :] = num_
                time_array[b, :] = time_
                mask_array[b, :] = 1
                pred_use_array[b,offset : shift+offset] = 1
            
            

    test_ = FogDataset(num_array, mask_array, train=False)
    test_loader = DataLoader(dataset=test_, batch_size=bs, shuffle = False)
    for n,m in enumerate(tdcsfog_model_list1):
        if n == 0:
            pred = make_pred(test_loader,m) / len(tdcsfog_model_list1)
        else:
            pred += make_pred(test_loader,m) / len(tdcsfog_model_list1)
    pred_list = []
    for i in range(batch):
        mask_ = pred_use_array[i]
        pred_ = pred[i, mask_ == 1, :]
        time_ = time_array[i, mask_ == 1]
        
        df_ = pd.DataFrame()
        df_["StartHesitation"] = pred_[:, 0] * w
        df_["Turn"] = pred_[:, 1] * w
        df_["Walking"] = pred_[:, 2] * w
        df_["Time"] = time_
        df_["Id"] = id_values
        df_["Id"] = df_["Id"].astype(str) + "_" + df_["Time"].astype(str)
        
        pred_list.append(df_)
    pred = pd.concat(pred_list).reset_index(drop=True)
    df_all.append(pred)

test_tdcsfog: 100%|██████████| 1/1 [00:00<00:00,  2.10it/s]

CPU times: user 203 ms, sys: 22.2 ms, total: 225 ms
Wall time: 483 ms


<br>

### **ex145, ex146, ex147** (tdcsfog3: 0.4)

#### load checkpoint

In [203]:
# TdcsfogRnnModel()

tdcsfog_model_list3_1 = []
for i in tdcsfog_path3_1:  # ex145
    model = TdcsfogRnnModel()
    model.load_state_dict(torch.load(i))
    model = model.to(device)
    model.eval()
    tdcsfog_model_list3_1.append(model)


tdcsfog_model_list3_2 = []
for i in tdcsfog_path3_2:  # ex146
    model = TdcsfogRnnModel()
    model.load_state_dict(torch.load(i))
    model = model.to(device)
    model.eval()
    tdcsfog_model_list3_2.append(model)


tdcsfog_model_list3_3 = []
for i in tdcsfog_path3_3:  # ex147
    model = TdcsfogRnnModel()
    model.load_state_dict(torch.load(i))
    model = model.to(device)
    model.eval()
    tdcsfog_model_list3_3.append(model)

<br>

#### predict

In [204]:
%%time

# =========================
# tdcsfog3
# =========================
th_len = 5000
w = 0.40

cols = ["AccV","AccML","AccAP"]
num_cols = ['AccV', 'AccML', 'AccAP', 
            'AccV_lag_diff',  'AccV_lead_diff', 'AccV_cumsum', 
            'AccML_lag_diff', 'AccML_lead_diff', 'AccML_cumsum', 
            'AccAP_lag_diff', 'AccAP_lead_diff', 'AccAP_cumsum']

# tdcsfog_list = glob.glob(TDCSFOG_DATA_PATH)
# for p in tqdm(tdcsfog_list):
for p in tqdm(test_tdcsfog, desc="test_tdcsfog: "):
    # id_values = p.split("/")[-1].split(".")[0]
    id_values = p.split("/")[-1].split(".")[0].split("\\")[-1]
    df = pd.read_csv(p)
    
    if len(df) > th_len:
        seq_len = 5000
        shift = 2500
        offset = 1250
    else:
        seq_len = 3000
        shift = 1500
        offset = 750
        
    batch = (len(df)-1) // shift
    if batch == 0:
        batch = 1
    for c in cols:
        df[f"{c}_lag_diff"] = df[c].diff()
        df[f"{c}_lead_diff"] = df[c].diff(-1)
        df[f"{c}_cumsum"] = df[c].cumsum()
    sc = RobustScaler()
    df[num_cols] = sc.fit_transform(df[num_cols].values)
    df[num_cols] = df[num_cols].fillna(0)
    num = df[num_cols].values
    time = df["Time"].values
    
    num_array = np.zeros([batch,seq_len, 12])
    mask_array = np.zeros([batch,seq_len], dtype=int)
    time_array = np.zeros([batch,seq_len], dtype=int)
    pred_use_array = np.zeros([batch,seq_len], dtype=int)
    
    if len(df) <= seq_len:
        b = 0
        num_ = num.copy()
        time_ = time.copy()
        num_len = len(num_)

        num_array[b,:num_len,:] = num_
        time_array[b,:num_len] = time_
        mask_array[b,:num_len] = 1
        pred_use_array[b,:num_len] = 1
    else:
        for n,b in enumerate(range(batch)):
            if b == (batch - 1):
                num_ = num[b*shift : ]
                time_ = time[b*shift : ]
                num_len = len(num_)

                num_array[b, :num_len, :] = num_
                time_array[b,:num_len] = time_
                mask_array[b,:num_len] = 1
                pred_use_array[b, offset:num_len] = 1
            elif b == 0:
                num_ = num[b*shift : b*shift+seq_len]
                time_ = time[b*shift : b*shift + seq_len]

                num_array[b, :, :] = num_
                time_array[b, :] = time_
                mask_array[b, :] = 1
                pred_use_array[b, :shift+offset] = 1
            else:
                num_ = num[b*shift : b*shift+seq_len]
                time_ = time[b*shift : b*shift + seq_len]

                num_array[b, :, :] = num_
                time_array[b, :] = time_
                mask_array[b, :] = 1
                pred_use_array[b,offset : shift+offset] = 1
            
            

    test_ = FogDataset(num_array, mask_array, train=False)
    test_loader = DataLoader(dataset=test_, batch_size=bs, shuffle = False)
    for n,m in enumerate(tdcsfog_model_list3_1):
        if n == 0:
            pred1 = make_pred(test_loader,m) / len(tdcsfog_model_list3_1)
        else:
            pred1 += make_pred(test_loader,m) / len(tdcsfog_model_list3_1)
    for n,m in enumerate(tdcsfog_model_list3_2):
        if n == 0:
            pred2 = make_pred(test_loader,m) / len(tdcsfog_model_list3_2)
        else:
            pred2 += make_pred(test_loader,m) / len(tdcsfog_model_list3_2)
    for n,m in enumerate(tdcsfog_model_list3_3):
        if n == 0:
            pred3 = make_pred(test_loader,m) / len(tdcsfog_model_list3_3)
        else:
            pred3 += make_pred(test_loader,m) / len(tdcsfog_model_list3_3)
    pred = pred1.copy()
    pred[:,:,1] = pred2[:,:,1]
    pred[:,:,2] = pred3[:,:,2]
    
    
    pred_list = []
    for i in range(batch):
        mask_ = pred_use_array[i]
        pred_ = pred[i, mask_ == 1, :]
        time_ = time_array[i, mask_ == 1]
        
        df_ = pd.DataFrame()
        df_["StartHesitation"] = pred_[:, 0] * w
        df_["Turn"] = pred_[:, 1] * w
        df_["Walking"] = pred_[:, 2] * w
        df_["Time"] = time_
        df_["Id"] = id_values
        df_["Id"] = df_["Id"].astype(str) + "_" + df_["Time"].astype(str)
        
        pred_list.append(df_)
    pred = pd.concat(pred_list).reset_index(drop=True)
    df_all.append(pred)

test_tdcsfog: 100%|██████████| 1/1 [00:00<00:00,  2.49it/s]

CPU times: user 382 ms, sys: 5.14 ms, total: 387 ms
Wall time: 407 ms


<br>

### **ex182, ex183, ex184** (tdcsfog4: 0.4)

#### load checkpoint

In [205]:
# TdcsfogRnnModel()

tdcsfog_model_list4_1 = []
for i in tdcsfog_path4_1:  # ex182
    model = TdcsfogRnnModel()
    model.load_state_dict(torch.load(i))
    model = model.to(device)
    model.eval()
    tdcsfog_model_list4_1.append(model)


tdcsfog_model_list4_2 = []
for i in tdcsfog_path4_2:  # ex183
    model = TdcsfogRnnModel()
    model.load_state_dict(torch.load(i))
    model = model.to(device)
    model.eval()
    tdcsfog_model_list4_2.append(model)


tdcsfog_model_list4_3 = []
for i in tdcsfog_path4_3:  # ex184
    model = TdcsfogRnnModel()
    model.load_state_dict(torch.load(i))
    model = model.to(device)
    model.eval()
    tdcsfog_model_list4_3.append(model)

<br>

#### predict

In [206]:
%%time

# =========================
# tdcsfog4
# =========================
th_len = 5000
w = 0.40

cols = ["AccV","AccML","AccAP"]
num_cols = ['AccV', 'AccML', 'AccAP', 
            'AccV_lag_diff',  'AccV_lead_diff', 'AccV_cumsum', 
            'AccML_lag_diff', 'AccML_lead_diff', 'AccML_cumsum', 
            'AccAP_lag_diff', 'AccAP_lead_diff', 'AccAP_cumsum']

# tdcsfog_list = glob.glob(TDCSFOG_DATA_PATH)
# for p in tqdm(tdcsfog_list):
for p in tqdm(test_tdcsfog, desc="test_tdcsfog: "):
    # id_values = p.split("/")[-1].split(".")[0]
    id_values = p.split("/")[-1].split(".")[0].split("\\")[-1]
    df = pd.read_csv(p)
    
    if len(df) > th_len:
        seq_len = 5000
        shift = 2500
        offset = 1250
    else:
        seq_len = 3000
        shift = 1500
        offset = 750
        
    batch = (len(df)-1) // shift
    if batch == 0:
        batch = 1
    for c in cols:
        df[f"{c}_lag_diff"] = df[c].diff()
        df[f"{c}_lead_diff"] = df[c].diff(-1)
        df[f"{c}_cumsum"] = df[c].cumsum()
    sc = RobustScaler()
    df[num_cols] = sc.fit_transform(df[num_cols].values)
    df[num_cols] = df[num_cols].fillna(0)
    num = df[num_cols].values
    time = df["Time"].values
    
    num_array = np.zeros([batch,seq_len, 12])
    mask_array = np.zeros([batch,seq_len], dtype=int)
    time_array = np.zeros([batch,seq_len], dtype=int)
    pred_use_array = np.zeros([batch,seq_len], dtype=int)
    
    if len(df) <= seq_len:
        b = 0
        num_ = num.copy()
        time_ = time.copy()
        num_len = len(num_)

        num_array[b,:num_len,:] = num_
        time_array[b,:num_len] = time_
        mask_array[b,:num_len] = 1
        pred_use_array[b,:num_len] = 1
    else:
        for n,b in enumerate(range(batch)):
            if b == (batch - 1):
                num_ = num[b*shift : ]
                time_ = time[b*shift : ]
                num_len = len(num_)

                num_array[b, :num_len, :] = num_
                time_array[b,:num_len] = time_
                mask_array[b,:num_len] = 1
                pred_use_array[b, offset:num_len] = 1
            elif b == 0:
                num_ = num[b*shift : b*shift+seq_len]
                time_ = time[b*shift : b*shift + seq_len]

                num_array[b, :, :] = num_
                time_array[b, :] = time_
                mask_array[b, :] = 1
                pred_use_array[b, :shift+offset] = 1
            else:
                num_ = num[b*shift : b*shift+seq_len]
                time_ = time[b*shift : b*shift + seq_len]

                num_array[b, :, :] = num_
                time_array[b, :] = time_
                mask_array[b, :] = 1
                pred_use_array[b,offset : shift+offset] = 1
            
            

    test_ = FogDataset(num_array, mask_array, train=False)
    test_loader = DataLoader(dataset=test_, batch_size=bs, shuffle = False)
    for n,m in enumerate(tdcsfog_model_list4_1):
        if n == 0:
            pred1 = make_pred(test_loader,m) / len(tdcsfog_model_list4_1)
        else:
            pred1 += make_pred(test_loader,m) / len(tdcsfog_model_list4_1)
    for n,m in enumerate(tdcsfog_model_list4_2):
        if n == 0:
            pred2 = make_pred(test_loader,m) / len(tdcsfog_model_list4_2)
        else:
            pred2 += make_pred(test_loader,m) / len(tdcsfog_model_list4_2)
    for n,m in enumerate(tdcsfog_model_list4_3):
        if n == 0:
            pred3 = make_pred(test_loader,m) / len(tdcsfog_model_list4_3)
        else:
            pred3 += make_pred(test_loader,m) / len(tdcsfog_model_list4_3)
    pred = pred1.copy()
    pred[:,:,0] = pred1[:,:,0]*0.5 + pred2[:,:,0]*0.5
    pred[:,:,1] = pred1[:,:,1]*0.5 + pred3[:,:,1]*0.5
    pred[:,:,2] = pred2[:,:,2]*0.5 + pred3[:,:,2]*0.5
    
    
    pred_list = []
    for i in range(batch):
        mask_ = pred_use_array[i]
        pred_ = pred[i, mask_ == 1, :]
        time_ = time_array[i, mask_ == 1]
        
        df_ = pd.DataFrame()
        df_["StartHesitation"] = pred_[:, 0] * w
        df_["Turn"] = pred_[:, 1] * w
        df_["Walking"] = pred_[:, 2] * w
        df_["Time"] = time_
        df_["Id"] = id_values
        df_["Id"] = df_["Id"].astype(str) + "_" + df_["Time"].astype(str)
        
        pred_list.append(df_)
    pred = pd.concat(pred_list).reset_index(drop=True)
    df_all.append(pred)

test_tdcsfog: 100%|██████████| 1/1 [00:00<00:00,  2.73it/s]

CPU times: user 370 ms, sys: 0 ns, total: 370 ms
Wall time: 370 ms


<br>

## <font color=red>defog models</font>

### checkpoint paths

In [207]:
# defog_path2 = [f"/kaggle/input/fog-ex153/ex153_{i}.pth" for i in range(5)] # len 30000 defog1 
# defog_path4 = [f"/kaggle/input/fog-ex179/ex179_{i}.pth" for i in range(5)] # len 30000 defog1
# defog_path5 = [f"/kaggle/input/fog-ex185/ex185_{i}.pth" for i in range(5)] # len 30000 defog1
# defog_path6 = [f"/kaggle/input/fog-ex204/ex204_{i}.pth" for i in range(5)] # len 30000 defog2

# defog_path7 = [f"/kaggle/input/pd-exp238/fold{i}_best.pth" for i in [0, 1, 2, 3, 4]]  # len 30000 Defog3Model

In [208]:
defog_path2 = glob.glob("/kaggle/input/parkinson-fog-prediction/ex153*")
defog_path4 = glob.glob("/kaggle/input/parkinson-fog-prediction/ex179*")
defog_path5 = glob.glob("/kaggle/input/parkinson-fog-prediction/ex185*")
defog_path6 = glob.glob("/kaggle/input/parkinson-fog-prediction/ex204*")

# defog_path7 = glob.glob(".\\output\\exp\\ex238_\\ex238_\\*")

<br>

### **ex153** (defog2: 0.35)

#### load checkpoint

In [209]:
# DefogRnnModel()

defog_model_list2 = []
for i in defog_path2:         # ex153
    model = DefogRnnModel()
    model.load_state_dict(torch.load(i))
    model = model.to(device)
    model.eval()
    defog_model_list2.append(model)

<br>

#### predict

In [210]:
# for p in train_defog:
#     id_values = p.split("/")[-1].split(".")[0].split("\\")[-1]
#     print(id_values)

In [211]:
%%time

# =========================
# defog2
# =========================
th_len = 200000
w = 0.35

cols = ["AccV","AccML","AccAP"]
num_cols = ["AccV", "AccML", "AccAP",
            'AccV_lag_diff',  'AccV_lead_diff', 
            'AccML_lag_diff', 'AccML_lead_diff',
            'AccAP_lag_diff', 'AccAP_lead_diff']

# defog_list = glob.glob(DEFOG_DATA_PATH)
# for p in tqdm(defog_list):
for p in tqdm(test_defog, desc="test_defog: "):
    # id_values = p.split("/")[-1].split(".")[0]
    id_values = p.split("/")[-1].split(".")[0].split("\\")[-1]
    
    df = pd.read_csv(p)
    if len(df) > th_len:
        seq_len = 30000
        shift = 15000
        offset = 7500
    else:
        seq_len = 15000
        shift = 7500
        offset = 3750
    batch = (len(df)-1) // shift
    if batch == 0:
        batch = 1
    for c in cols:
        df[f"{c}_lag_diff"] = df[c].diff()
        df[f"{c}_lead_diff"] = df[c].diff(-1)
    sc = StandardScaler()
    df[num_cols] = sc.fit_transform(df[num_cols].values)
    df[num_cols] = df[num_cols].fillna(0)
    num = df[num_cols].values
    time = df["Time"].values
    
    num_array = np.zeros([batch,seq_len, 9])
    mask_array = np.zeros([batch,seq_len], dtype=int)
    time_array = np.zeros([batch,seq_len], dtype=int)
    pred_use_array = np.zeros([batch,seq_len], dtype=int)
    
    if len(df) <= seq_len:
        b = 0
        num_len = len(num)
        num_array[b,  :num_len, :] = num
        time_array[b, :num_len] = time
        mask_array[b, :num_len] = 1
        pred_use_array[b, :num_len] = 1
    else:
        for n,b in enumerate(range(batch)):
            if b == (batch - 1):
                num_ = num[b*shift : ]
                time_ = time[b*shift : ]
                num_len = len(num_)

                num_array[b,  :num_len, :] = num_
                time_array[b, :num_len] = time_
                mask_array[b, :num_len] = 1
                pred_use_array[b, offset:num_len] = 1
            elif b == 0:
                num_ = num[b*shift : b*shift+seq_len]
                time_ = time[b*shift : b*shift + seq_len]

                num_array[b,  :, :] = num_
                time_array[b, :] = time_
                mask_array[b, :] = 1
                pred_use_array[b,:shift+offset] = 1
            else:
                num_ = num[b*shift : b*shift+seq_len]
                time_ = time[b*shift : b*shift + seq_len]

                num_array[b, :, :] = num_
                time_array[b,:] = time_
                mask_array[b,:] = 1
                pred_use_array[b,offset:shift+offset] = 1  
    
    test_ = FogDataset(num_array, mask_array, train=False)
    test_loader = DataLoader(dataset=test_, batch_size=bs, shuffle = False)
    for n,m in enumerate(defog_model_list2):
        if n == 0:
            pred = make_pred(test_loader,m) / len(defog_model_list2)
        else:
            pred += make_pred(test_loader,m) / len(defog_model_list2)
    
    pred_list = []
    for i in range(batch):
        mask_ = pred_use_array[i]
        pred_ = pred[i,mask_ == 1,:]
        time_ = time_array[i, mask_ == 1]
        
        df_ = pd.DataFrame()
        df_["StartHesitation"] = pred_[:,0] * w
        df_["Turn"] = pred_[:,1] * w
        df_["Walking"] = pred_[:,2] * w
        df_["Time"] = time_
        df_["Id"] = id_values
        df_["Id"] = df_["Id"].astype(str) + "_" + df_["Time"].astype(str)
        
        pred_list.append(df_)
    pred = pd.concat(pred_list).reset_index(drop=True)
    df_all.append(pred)

test_defog: 100%|██████████| 1/1 [00:02<00:00,  2.30s/it]

CPU times: user 2.11 s, sys: 20.2 ms, total: 2.13 s
Wall time: 2.31 s


<br>

### **ex179** (defog4: 0.25)

#### load checkpoint

In [212]:
defog_model_list4 = []
for i in defog_path4:       # ex179
    model = DefogRnnModel()
    model.load_state_dict(torch.load(i))
    model = model.to(device)
    model.eval()
    defog_model_list4.append(model)

<br>

#### predict

In [213]:
%%time

# =========================
# defog4
# =========================
th_len = 200000
w = 0.25

cols = ["AccV","AccML","AccAP"]
num_cols = ["AccV", "AccML", "AccAP",
            'AccV_lag_diff',  'AccV_lead_diff', 
            'AccML_lag_diff', 'AccML_lead_diff',
            'AccAP_lag_diff', 'AccAP_lead_diff']

# defog_list = glob.glob(DEFOG_DATA_PATH)
# for p in tqdm(defog_list):
for p in tqdm(test_defog, desc="test_defog: "):
    # id_values = p.split("/")[-1].split(".")[0]
    id_values = p.split("/")[-1].split(".")[0].split("\\")[-1]
    
    df = pd.read_csv(p)
    if len(df) > th_len:
        seq_len = 30000
        shift = 15000
        offset = 7500
    else:
        seq_len = 15000
        shift = 7500
        offset = 3750
    batch = (len(df)-1) // shift
    if batch == 0:
        batch = 1
    for c in cols:
        df[f"{c}_lag_diff"] = df[c].diff()
        df[f"{c}_lead_diff"] = df[c].diff(-1)
    sc = StandardScaler()
    df[num_cols] = sc.fit_transform(df[num_cols].values)
    df[num_cols] = df[num_cols].fillna(0)
    num = df[num_cols].values
    time = df["Time"].values
    
    num_array = np.zeros([batch,seq_len, 9])
    mask_array = np.zeros([batch,seq_len], dtype=int)
    time_array = np.zeros([batch,seq_len], dtype=int)
    pred_use_array = np.zeros([batch,seq_len], dtype=int)
    
    if len(df) <= seq_len:
        b = 0
        num_len = len(num)
        num_array[b,  :num_len, :] = num
        time_array[b, :num_len] = time
        mask_array[b, :num_len] = 1
        pred_use_array[b, :num_len] = 1
    else:
        for n,b in enumerate(range(batch)):
            if b == (batch - 1):
                num_ = num[b*shift : ]
                time_ = time[b*shift : ]
                num_len = len(num_)

                num_array[b,  :num_len, :] = num_
                time_array[b, :num_len] = time_
                mask_array[b, :num_len] = 1
                pred_use_array[b, offset:num_len] = 1
            elif b == 0:
                num_ = num[b*shift : b*shift+seq_len]
                time_ = time[b*shift : b*shift + seq_len]

                num_array[b,  :, :] = num_
                time_array[b, :] = time_
                mask_array[b, :] = 1
                pred_use_array[b,:shift+offset] = 1
            else:
                num_ = num[b*shift : b*shift+seq_len]
                time_ = time[b*shift : b*shift + seq_len]

                num_array[b, :, :] = num_
                time_array[b,:] = time_
                mask_array[b,:] = 1
                pred_use_array[b,offset:shift+offset] = 1  
    
    test_ = FogDataset(num_array, mask_array, train=False)
    test_loader = DataLoader(dataset=test_, batch_size=bs, shuffle = False)
    for n,m in enumerate(defog_model_list4):
        if n == 0:
            pred = make_pred(test_loader,m) / len(defog_model_list4)
        else:
            pred += make_pred(test_loader,m) / len(defog_model_list4)
    
    pred_list = []
    for i in range(batch):
        mask_ = pred_use_array[i]
        pred_ = pred[i,mask_ == 1,:]
        time_ = time_array[i, mask_ == 1]
        
        df_ = pd.DataFrame()
        df_["StartHesitation"] = pred_[:,0] * w
        df_["Turn"] = pred_[:,1] * w
        df_["Walking"] = pred_[:,2] * w
        df_["Time"] = time_
        df_["Id"] = id_values
        df_["Id"] = df_["Id"].astype(str) + "_" + df_["Time"].astype(str)
        
        pred_list.append(df_)
    pred = pd.concat(pred_list).reset_index(drop=True)
    df_all.append(pred)

test_defog: 100%|██████████| 1/1 [00:02<00:00,  2.13s/it]

CPU times: user 2.11 s, sys: 9.9 ms, total: 2.12 s
Wall time: 2.13 s


<br>

### **ex185** (defog5: 0.25)

#### load checkpoint

In [214]:
defog_model_list5 = []
for i in defog_path5:        # ex185
    model = DefogRnnModel()
    model.load_state_dict(torch.load(i))
    model = model.to(device)
    model.eval()
    defog_model_list5.append(model)

<br>

#### predict

In [215]:
%%time

# =========================
# defog5
# =========================
th_len = 200000
w = 0.25

cols = ["AccV","AccML","AccAP"]
num_cols = ["AccV", "AccML", "AccAP",
            'AccV_lag_diff',  'AccV_lead_diff', 
            'AccML_lag_diff', 'AccML_lead_diff',
            'AccAP_lag_diff', 'AccAP_lead_diff']

# defog_list = glob.glob(DEFOG_DATA_PATH)
# for p in tqdm(defog_list):
for p in tqdm(test_defog, desc="test_defog: "):
    # id_values = p.split("/")[-1].split(".")[0]
    id_values = p.split("/")[-1].split(".")[0].split("\\")[-1]
    
    df = pd.read_csv(p)
    if len(df) > th_len:
        seq_len = 30000
        shift = 15000
        offset = 7500
    else:
        seq_len = 15000
        shift = 7500
        offset = 3750
    batch = (len(df)-1) // shift
    if batch == 0:
        batch = 1
    for c in cols:
        df[f"{c}_lag_diff"] = df[c].diff()
        df[f"{c}_lead_diff"] = df[c].diff(-1)
    sc = StandardScaler()
    df[num_cols] = sc.fit_transform(df[num_cols].values)
    df[num_cols] = df[num_cols].fillna(0)
    num = df[num_cols].values
    time = df["Time"].values
    
    num_array = np.zeros([batch,seq_len, 9])
    mask_array = np.zeros([batch,seq_len], dtype=int)
    time_array = np.zeros([batch,seq_len], dtype=int)
    pred_use_array = np.zeros([batch,seq_len], dtype=int)
    
    if len(df) <= seq_len:
        b = 0
        num_len = len(num)
        num_array[b,  :num_len, :] = num
        time_array[b, :num_len] = time
        mask_array[b, :num_len] = 1
        pred_use_array[b, :num_len] = 1
    else:
        for n,b in enumerate(range(batch)):
            if b == (batch - 1):
                num_ = num[b*shift : ]
                time_ = time[b*shift : ]
                num_len = len(num_)

                num_array[b,  :num_len, :] = num_
                time_array[b, :num_len] = time_
                mask_array[b, :num_len] = 1
                pred_use_array[b, offset:num_len] = 1
            elif b == 0:
                num_ = num[b*shift : b*shift+seq_len]
                time_ = time[b*shift : b*shift + seq_len]

                num_array[b,  :, :] = num_
                time_array[b, :] = time_
                mask_array[b, :] = 1
                pred_use_array[b,:shift+offset] = 1
            else:
                num_ = num[b*shift : b*shift+seq_len]
                time_ = time[b*shift : b*shift + seq_len]

                num_array[b, :, :] = num_
                time_array[b,:] = time_
                mask_array[b,:] = 1
                pred_use_array[b,offset:shift+offset] = 1  
    
    test_ = FogDataset(num_array, mask_array, train=False)
    test_loader = DataLoader(dataset=test_, batch_size=bs, shuffle = False)
    for n,m in enumerate(defog_model_list5):
        if n == 0:
            pred = make_pred(test_loader,m) / len(defog_model_list5)
        else:
            pred += make_pred(test_loader,m) / len(defog_model_list5)
    
    pred_list = []
    for i in range(batch):
        mask_ = pred_use_array[i]
        pred_ = pred[i,mask_ == 1,:]
        time_ = time_array[i, mask_ == 1]
        
        df_ = pd.DataFrame()
        df_["StartHesitation"] = pred_[:,0] * w
        df_["Turn"] = pred_[:,1] * w
        df_["Walking"] = pred_[:,2] * w
        df_["Time"] = time_
        df_["Id"] = id_values
        df_["Id"] = df_["Id"].astype(str) + "_" + df_["Time"].astype(str)
        
        pred_list.append(df_)
    pred = pd.concat(pred_list).reset_index(drop=True)
    df_all.append(pred)

test_defog: 100%|██████████| 1/1 [00:02<00:00,  2.11s/it]

CPU times: user 2.1 s, sys: 12.5 ms, total: 2.11 s
Wall time: 2.11 s


<br>

### **ex204** (defog6: 0.10)

#### load checkpoint

In [216]:
# DefogRnnModel2()

defog_model_list6 = []
for i in defog_path6:           # ex204
    model = DefogRnnModel2()
    model.load_state_dict(torch.load(i))
    model = model.to(device)
    model.eval()
    defog_model_list6.append(model)

<br>

#### predict

In [217]:
%%time


# =========================
# defog6
# =========================
th_len = 200000
# w = 0.10
w = 0.15

cols = ["AccV","AccML","AccAP"]
num_cols = ["AccV", "AccML", "AccAP",
            'AccV_lag_diff',  'AccV_lead_diff', 
            'AccML_lag_diff', 'AccML_lead_diff',
            'AccAP_lag_diff', 'AccAP_lead_diff']

# defog_list = glob.glob(DEFOG_DATA_PATH)
# for p in tqdm(defog_list):
for p in tqdm(test_defog, desc="test_defog: "):
    # id_values = p.split("/")[-1].split(".")[0]
    id_values = p.split("/")[-1].split(".")[0].split("\\")[-1]
    
    df = pd.read_csv(p)
    if len(df) > th_len:
        seq_len = 30000
        shift = 15000
        offset = 7500
    else:
        seq_len = 15000
        shift = 7500
        offset = 3750
    batch = (len(df)-1) // shift
    if batch == 0:
        batch = 1
    for c in cols:
        df[f"{c}_lag_diff"] = df[c].diff()
        df[f"{c}_lead_diff"] = df[c].diff(-1)
    sc = StandardScaler()
    df[num_cols] = sc.fit_transform(df[num_cols].values)
    df[num_cols] = df[num_cols].fillna(0)
    num = df[num_cols].values
    time = df["Time"].values
    
    num_array = np.zeros([batch,seq_len, 9])
    mask_array = np.zeros([batch,seq_len], dtype=int)
    time_array = np.zeros([batch,seq_len], dtype=int)
    pred_use_array = np.zeros([batch,seq_len], dtype=int)
    
    if len(df) <= seq_len:
        b = 0
        num_len = len(num)
        num_array[b,  :num_len, :] = num
        time_array[b, :num_len] = time
        mask_array[b, :num_len] = 1
        pred_use_array[b, :num_len] = 1
    else:
        for n,b in enumerate(range(batch)):
            if b == (batch - 1):
                num_ = num[b*shift : ]
                time_ = time[b*shift : ]
                num_len = len(num_)

                num_array[b,  :num_len, :] = num_
                time_array[b, :num_len] = time_
                mask_array[b, :num_len] = 1
                pred_use_array[b, offset:num_len] = 1
            elif b == 0:
                num_ = num[b*shift : b*shift+seq_len]
                time_ = time[b*shift : b*shift + seq_len]

                num_array[b,  :, :] = num_
                time_array[b, :] = time_
                mask_array[b, :] = 1
                pred_use_array[b,:shift+offset] = 1
            else:
                num_ = num[b*shift : b*shift+seq_len]
                time_ = time[b*shift : b*shift + seq_len]

                num_array[b, :, :] = num_
                time_array[b,:] = time_
                mask_array[b,:] = 1
                pred_use_array[b,offset:shift+offset] = 1  
    
    test_ = FogDataset(num_array, mask_array, train=False)
    test_loader = DataLoader(dataset=test_, batch_size=bs, shuffle = False)
    for n,m in enumerate(defog_model_list6):
        if n == 0:
            pred = make_pred(test_loader,m) / len(defog_model_list6)
        else:
            pred += make_pred(test_loader,m) / len(defog_model_list6)
    
    pred_list = []
    for i in range(batch):
        mask_ = pred_use_array[i]
        pred_ = pred[i,mask_ == 1,:]
        time_ = time_array[i, mask_ == 1]
        
        df_ = pd.DataFrame()
        df_["StartHesitation"] = pred_[:,0] * w
        df_["Turn"] = pred_[:,1] * w
        df_["Walking"] = pred_[:,2] * w
        df_["Time"] = time_
        df_["Id"] = id_values
        df_["Id"] = df_["Id"].astype(str) + "_" + df_["Time"].astype(str)
        
        pred_list.append(df_)
    pred = pd.concat(pred_list).reset_index(drop=True)
    df_all.append(pred)

test_defog: 100%|██████████| 1/1 [00:11<00:00, 11.06s/it]

CPU times: user 9.22 s, sys: 1.76 s, total: 11 s
Wall time: 11.1 s


<br>

### **ex238** (defog7: 0.05)

#### load checkpoint

In [218]:
# # Defog3Model()

# defog_model_list7 = []
# for path in defog_path7:    # ex238
#     model = Defog3Model()
#     state = torch.load(path, map_location=torch.device("cpu"))
#     model.load_state_dict(state["model"])
#     model = model.to(device)
#     model.eval()
#     defog_model_list7.append(model)
#     print(f"load weights from {path}")

<br>

#### predict

In [219]:
# %%time

# # =========================
# # defog7
# # =========================
# th_len = 200000
# w = 0.05

# cols = ["AccV","AccML","AccAP"]
# num_cols = ["AccV", "AccML", "AccAP",
#             'AccV_lag_diff',  'AccV_lead_diff', 
#             'AccML_lag_diff', 'AccML_lead_diff',
#             'AccAP_lag_diff', 'AccAP_lead_diff']

# # defog_list = glob.glob(DEFOG_DATA_PATH)
# # for p in tqdm(defog_list):
# for p in tqdm(test_defog, desc="test_defog: "):
#     # id_values = p.split("/")[-1].split(".")[0]
#     id_values = p.split("/")[-1].split(".")[0].split("\\")[-1]
    
#     df = pd.read_csv(p)
#     if len(df) > th_len:
#         seq_len = 30000
#         shift = 15000
#         offset = 7500
#     else:
#         seq_len = 15000
#         shift = 7500
#         offset = 3750
#     batch = (len(df)-1) // shift
#     if batch == 0:
#         batch = 1
#     for c in cols:
#         df[f"{c}_lag_diff"] = df[c].diff()
#         df[f"{c}_lead_diff"] = df[c].diff(-1)
#     sc = StandardScaler()
#     df[num_cols] = sc.fit_transform(df[num_cols].values)
#     df[num_cols] = df[num_cols].fillna(0)
#     num = df[num_cols].values
#     time = df["Time"].values
    
#     num_array = np.zeros([batch,seq_len, 9])
#     mask_array = np.zeros([batch,seq_len], dtype=int)
#     time_array = np.zeros([batch,seq_len], dtype=int)
#     pred_use_array = np.zeros([batch,seq_len], dtype=int)
    
#     if len(df) <= seq_len:
#         b = 0
#         num_len = len(num)
#         num_array[b,  :num_len, :] = num
#         time_array[b, :num_len] = time
#         mask_array[b, :num_len] = 1
#         pred_use_array[b, :num_len] = 1
#     else:
#         for n,b in enumerate(range(batch)):
#             if b == (batch - 1):
#                 num_ = num[b*shift : ]
#                 time_ = time[b*shift : ]
#                 num_len = len(num_)

#                 num_array[b,  :num_len, :] = num_
#                 time_array[b, :num_len] = time_
#                 mask_array[b, :num_len] = 1
#                 pred_use_array[b, offset:num_len] = 1
#             elif b == 0:
#                 num_ = num[b*shift : b*shift+seq_len]
#                 time_ = time[b*shift : b*shift + seq_len]

#                 num_array[b,  :, :] = num_
#                 time_array[b, :] = time_
#                 mask_array[b, :] = 1
#                 pred_use_array[b,:shift+offset] = 1
#             else:
#                 num_ = num[b*shift : b*shift+seq_len]
#                 time_ = time[b*shift : b*shift + seq_len]

#                 num_array[b, :, :] = num_
#                 time_array[b,:] = time_
#                 mask_array[b,:] = 1
#                 pred_use_array[b,offset:shift+offset] = 1  
    
#     test_ = FogDataset(num_array, mask_array, train=False)
#     test_loader = DataLoader(dataset=test_, batch_size=bs, shuffle = False)
#     for n,m in enumerate(defog_model_list7):
#         if n == 0:
#             pred = make_pred(test_loader,m) / len(defog_model_list7)
#         else:
#             pred += make_pred(test_loader,m) / len(defog_model_list7)
    
#     pred_list = []
#     for i in range(batch):
#         mask_ = pred_use_array[i]
#         pred_ = pred[i,mask_ == 1,:]
#         time_ = time_array[i, mask_ == 1]
        
#         df_ = pd.DataFrame()
#         df_["StartHesitation"] = pred_[:,0] * w
#         df_["Turn"] = pred_[:,1] * w
#         df_["Walking"] = pred_[:,2] * w
#         df_["Time"] = time_
#         df_["Id"] = id_values
#         df_["Id"] = df_["Id"].astype(str) + "_" + df_["Time"].astype(str)
        
#         pred_list.append(df_)
#     pred = pd.concat(pred_list).reset_index(drop=True)
#     df_all.append(pred)

<br>
<br>


# **Submission** 

In [220]:
df_all = pd.concat(df_all).reset_index(drop=True)
df_all = df_all.groupby(by="Id")[['StartHesitation', 'Turn', 'Walking']].sum().reset_index()
df_all[['Id', 'StartHesitation', 'Turn', 'Walking']].to_csv("submission.csv",index=False)

In [221]:
df_all

,Id,StartHesitation,Turn,Walking
0,003f117e14_0,0.002476,0.002984,0.000762
1,003f117e14_1,0.002400,0.002903,0.000708
2,003f117e14_10,0.001943,0.002561,0.000587
3,003f117e14_100,0.001559,0.002188,0.000472
4,003f117e14_1000,0.002783,0.042777,0.000950
...,...,...,...,...
286365,02ab235146_99995,0.000067,0.033726,0.004219
286366,02ab235146_99996,0.000066,0.032923,0.004129
286367,02ab235146_99997,0.000065,0.032285,0.004020
286368,02ab235146_99998,0.000064,0.031890,0.003912
